In [ ]:
!pip install pytest cython tensorflow numba sympy pandas scikit-learn click deap pathos seaborn progress tqdm commentjson PyYAML prettytable


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9614 sha256=53cc52ae141fec1dab25eff12bcdf7e8d8d7d406aba2a8b304628bd36a233063
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12074 sha256=cef050bfa4a9abdefeb7401472000ea44

In [ ]:
import sympy.parsing.sympy_parser as sympy_parser
import sympy

from typing import Callable

In [ ]:
import collections
import copy
import functools
import numpy as np
import time
import importlib
import random
import re
import os
import pandas as pd

import sympy.parsing.sympy_parser as sympy_parser
import sympy

from typing import Callable

In [ ]:
#utilsvs
def preserve_global_rng_state(f: Callable):
    """
    Decorator that saves the internal state of the global random number
    generator before call to function and sets it back to that state
    after the call

    Parameters
    ----------
    f : Callable
        Function to decorate

    Returns
    _______
    Callable
        Decorated function that saves global random state and resets to it after
    """
    @functools.wraps(f)
    def decorated(*args, **kwargs):
        rng_state = random.getstate()
        result = f(*args, **kwargs)
        random.setstate(rng_state)
        return result
    return decorated


# We wrap the sympy functions with preserve_global_rng_state
# as the sympy routines appear to non-deterministically
# re-seed the global random generator which can influence GP results.
# This problem seems to be resolved in sympy in commit
# https://github.com/sympy/sympy/pull/22433
# These functions should be used instead of the sympy functions directly
pretty = preserve_global_rng_state(sympy.pretty)
parse_expr = preserve_global_rng_state(sympy_parser.parse_expr)


def is_float(s):
    """Determine whether the input variable can be cast to float."""

    try:
        float(s)
        return True
    except ValueError:
        return False


# Adapted from: https://stackoverflow.com/questions/32791911/fast-calculation-of-pareto-front-in-python
def is_pareto_efficient(costs):
    """
    Find the pareto-efficient points given an array of costs.

    Parameters
    ----------

    costs : np.ndarray
        Array of shape (n_points, n_costs).

    Returns
    -------

    is_efficient_maek : np.ndarray (dtype:bool)
        Array of which elements in costs are pareto-efficient.
    """

    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    is_efficient_mask = np.zeros(n_points, dtype=bool)
    is_efficient_mask[is_efficient] = True
    return is_efficient_mask


class cached_property(object):
    """
    Decorator used for lazy evaluation of an object attribute. The property
    should be non-mutable, since it replaces itself.
    """

    def __init__(self, getter):
        self.getter = getter

        functools.update_wrapper(self, getter)

    def __get__(self, obj, cls):
        if obj is None:
            return self

        value = self.getter(obj)
        setattr(obj, self.getter.__name__, value)
        return value


def weighted_quantile(values, weights, q):
    """
    Computes the weighted quantile, equivalent to the exact quantile of the
    empirical distribution.

    Given ordered samples x_1 <= ... <= x_n, with corresponding weights w_1,
    ..., w_n, where sum_i(w_i) = 1.0, the weighted quantile is the minimum x_i
    for which the cumulative sum up to x_i is greater than or equal to 1.

    Quantile = min{ x_i | x_1 + ... + x_i >= q }
    """

    sorted_indices = np.argsort(values)
    sorted_weights = weights[sorted_indices]
    sorted_values = values[sorted_indices]
    cum_sorted_weights = np.cumsum(sorted_weights)
    i_quantile = np.argmax(cum_sorted_weights >= q)
    quantile = sorted_values[i_quantile]

    # NOTE: This implementation is equivalent to (but much faster than) the
    # following:
    # from scipy import stats
    # empirical_dist = stats.rv_discrete(name='empirical_dist', values=(values, weights))
    # quantile = empirical_dist.ppf(q)

    return quantile


# Entropy computation in batch
def empirical_entropy(labels):

    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    value,counts = np.unique(labels, return_counts=True)
    probs = counts / n_labels
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.
    # Compute entropy
    for i in probs:
        ent -= i * np.log(i)

    return np.array(ent, dtype=np.float32)


def get_duration(start_time):
    return get_human_readable_time(time.time() - start_time)


def get_human_readable_time(s):
    m, s = divmod(s, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)
    return "{:02d}:{:02d}:{:02d}:{:05.2f}".format(int(d), int(h), int(m), s)


def safe_merge_dicts(base_dict, update_dict):
    """Merges two dictionaries without changing the source dictionaries.

    Parameters
    ----------
        base_dict : dict
            Source dictionary with initial values.
        update_dict : dict
            Dictionary with changed values to update the base dictionary.

    Returns
    -------
        new_dict : dict
            Dictionary containing values from the merged dictionaries.
    """
    if base_dict is None:
        return update_dict
    base_dict = copy.deepcopy(base_dict)
    for key, value in update_dict.items():
        if isinstance(value, collections.Mapping):
            base_dict[key] = safe_merge_dicts(base_dict.get(key, {}), value)
        else:
            base_dict[key] = value
    return base_dict


def safe_update_summary(csv_path, new_data):
    """Updates a summary csv file with new rows. Adds new columns
    in existing data if necessary. New rows are distinguished by
    the run seed.

    Parameters
    ----------
        csv_path : str
            String with the path to the csv file.
        new_data : dict
            Dictionary containing values to be saved in the csv file.

    Returns
    -------
        bool
            Boolean value to indicate if saving the data to file worked.
    """
    try:
        new_data_pd = pd.DataFrame(new_data, index=[0])
        new_data_pd.set_index('seed', inplace=True)
        if os.path.isfile(csv_path):
            old_data_pd = pd.read_csv(csv_path)
            old_data_pd.set_index('seed', inplace=True)
            merged_df = pd.concat([old_data_pd, new_data_pd], axis=0, ignore_index=False)
            merged_df.to_csv(csv_path, header=True, mode='w+', index=True)
        else:
            new_data_pd.to_csv(csv_path, header=True, mode='w+', index=True)
        return True
    except:
        return False


def import_custom_source(import_source):
    """
    Provides a way to import custom modules. The return will be a reference to the desired source
    Parameters
    ----------
        import_source : import path
            Source to import from, for most purposes: <module_name>:<class or function name>

    Returns
    -------
        mod : ref
            reference to the imported module
    """

    # Partially validates if the import_source is in correct format
    regex = '[\w._]+:[\w._]+' #lib_name:class_name
    m = re.match(pattern=regex, string=import_source)
    # Partial matches mean that the import will fail
    assert m is not None and m.end() == len(import_source), "*** Failed to import malformed source string: "+import_source

    source, type = import_source.split(':')

    # Dynamically imports the configured source
    mod = importlib.import_module(source)
    func = getattr(mod, type)

    return func

def pad_action_obs_priors(actions, obs, priors, pad_length):
    """
    Will pad action, obs, priors with zeros.

    Parameters
    ----------
        actions : np array
            Standard actions array of tokens
        obs : np array
            Standard observations array
        priors : np array
            Standard priors array
        pdd_length : int

    Returns
    -------
        actions : np array
            Standard actions array of tokens padded with zeros at the end columns
        obs : np array
            Standard observations array padded with zeros at the end columns
        priors : np array
            Standard priors array padded with zeros at the end columns
    """
    assert isinstance(pad_length,int)
    assert pad_length >= 0

    actions = np.pad(actions, ((0,0),(0,pad_length)), 'constant', constant_values=((0,0),(0,0)))
    obs = [ np.pad(o, ((0,0),(0,pad_length)), 'constant', constant_values=((0,0),(0,0))) for o in obs ]
    priors = np.pad(priors, ((0,0),(0,pad_length),(0,0)), 'constant', constant_values=((0,0),(0,0),(0,0)))

    return actions, obs, priors


def make_batch_ph(name : str, n_choices : int):
    """
    Generates dictionary containing placeholders needed for a batch of sequences.

    Parameters
    ----------
        names : str
            Name of tensorflow scope for this batch

        n_choices : int
            Number of choices in priors

    Returns
    -------
        batch_ph : dict
            Dictionary of placeholders
    """

    # Lazy import
    import tensorflow as tf
    from dso.memory import Batch
    from dso.program import Program

    with tf.name_scope(name):
        batch_ph = {
            "actions": tf.placeholder(tf.int32, [None, None]),
            "obs": tf.placeholder(tf.float32, [None, Program.task.OBS_DIM, None]),
            "priors": tf.placeholder(tf.float32, [None, None, n_choices]),
            "lengths": tf.placeholder(tf.int32, [None, ]),
            "rewards": tf.placeholder(tf.float32, [None], name="r"),
            "on_policy": tf.placeholder(tf.int32, [None, ])
         }
        batch_ph = Batch(**batch_ph)
    return batch_ph

In [ ]:
from collections import defaultdict

In [ ]:
#library
class Token():
    """
    An arbitrary token or "building block" of a Program object.

    Attributes
    ----------
    name : str
        Name of token.

    arity : int
        Arity (number of arguments) of token.

    complexity : float
        Complexity of token.

    function : callable
        Function associated with the token; used for exectuable Programs.

    input_var : int or None
        Index of input if this Token is an input variable, otherwise None.

    Methods
    -------
    __call__(input)
        Call the Token's function according to input.
    """

    def __init__(self, function, name, arity, complexity, input_var=None):
        self.function = function
        self.name = name
        self.arity = arity
        self.complexity = complexity
        self.input_var = input_var

        if input_var is not None:
            assert function is None, "Input variables should not have functions."
            assert arity == 0, "Input variables should have arity zero."

    def __call__(self, *args):
        assert self.function is not None, \
            "Token {} is not callable.".format(self.name)

        return self.function(*args)

    def __repr__(self):
        return self.name


class HardCodedConstant(Token):
    """
    A Token with a "value" attribute, whose function returns the value.

    Parameters
    ----------
    value : float
        Value of the constant.
    """

    def __init__(self, value=None, name=None):
        assert value is not None, "Constant is not callable with value None. Must provide a floating point number or string of a float."
        assert U.is_float(value)
        value = np.atleast_1d(np.float32(value))
        self.value = value
        if name is None:
            name = str(self.value[0])
        super().__init__(function=self.function, name=name, arity=0, complexity=1)

    def function(self):
        return self.value


class PlaceholderConstant(Token):
    """
    A Token for placeholder constants that will be optimized with respect to
    the reward function. The function simply returns the "value" attribute.

    Parameters
    ----------
    value : float or None
        Current value of the constant, or None if not yet set.
    """

    def __init__(self, value=None):
        if value is not None:
            value = np.atleast_1d(value)
        self.value = value
        super().__init__(function=self.function, name="const", arity=0, complexity=1)

    def function(self):
        assert self.value is not None, \
            "Constant is not callable with value None."
        return self.value

    def __repr__(self):
        if self.value is None:
            return self.name
        return str(self.value[0])


class Polynomial(Token):
    """
    A Token representing a polynomial of the input variables of the form:
        p(x1, ..., xn) = c1 * x1**e11 * ... * xn**e1n + ... + cm * x1**em1 * ... * xn**emn.
    Note that only the terms with a nonzero coefficient are stored.

    Parameters
    ----------
    exponents : list of tuples of nonnegative integers
        Exponents of the nonzero terms [(e11, ..., e1n), ..., (em1, ..., emn)].
    coef : list of float
        A list of coefficients [c1, c2, ..., cm] corresponding to the terms in exponents.
    """
    def __init__(self, exponents=None, coef=None):
        self.exponents = exponents
        self.coef = coef
        complexity = 1 if coef is None else len(coef)
        super().__init__(self.eval_poly, "poly", arity=0, complexity=complexity)

    @staticmethod
    def eval_monomials(X, monomials_exponents):
        """
        Compute the monomials x1**ej1 * ... * xn**ejn for each data point [x1, ..., xn] in X.
        Parameters
        ----------
        X : ndarray
            A dataset of shape (number of points, number of variables).
        monomials_exponents : list of tuples of nonnegative integers
            Exponents of the monomials to be computed: [(e11, ..., e1n), ..., (em1, ..., emn)].
        """
        monomials = np.ones(shape=(X.shape[0], len(monomials_exponents)))
        for basis_count, exponents in enumerate(monomials_exponents):
            for i in range(len(exponents)):
                if exponents[i] != 0:
                    monomials[:, basis_count] *= X[:, i] ** exponents[i]
        return monomials

    def eval_poly(self, X):
        assert self.exponents is not None and self.coef is not None
        if len(self.coef) == 0:
            return np.zeros(shape=(X.shape[0], 1))
        return np.dot(self.eval_monomials(X, self.exponents), self.coef)

    def __repr__(self):
        if self.exponents is None and self.coef is None:
            return self.name

        if len(self.coef) == 0:
            return "0.0"

        assert len(self.exponents) == self.coef.shape[0]
        names = ["-" if self.coef[0] < 0 else ""]
        for basis_count, exponents in enumerate(self.exponents):
            basis_name = [str(format(np.abs(self.coef[basis_count]), '.6'))]
            for i in range(len(exponents)):
                if exponents[i] == 1:
                    basis_name.append("x{}".format(i + 1))
                elif exponents[i] > 1:
                    basis_name.append("x{}**{}".format(i + 1, int(exponents[i])))
            names.append("*".join(basis_name))
            if basis_count < len(self.coef) - 1:
                names.append("-" if self.coef[basis_count + 1] < 0 else "+")
        return "".join(names)

    def to_str_tokens(self):
        """
        Return a list of tokens of add, mul, inputs, and constants that
        is equivalent to this poly token.
        """
        if self.exponents is None and self.coef is None:
            return []

        assert len(self.exponents) == self.coef.shape[0]
        out = [] if len(self.coef) == 0 else ["add"]*(len(self.coef)-1)
        for n, exponents in enumerate(self.exponents):
            out.extend(["mul"]*np.count_nonzero(exponents))
            for i in range(len(exponents)):
                if exponents[i] >= 1:
                    out.extend(["mul"]*(exponents[i]-1))
                    out.extend(["x{}".format(i + 1)]*exponents[i])
            out.append(self.coef[n])
        return out


class StateChecker(Token):
    """
    A Token for making decisions in decision trees. Given the i-th state
    variable xi and a threshold tj, the associated function is:
        xi_<_tj(f, g) = f if xi < tj else g.
    Here, the index i is indicated by state_index in __init__.

    Note that:
        1. The arity of StateChecker is 2, so subroutines designed for Tokens
           of arity <= 2 can also be applied to StateChecker.
        2. If StateCheckers are included in the library, it is recommended to
           turn on the StateCheckerConstraint, which prevents sampling of
           Tokens that lead to degenerate situations like "checking if xi < 6
           when we know xi < 3".
        3. When a StateChecker is initialized, state_value (the value of xi)
           is unknown. In order to evaluate the return value of the associated
           function, state_value needs to be set before the evaluation
           (typically during the execution of a Program). See cyfunc.pyx for
           an example usage.

    Parameters
    ----------
    state_index : int
        Index of the state variable associated with the token.

    threshold : float
        Value to which the state variable is compared when making decisions.
    """

    def __init__(self, state_index, threshold):
        assert threshold is not None, \
                "StateChecker requires a float value for threshold."
        self.state_index = state_index
        self.state_value = None
        self.threshold = threshold

        name = "x{} < {}".format(state_index + 1, self.threshold)
        super().__init__(function=self.function, name=name, arity=2, complexity=1)

    def set_state_value(self, state_value):
        self.state_value = state_value

    def function(self, value_if_true, value_if_false):
        assert self.state_value is not None, "StateChecker.state_value has not been set."
        return np.where(np.less(self.state_value, self.threshold), value_if_true, value_if_false)


class DiscreteAction(HardCodedConstant):
    """
    This class is intended to be used for learning decision tree policies when
    the env of the control problem has a Discrete action space.
    Discrete action a_i corresponds to constant value i-1, i = 1, 2, 3, ...
    """
    def __init__(self, value):
        assert isinstance(value, int) and value >= 0
        super().__init__(value, "a_{}".format(value+1))


class MultiDiscreteAction(Token):
    """
    This class is intended to be used for learning decision tree policies when
    the env of the control problem has a MultiDiscrete action space.

    Tokens in this class are ai_j (unary) and STOP (terminal). When executed:
        - ai_j returns an array that sets the (i-1)-th value of input to j-1.
        - STOP returns an array of default actions of all dimensions.
    For example, supposed the default actions are set to be [2, 1, 0].
    The traversal [a1_2, STOP] corresponds to the constant action [1, 1, 0],
    while the traversal [a1_3, a3_3, a2_1, STOP] corresponds to [2, 0, 2].
    """
    n_dims = None # total number of action dimensions
    def __init__(self, value, action_dim=None):
        """
        Parameters
        ----------
        value : int or list of int
            If action_dim is not None, the token corresponds to the value-th
            discrete action in action dimension action_dim.
            Otherwise, it is a list of integers specifying the default discrete
            actions of all action dimensions.

        action_dim : int or None
            Action dimension. If None, the STOP token will be constructed.
        """
        self.value = value
        self.action_dim = action_dim

        if action_dim is None:
            assert isinstance(self.value, list)
            MultiDiscreteAction.n_dims = len(self.value)
            self.value = np.array(self.value)
            name = "STOP"
            super().__init__(function=self.apply_action, name=name, arity=0, complexity=1)
        else:
            assert isinstance(value, int) and value >= 0
            name = "a{}_{}".format(action_dim+1, value+1)
            super().__init__(function=self.apply_action, name=name, arity=1, complexity=1)

    def apply_action(self, *args):
        if self.action_dim is None:
            return np.array([self.value.copy()])
        else:
            args[0][0, self.action_dim] = self.value
            return args[0]


class Library():
    """
    Library of Tokens. We use a list of Tokens (instead of set or dict) since
    we so often index by integers given by the Controller.

    Attributes
    ----------
    tokens : list of Token
        List of available Tokens in the library.

    names : list of str
        Names corresponding to Tokens in the library.

    arities : list of int
        Arities corresponding to Tokens in the library.
    """

    def __init__(self, tokens):

        self.tokens = tokens
        self.L = len(tokens)
        self.names = [t.name for t in tokens]
        self.arities = np.array([t.arity for t in tokens], dtype=np.int32)

        self.input_tokens = np.array(
            [i for i, t in enumerate(self.tokens) if t.input_var is not None],
            dtype=np.int32)

        self.state_checker_tokens = np.array(
            [i for i, t in enumerate(self.tokens) if isinstance(t, StateChecker)],
            dtype=np.int32)

        self.multi_discrete_tokens = np.array(
            [i for i, t in enumerate(self.tokens) if isinstance(t, MultiDiscreteAction)],
            dtype=np.int32)

        def get_tokens_of_arity(arity):
            _tokens = [i for i in range(self.L) if self.arities[i] == arity]
            return np.array(_tokens, dtype=np.int32)

        self.tokens_of_arity = defaultdict(lambda : np.array([], dtype=np.int32))
        for arity in self.arities:
            self.tokens_of_arity[arity] = get_tokens_of_arity(arity)
        self.terminal_tokens = self.tokens_of_arity[0]
        self.unary_tokens = self.tokens_of_arity[1]
        self.binary_tokens = self.tokens_of_arity[2]

        try:
            self.const_token = self.names.index("const")
        except ValueError:
            self.const_token = None
        try:
            self.poly_token = self.names.index("poly")
        except ValueError:
            self.poly_token = None
        self.parent_adjust = np.full_like(self.arities, -1)
        count = 0
        for i in range(len(self.arities)):
            if self.arities[i] > 0:
                self.parent_adjust[i] = count
                count += 1

        trig_names = ["sin", "cos", "tan", "csc", "sec", "cot"]
        trig_names += ["arc" + name for name in trig_names]

        self.float_tokens = np.array(
            [i for i, t in enumerate(self.tokens) if t.arity == 0 and t.input_var is None],
            dtype=np.int32)
        self.trig_tokens = np.array(
            [i for i, t in enumerate(self.tokens) if t.name in trig_names],
            dtype=np.int32)

        inverse_tokens = {
            "inv" : "inv",
            "neg" : "neg",
            "exp" : "log",
            "log" : "exp",
            "sqrt" : "n2",
            "n2" : "sqrt"
        }
        token_from_name = {t.name : i for i, t in enumerate(self.tokens)}
        self.inverse_tokens = {token_from_name[k] : token_from_name[v] for k, v in inverse_tokens.items() if k in token_from_name and v in token_from_name}

        self.n_action_inputs = self.L + 1 # Library tokens + empty token
        self.n_parent_inputs = self.L + 1 - len(self.terminal_tokens) # Parent sub-lib tokens + empty token
        self.n_sibling_inputs = self.L + 1 # Library tokens + empty token
        self.n_input_tokens = len(self.input_tokens)
        self.EMPTY_ACTION = self.n_action_inputs - 1
        self.EMPTY_PARENT = self.n_parent_inputs - 1
        self.EMPTY_SIBLING = self.n_sibling_inputs - 1

    def __getitem__(self, val):
        """Shortcut to get Token by name or index."""

        if isinstance(val, str):
            try:
                i = self.names.index(val)
            except ValueError:
                raise TokenNotFoundError("Token {} does not exist.".format(val))
        elif isinstance(val, (int, np.integer)):
            i = val
        else:
            raise TokenNotFoundError("Library must be indexed by str or int, not {}.".format(type(val)))

        try:
            token = self.tokens[i]
        except IndexError:
            raise TokenNotFoundError("Token index {} does not exist".format(i))
        return token

    def tokenize(self, inputs):
        """Convert inputs to list of Tokens."""

        # TBD non-list should return non-list

        if isinstance(inputs, str):
            inputs = inputs.split(',')
        elif not isinstance(inputs, list) and not isinstance(inputs, np.ndarray): # TBD FIX HACK
            inputs = [inputs]
        tokens = [input_ if isinstance(input_, Token) else self[input_] for input_ in inputs]
        return tokens

    def actionize(self, inputs):
        """Convert inputs to array of 'actions', i.e. ints corresponding to
        Tokens in the Library."""

        tokens = self.tokenize(inputs)
        actions = np.array([self.tokens.index(t) for t in tokens],
                           dtype=np.int32)
        return actions


class TokenNotFoundError(Exception):
    pass

In [ ]:
from functools import partial

import numpy as np
from scipy.optimize import minimize

In [ ]:
#const
def make_const_optimizer(name, **kwargs):
    """Returns a ConstOptimizer given a name and keyword arguments"""

    const_optimizers = {
        None : Dummy,
        "dummy" : Dummy,
        "scipy" : ScipyMinimize,
    }

    return const_optimizers[name](**kwargs)


class ConstOptimizer(object):
    """Base class for constant optimizer"""

    def __init__(self, **kwargs):
        self.kwargs = kwargs


    def __call__(self, f, x0):
        """
        Optimizes an objective function from an initial guess.

        The objective function is the negative of the base reward (reward
        without penalty) used for training. Optimization excludes any penalties
        because they are constant w.r.t. to the constants being optimized.

        Parameters
        ----------
        f : function mapping np.ndarray to float
            Objective function (negative base reward).

        x0 : np.ndarray
            Initial guess for constant placeholders.

        Returns
        -------
        x : np.ndarray
            Vector of optimized constants.
        """
        raise NotImplementedError


class Dummy(ConstOptimizer):
    """Dummy class that selects the initial guess for each constant"""

    def __init__(self, **kwargs):
        super(Dummy, self).__init__(**kwargs)


    def __call__(self, f, x0):
        return x0


class ScipyMinimize(ConstOptimizer):
    """SciPy's non-linear optimizer"""

    def __init__(self, **kwargs):
        super(ScipyMinimize, self).__init__(**kwargs)


    def __call__(self, f, x0):
        with np.errstate(divide='ignore'):
            opt_result = partial(minimize, **self.kwargs)(f, x0)
        x = opt_result['x']
        return x

In [ ]:
import warnings
from textwrap import indent

In [ ]:
#program
def _finish_tokens(tokens):

    """
    Complete a possibly unfinished string of tokens.

    Parameters
    ----------
    tokens : list of integers
        A list of integers corresponding to tokens in the library. The list
        defines an expression's pre-order traversal.

    Returns
    _______
    tokens : list of ints
        A list of integers corresponding to tokens in the library. The list
        defines an expression's pre-order traversal. "Dangling" programs are
        completed with repeated "x1" until the expression completes.

    """

    if Program.task.task_type == "binding":
        return tokens

    arities = np.array([Program.library.arities[t] for t in tokens])
    # Number of dangling nodes, returns the cumsum up to each point
    # Note that terminal nodes are -1 while functions will be >= 0 since arities - 1
    dangling = 1 + np.cumsum(arities - 1)

    if -1 in (dangling - 1):
        # chop off tokens once the cumsum reaches 0, this is the last valid point in the tokens
        expr_length = 1 + np.argmax((dangling - 1) == -1)
        tokens = tokens[:expr_length]
    else:
        # Extend with valid variables until string is valid
        # NOTE: This only appends onto the end of a set of tokens, even in the multi-object case!
        if Program.task.task_type != 'binding':
            tokens = np.append(tokens, np.random.choice(Program.library.input_tokens, size=dangling[-1]))

    return tokens


def from_str_tokens(str_tokens, skip_cache=False):
    """
    Memoized function to generate a Program from a list of str and/or float.
    See from_tokens() for details.

    Parameters
    ----------
    str_tokens : str | list of (str | float)
        Either a comma-separated string of tokens and/or floats, or a list of
        str and/or floats.

    skip_cache : bool
        See from_tokens().

    Returns
    -------
    program : Program
        See from_tokens().
    """

    # Convert str to list of str
    if isinstance(str_tokens, str):
        str_tokens = str_tokens.split(",")

    # Convert list of str|float to list of tokens
    if isinstance(str_tokens, list):
        traversal = []
        constants = []
        for s in str_tokens:
            if s in Program.library.names:
                t = Program.library.names.index(s.lower())
            elif U.is_float(s):
                assert "const" not in str_tokens, "Currently does not support both placeholder and hard-coded constants."
                t = Program.library.const_token
                constants.append(float(s))
            else:
                raise ValueError("Did not recognize token {}.".format(s))
            traversal.append(t)
        traversal = np.array(traversal, dtype=np.int32)
    else:
        raise ValueError("Input must be list or string.")

    # Generate base Program (with "const" for constants)
    p = from_tokens(traversal, skip_cache=skip_cache)

    # Replace any constants
    p.set_constants(constants)

    return p


def from_tokens(tokens, skip_cache=False, on_policy=True, finish_tokens=True):

    """
    Memoized function to generate a Program from a list of tokens.

    Since some tokens are nonfunctional, this first computes the corresponding
    traversal. If that traversal exists in the cache, the corresponding Program
    is returned. Otherwise, a new Program is returned.

    Parameters
    ----------
    tokens : list of integers
        A list of integers corresponding to tokens in the library. The list
        defines an expression's pre-order traversal. "Dangling" programs are
        completed with repeated "x1" until the expression completes.

    skip_cache : bool
        Whether to bypass the cache when creating the program (used for
        previously learned symbolic actions in DSP).

    finish_tokens: bool
        Do we need to finish this token. There are instances where we have
        already done this. Most likely you will want this to be True.

    Returns
    _______
    program : Program
        The Program corresponding to the tokens, either pulled from memoization
        or generated from scratch.
    """

    '''
        Truncate expressions that complete early; extend ones that don't complete
    '''

    if finish_tokens:
        tokens = _finish_tokens(tokens)

    # For stochastic Tasks, there is no cache; always generate a new Program.
    # For deterministic Programs, if the Program is in the cache, return it;
    # otherwise, create a new one and add it to the cache.
    if skip_cache or Program.task.stochastic:
        p = Program(tokens, on_policy=on_policy)
    else:
        key = tokens.tostring()
        try:
            p = Program.cache[key]
            if on_policy:
                p.on_policy_count += 1
            else:
                p.off_policy_count += 1
        except KeyError:
            p = Program(tokens, on_policy=on_policy)
            Program.cache[key] = p

    return p


class Program(object):
    """
    The executable program representing the symbolic expression.

    The program comprises unary/binary operators, constant placeholders
    (to-be-optimized), input variables, and hard-coded constants.

    Parameters
    ----------
    tokens : list of integers
        A list of integers corresponding to tokens in the library. "Dangling"
        programs are completed with repeated "x1" until the expression
        completes.

    Attributes
    ----------
    traversal : list
        List of operators (type: Function) and terminals (type: int, float, or
        str ("const")) encoding the pre-order traversal of the expression tree.

    tokens : np.ndarry (dtype: int)
        Array of integers whose values correspond to indices

    const_pos : list of int
        A list of indicies of constant placeholders along the traversal.

    float_pos : list of float
        A list of indices of constants placeholders or floating-point constants
        along the traversal.

    poly_pos : int
        Index of poly token in the traversal if it has one.

    sympy_expr : str
        The (lazily calculated) SymPy expression corresponding to the program.
        Used for pretty printing _only_.

    complexity : float
        The (lazily calcualted) complexity of the program.

    r : float
        The (lazily calculated) reward of the program.

    count : int
        The number of times this Program has been sampled.

    str : str
        String representation of tokens. Useful as unique identifier.
    """

    # Static variables
    task = None             # Task
    library = None          # Library
    const_optimizer = None  # Function to optimize constants
    cache = {}

    # Cython-related static variables
    have_cython = None      # Do we have cython installed
    execute = None          # Link to execute. Either cython or python

    def __init__(self, tokens=None, on_policy=True):
        """
        Builds the Program from a list of of integers corresponding to Tokens.
        """

        # Can be empty if we are unpickling
        if tokens is not None:
            self._init(tokens, on_policy)

    def _init(self, tokens, on_policy=True):

        self.traversal = [Program.library[t] for t in tokens]
        self.const_pos = [i for i, t in enumerate(self.traversal) if isinstance(t, PlaceholderConstant)]
        poly_pos = [i for i, t in enumerate(self.traversal) if isinstance(t, Polynomial)]
        assert len(poly_pos) <= 1, "A program cannot contain more than one 'poly' token"
        self.poly_pos = poly_pos[0] if len(poly_pos) > 0 else None
        self.len_traversal = len(self.traversal)

        if self.have_cython and self.len_traversal > 1:
            self.is_input_var = array.array('i', [t.input_var is not None for t in self.traversal])

        self.invalid = False
        self.str = tokens.tostring()
        self.tokens = tokens

        self.on_policy_count = 1 if on_policy else 0
        self.off_policy_count = 0 if on_policy else 1
        self.originally_on_policy = on_policy # Note if a program was created on policy

    def execute(self, X):
        """
        Execute program on input X.

        Parameters
        ==========

        X : np.array
            Input to execute the Program over.

        Returns
        =======

        result : np.array or list of np.array
            In a single-object Program, returns just an array. In a multi-object Program, returns a list of arrays.
        """
        if not Program.protected:
            result, self.invalid, self.error_node, self.error_type = Program.execute_function(self.traversal, X)
        else:
            result = Program.execute_function(self.traversal, X)
        return result

    def optimize(self):
        """
        Optimizes PlaceholderConstant tokens against the reward function. The
        optimized values are stored in the traversal.
        """

        # TBD: Should use np.float32

        if len(self.const_pos) == 0:
            return

        # Define the objective function: negative reward
        def f(consts):
            self.set_constants(consts)
            r = self.task.reward_function(self, optimizing=True)
            obj = -r # Constant optimizer minimizes the objective function

            # Need to reset to False so that a single invalid call during
            # constant optimization doesn't render the whole Program invalid.
            self.invalid = False

            return obj

        # Do the optimization
        x0 = np.ones(len(self.const_pos)) # Initial guess
        optimized_constants = Program.const_optimizer(f, x0)

        # Set the optimized constants
        self.set_constants(optimized_constants)

    def get_constants(self):
        """Returns the values of a Program's constants."""

        return [t.value for t in self.traversal if isinstance(t, PlaceholderConstant)]

    def set_constants(self, consts):
        """Sets the program's constants to the given values"""

        for i, const in enumerate(consts):
            assert U.is_float, "Input to program constants must be of a floating point type"
            # Create a new instance of PlaceholderConstant instead of changing
            # the "values" attribute, otherwise all Programs will have the same
            # instance and just overwrite each other's value.
            self.traversal[self.const_pos[i]] = PlaceholderConstant(const)

    def get_poly(self):
        """Returns a Program's Polynomial token if it has one."""

        return None if self.poly_pos is None else self.traversal[self.poly_pos]

    def set_poly(self, poly_token):
        """Sets the program's Polynomial token to the given token"""

        if self.poly_pos is not None:
            self.traversal[self.poly_pos] = poly_token


    @classmethod
    def clear_cache(cls):
        """Clears the class' cache"""

        cls.cache = {}


    @classmethod
    def set_task(cls, task):
        """Sets the class' Task"""

        Program.task = task
        Program.library = task.library


    @classmethod
    def set_const_optimizer(cls, name, **kwargs):
        """Sets the class' constant optimizer"""

        const_optimizer = make_const_optimizer(name, **kwargs)
        Program.const_optimizer = const_optimizer

    @classmethod
    def set_complexity(cls, name):
        """Sets the class' complexity function"""

        all_functions = {
            # No complexity
            None : lambda p : 0.0,

            # Length of sequence
            "length" : lambda p : len(p.traversal),

            # Sum of token-wise complexities
            "token" : lambda p : sum([t.complexity for t in p.traversal]),

            # Binding complexity: % of mutations relative to master seq
            "mutations" : lambda p : Program.task.compute_mutational_distance(p)
        }

        assert name in all_functions, "Unrecognized complexity function name."

        Program.complexity_function = lambda p : all_functions[name](p)

    @classmethod
    def set_execute(cls, protected):
        """Sets which execute method to use"""

        # Check if cython_execute can be imported; if not, fall back to python_execute
        try:
            from dso.execute import cython_execute
            execute_function = cython_execute
            Program.have_cython = True
        except ImportError:
            from dso.execute import python_execute
            execute_function = python_execute
            Program.have_cython = False

        if protected:
            Program.protected = True
            Program.execute_function = execute_function
        else:
            Program.protected = False
            class InvalidLog():
                """Log class to catch and record numpy warning messages"""

                def __init__(self):
                    self.error_type = None # One of ['divide', 'overflow', 'underflow', 'invalid']
                    self.error_node = None # E.g. 'exp', 'log', 'true_divide'
                    self.new_entry = False # Flag for whether a warning has been encountered during a call to Program.execute()

                def write(self, message):
                    """This is called by numpy when encountering a warning"""

                    if not self.new_entry: # Only record the first warning encounter
                        message = message.strip().split(' ')
                        self.error_type = message[1]
                        self.error_node = message[-1]
                    self.new_entry = True

                def update(self):
                    """If a floating-point error was encountered, set Program.invalid
                    to True and record the error type and error node."""

                    if self.new_entry:
                        self.new_entry = False
                        return True, self.error_type, self.error_node
                    else:
                        return False, None, None


            invalid_log = InvalidLog()
            np.seterrcall(invalid_log) # Tells numpy to call InvalidLog.write() when encountering a warning

            # Define closure for execute function
            def unsafe_execute(traversal, X):
                """This is a wrapper for execute_function. If a floating-point error
                would be hit, a warning is logged instead, p.invalid is set to True,
                and the appropriate nan/inf value is returned. It's up to the task's
                reward function to decide how to handle nans/infs."""

                with np.errstate(all='log'):
                    y = execute_function(traversal, X)
                    invalid, error_node, error_type = invalid_log.update()
                    return y, invalid, error_node, error_type

            Program.execute_function = unsafe_execute

    @cached_property
    def r(self):
        """Evaluates and returns the reward of the program"""
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            # Optimize any PlaceholderConstants
            self.optimize()

            # Return final reward after optimizing
            return self.task.reward_function(self)

    @cached_property
    def complexity(self):
        """Evaluates and returns the complexity of the program"""

        return Program.complexity_function(self)

    @cached_property
    def evaluate(self):
        """Evaluates and returns the evaluation metrics of the program."""

        # Program must be optimized before computing evaluate
        if "r" not in self.__dict__:
            print("WARNING: Evaluating Program before computing its reward." \
                  "Program will be optimized first.")
            self.optimize()

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            return self.task.evaluate(self)

    @cached_property
    def sympy_expr(self):
        """
        Returns the attribute self.sympy_expr.

        This is actually a bit complicated because we have to go: traversal -->
        tree --> serialized tree --> SymPy expression
        """

        tree = self.traversal.copy()
        tree = build_tree(tree)
        tree = convert_to_sympy(tree)
        try:
            expr = U.parse_expr(tree.__repr__()) # SymPy expression
        except:
            expr = tree.__repr__()
        return expr

    def pretty(self):
        """Returns pretty printed string of the program"""

        if self.task.task_type != "binding":
            return U.pretty(self.sympy_expr)
        else:
            return None

    def print_stats(self):
        """Prints the statistics of the program

            We will print the most honest reward possible when using validation.
        """

        print("\tReward: {}".format(self.r))
        print("\tCount Off-policy: {}".format(self.off_policy_count))
        print("\tCount On-policy: {}".format(self.on_policy_count))
        print("\tOriginally on Policy: {}".format(self.originally_on_policy))
        print("\tInvalid: {}".format(self.invalid))
        print("\tTraversal: {}".format(self))
        if self.task.task_type != 'binding':
            print("\tExpression:")
            print("{}\n".format(indent(self.pretty(), '\t  ')))

    def __repr__(self):
        """Prints the program's traversal"""
        return ','.join([repr(t) for t in self.traversal])


###############################################################################
# Everything below this line is currently only being used for pretty printing #
###############################################################################


# Possible library elements that sympy capitalizes
capital = ["add", "mul", "pow"]


class Node(object):
    """Basic tree class supporting printing"""

    def __init__(self, val):
        self.val = val
        self.children = []

    def __repr__(self):
        children_repr = ",".join(repr(child) for child in self.children)
        if len(self.children) == 0:
            return self.val # Avoids unnecessary parantheses, e.g. x1()
        return "{}({})".format(self.val, children_repr)


def build_tree(traversal):
    """Recursively builds tree from pre-order traversal"""

    op = traversal.pop(0)
    n_children = op.arity
    val = repr(op)
    if val in capital:
        val = val.capitalize()

    node = Node(val)

    for _ in range(n_children):
        node.children.append(build_tree(traversal))

    return node


def convert_to_sympy(node):
    """Adjusts trees to only use node values supported by sympy"""

    if node.val == "div":
        node.val = "Mul"
        new_right = Node("Pow")
        new_right.children.append(node.children[1])
        new_right.children.append(Node("-1"))
        node.children[1] = new_right

    elif node.val == "sub":
        node.val = "Add"
        new_right = Node("Mul")
        new_right.children.append(node.children[1])
        new_right.children.append(Node("-1"))
        node.children[1] = new_right

    elif node.val == "inv":
        node.val = Node("Pow")
        node.children.append(Node("-1"))

    elif node.val == "neg":
        node.val = Node("Mul")
        node.children.append(Node("-1"))

    elif node.val == "n2":
        node.val = "Pow"
        node.children.append(Node("2"))

    elif node.val == "n3":
        node.val = "Pow"
        node.children.append(Node("3"))

    elif node.val == "n4":
        node.val = "Pow"
        node.children.append(Node("4"))

    for child in node.children:
        convert_to_sympy(child)

    return node

In [ ]:
from numba import jit, prange
import numpy as np

In [ ]:
#subrotines
@jit(nopython=True, parallel=True)
def parents_siblings(tokens, arities, parent_adjust, empty_parent, empty_sibling):
    """
    Given a batch of action sequences, computes and returns the parents and
    siblings of the next element of the sequence.

    The batch has shape (N, L), where N is the number of sequences (i.e. batch
    size) and L is the length of each sequence. In some cases, expressions may
    already be complete; in these cases, this function sees the start of a new
    expression, even though the return value for these elements won't matter
    because their gradients will be zero because of sequence_length.

    Parameters
    __________

    tokens : np.ndarray, shape=(N, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    arities : np.ndarray, dtype=np.int32
        Array of arities corresponding to library indices.

    parent_adjust : np.ndarray, dtype=np.int32
        Array of parent sub-library index corresponding to library indices.

    empty_parent : int
        Integer value for an empty parent token. This is initially computed in controller.py.

    empty_sibling : int
        Integer value for an empty sibling token. This is intially computed in controller.py

    Returns
    _______

    adj_parents : np.ndarray, shape=(N,), dtype=np.int32
        Adjusted parents of the next element of each action sequence.

    siblings : np.ndarray, shape=(N,), dtype=np.int32
        Siblings of the next element of each action sequence.

    """
    N, L = tokens.shape

    adj_parents = np.full(shape=(N,), fill_value=empty_parent, dtype=np.int32)
    siblings = np.full(shape=(N,), fill_value=empty_sibling, dtype=np.int32)
    # Parallelized loop over action sequences
    for r in prange(N):
        arity = arities[tokens[r, -1]]
        if arity > 0: # Parent is the previous element; no sibling
            adj_parents[r] = parent_adjust[tokens[r, -1]]
            continue
        dangling = 0
        # Loop over elements in an action sequence
        for c in range(L):
            arity = arities[tokens[r, L - c - 1]]
            dangling += arity - 1
            if dangling == 0: # Parent is L-c-1, sibling is the next
                adj_parents[r] = parent_adjust[tokens[r, L - c - 1]]
                siblings[r] = tokens[r, L - c]
                break
    return adj_parents, siblings


# TBD: Refactor to compute hierarchical obs
@jit(nopython=True, parallel=False)
def jit_parents_siblings_at_once(tokens, arities, parent_adjust):
    """
    Given a batch of action sequences, computes and returns the parents and
    siblings over the entire sequence at once.

    This version will give all parents and siblings at once over the full
    and complete set of tokens. This is useful for Deap because it generates
    each sequence in one go rather than one token at a time.

    The batch has shape (N, L), where N is the number of sequences (i.e. batch
    size) and L is the length of each sequence. In some cases, expressions may
    already be complete; in these cases, this function sees the start of a new
    expression, even though the return value for these elements won't matter
    because their gradients will be zero because of sequence_length.

    >>> This has been tested and gives the same answer as the regular parent_sibling class for
        DEAP functions.

    Parameters
    __________

    tokens : np.ndarray, shape=(N, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    arities : np.ndarray, dtype=np.int32
        Array of arities corresponding to library indices.

    parent_adjust : np.ndarray, dtype=np.int32
        Array of parent sub-library index corresponding to library indices.

    Returns
    _______

    adj_parents : np.ndarray, shape=(N, L), dtype=np.int32
        Adjusted parents of the next element of each action sequence.

    siblings : np.ndarray, shape=(N, L), dtype=np.int32
        Siblings of the next element of each action sequence.



    """
    N, L = tokens.shape

    empty_parent    = np.max(parent_adjust) + 1 # Empty token is after all non-empty tokens
    empty_sibling   = len(arities) # Empty token is after all non-empty tokens
    adj_parents     = np.full(shape=(N,L), fill_value=empty_parent, dtype=np.int32)
    siblings        = np.full(shape=(N,L), fill_value=empty_sibling, dtype=np.int32)

    # Parallelization is slower here ...

    # We loop over actions since frequently, N is 1 when used with Deap.
    for b in range(1, L):
        for r in range(N):
            # This part is optimal
            arity = arities[tokens[r, b - 1]]
            if arity > 0: # Parent is the previous element; no sibling
                adj_parents[r, b]   = parent_adjust[tokens[r, b - 1]]
                continue

            # This part may not be optimal here, but is fast enough for now
            dangling = 0
            # Loop over elements in an action sequence GOING BACKWARDS
            for c in range(b):
                arity = arities[tokens[r, b - c - 1]]
                dangling += arity - 1

                # Most recent non-dangling action
                if dangling == 0:
                    # Parent is b-c-1, sibling is the next
                    adj_parents[r, b]   = parent_adjust[tokens[r, b - c - 1]]
                    siblings[r, b]      = tokens[r, b - c]
                    break

    return adj_parents, siblings


@jit(nopython=True, parallel=True)
def ancestors(actions, arities, ancestor_tokens):
    """
    Given a batch of action sequences, determines whether the next element of
    the sequence has an ancestor in ancestor_tokens.

    The batch has shape (N, L), where N is the number of sequences (i.e. batch
    size) and L is the length of each sequence. In some cases, expressions may
    already be complete; in these cases, this function sees the start of a new
    expression, even though the return value for these elements won't matter
    because their gradients will be zero because of sequence_length.

    Parameters
    __________

    actions : np.ndarray, shape=(N, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    arities : np.ndarray, dtype=np.int32
        Array of arities corresponding to library indices.

    ancestor_tokens : np.ndarray, dtype=np.int32
        Array of ancestor library indices to check.

    Returns
    _______

    mask : np.ndarray, shape=(N,), dtype=np.bool_
        Mask of whether the next element of each sequence has an ancestor in
        ancestor_tokens.
    """

    for token in ancestor_tokens:
        assert arities[token] == 1, "subroutine 'ancestors' may not work" \
            "properly for non-unary ancestor_tokens"

    N, L = actions.shape
    mask = np.zeros(shape=(N,), dtype=np.bool_)
    # Parallelized loop over action sequences
    for r in prange(N):
        dangling = 0
        threshold = None # If None, current branch does not have trig ancestor
        for c in range(L):
            arity = arities[actions[r, c]]
            dangling += arity - 1
            # Turn "on" if a trig function is found
            # Remain "on" until branch completes
            if threshold is None:
                for trig_token in ancestor_tokens:
                    if actions[r, c] == trig_token:
                        threshold = dangling - 1
                        break
            # Turn "off" once the branch completes
            else:
                if dangling == threshold:
                    threshold = None
        # If the sequences ended "on", then there is a trig ancestor
        if threshold is not None:
            mask[r] = True
    return mask


@jit(nopython=True, parallel=False)
def jit_check_constraint_violation(actions, actions_tokens, other, other_tokens):
    r"""
    Given an action sequence, another type of sequences such as siblings
    or children and constraint tokens, this will return a bool which tells if
    the constraint was violated.

    The batch has shape (1, L), L is the length of the sequence.

    This does the same thing as:

        np.any(np.logical_and(np.isin(actions, actions_tokens), np.isin(other, other_tokens)))

    but is much faster because it can quit when a single constraint is violated.

    >>> This has been tested against the old inverse token constraint and gives the same answer.

    Parameters
    __________

    actions : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    actions_tokens : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    other : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of other sequences. Values correspond to library indices.

    other_tokens : np.ndarray, dtype=np.int32
        Array of constraint tokens to match other against.

    Returns
    _______

    bool : Was the constraint violated.

    """
    # Is this token item A found in the list of tokens in B?
    def a_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return True
        return False

    _,L     = actions.shape
    A       = actions_tokens.shape[0]
    O       = other_tokens.shape[0]

    # For each action:
    for l in range(L):
        # Check if this token matches a constraint token
        # And check if the other also matches one of its constraints
        if a_in_b(actions[0,l], actions_tokens, A) and a_in_b(other[0,l], other_tokens, O):
            return True

    return False


@jit(nopython=True, parallel=False)
def jit_check_constraint_violation_uchild(actions, parent, sibling, actions_tokens,
                                          adj_unary_effectors, adj_effectors):
    r"""
    Given an action sequence, another type of sequences such as siblings
    or children and constraint tokens, this will return a bool which tells if
    the constraint was violated.

    The batch has shape (1, L), L is the length of the sequence.

    This does the same thing as:

        for i, a in enumerate(actions):
            if (parent[i] in adj_unary_effectors) or (sibling[i] in self.targets and parent[i] in adj_effectors)
                if a in self.targets:
                    return True

    but is much faster because it can quit when a single constraint is violated.


    Parameters
    __________

    actions : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    parent  : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of parent sequences. Values correspond to library indices.

    sibling : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of sibling sequences. Values correspond to library indices.

    actions_tokens : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    adj_unary_effectorss : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    adj_effectors : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    Returns
    _______

    bool : Was the constraint violated.

    """
    # Is this token item A found in the list of tokens in B?
    def a_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return True
        return False

    _,L     = actions.shape
    A       = actions_tokens.shape[0]
    U       = adj_unary_effectors.shape[0]
    E       = adj_effectors.shape[0]

    # For each action:
    for l in range(L):

        # Is this the right action?
        if a_in_b(actions[0,l], actions_tokens, A):
            # CASE 1: parent is a unary effector
            if a_in_b(parent[0,l], adj_unary_effectors, U):
                return True

            # CASE 2: sibling is a target and parent is an effector
            if a_in_b(sibling[0,l], actions_tokens, A) and a_in_b(parent[0,l], adj_effectors, E):
                return True

    return False


@jit(nopython=True, parallel=False)
def jit_check_constraint_violation_descendant_no_target_tokens(\
        actions, effector_tokens, binary_tokens, unary_tokens):

    r"""
    Given an action sequence, another type of sequences such as siblings
    or children and constraint tokens, this will return a bool which tells if
    the constraint was violated.

    The batch has shape (1, L), L is the length of the sequence.

    This can be used (for instance) to check for trig constraints.

    This does the same thing as:

        descendant = False # True when current node is a descendant of operator

        for a in actions:
            if a in self.targets:
                if descendant:
                    return True
                descendant = True
                dangling   = 1
            elif descendant:
                if a in library.binary_tokens:
                    dangling += 1
                elif a not in library.unary_tokens:
                    dangling -= 1
                if dangling == 0:
                    descendant = False

        return False

    >>> This has been tested against the old Trig constraint and gives the same answer

    Parameters
    __________

    actions : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    effector_tokens : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    binrary_tokens : np.ndarray, dtype=np.int32
        Array of binary function tokens in the current library.

    uniary_tokens : np.ndarray, dtype=np.int32
        Array of unary function tokens in the current library.

    Returns
    _______

    bool : Was the constraint violated.

    """

    # Is this token item A found in the list of tokens in B?
    def a_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return True
        return False

    # Is this token item A NOT found in the list of tokens in B?
    def a_not_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return False
        return True

    _,L     = actions.shape
    E       = effector_tokens.shape[0]
    B       = binary_tokens.shape[0]
    U       = unary_tokens.shape[0]

    descendant = False # True when current node is a descendant of operator

    # For each action:
    for l in range(L):

        action = actions[0,l]

        if a_in_b(action, effector_tokens, E):
            # Does action match a target token?
            if descendant:
                # a token was found previously, but
                # we are still in a dangling node, therefore
                # we have a token inside a token expression
                # that we are not allowed to have e.g.
                # sin(sin(x)) .

                return True
            descendant  = True
            dangling    = 1
        elif descendant:
            if a_in_b(action, binary_tokens, B):
                # Does action match a binary token?
                # Then add one to dangling.
                dangling += 1
            elif a_not_in_b(action, unary_tokens, U):
                # Does action match a terminal token?
                # Then subtract one from dangling.
                # We skip the instance of unary since
                # this leaves dangling to be += 0
                # and assume that any token not binary
                # and unary is a terminal.
                dangling -= 1

            # If we no longer have any dangling nodes,
            # Then we cannot be a descendant.
            if dangling == 0:
                descendant = False

    return False

@jit(nopython=True, parallel=False)
def jit_check_constraint_violation_descendant_with_target_tokens(\
        actions, target_tokens, effector_tokens, binary_tokens, unary_tokens):

    r"""

    Parameters
    __________

    actions : np.ndarray, shape=(1, L), dtype=np.int32
        Batch of action sequences. Values correspond to library indices.

    target_tokens : np.ndarray, dtype=np.int32
        Array of constraint tokens to match action against.

    binrary_tokens : np.ndarray, dtype=np.int32
        Array of binary function tokens in the current library.

    uniary_tokens : np.ndarray, dtype=np.int32
        Array of unary function tokens in the current library.

    Returns
    _______

    bool : Was the constraint violated.

    """

    # Is this token item A found in the list of tokens in B?
    def a_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return True
        return False

    # Is this token item A NOT found in the list of tokens in B?
    def a_not_in_b(a, B_tokens, B):
        for b in range(B):
            if a == B_tokens[b]:
                return False
        return True

    _,L     = actions.shape
    T       = target_tokens.shape[0]
    B       = binary_tokens.shape[0]
    U       = unary_tokens.shape[0]

    descendant = False # True when current node is a descendant of operator

    # For each action:
    for l in range(L):

        action = actions[0,l]

        if a_in_b(action, effector_tokens, T):
            # Does action match a target token?
            descendant  = True
            dangling    = 1
        elif a_in_b(action, target_tokens, T):
            if descendant:
                return True
        elif descendant:
            if a_in_b(action, binary_tokens, B):
                # Does action match a binary token?
                # Then add one to dangling.
                dangling += 1
            elif a_not_in_b(action, unary_tokens, U):
                # Does action match a terminal token?
                # Then subtract one from dangling.
                # We skip the instance of unary since
                # this leaves dangling to be += 0
                # and assume that any token not binary
                # and unary is a terminal.
                dangling -= 1

            # If we no longer have any dangling nodes,
            # Then we cannot be a descendant.
            if dangling == 0:
                descendant = False

    return False

In [ ]:
from abc import ABC, abstractmethod
import numpy as np

In [ ]:
#task
class Task(ABC):
    """
    Object specifying a symbolic search task.

    Attributes
    ----------
    library : Library
        Library of Tokens.

    stochastic : bool
        Whether the reward function of the task is stochastic.

    task_type : str
        Task type: regression, control, or binding.

    name : str
        Unique name for instance of this task.
    """

    task_type = None

    @abstractmethod
    def reward_function(self, program, optimizing=False):
        """
        The reward function for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to compute reward of.

        optimizing : bool

            Whether the reward is computed for PlaceholderConstant optimization.

        Returns
        -------
        reward : float

            Fitness/reward of the program.
        """
        raise NotImplementedError

    @abstractmethod
    def evaluate(self, program):
        """
        The evaluation metric for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to evaluate.

        Returns
        -------

        info : dict

            Dictionary of evaluation metrics. Special key "success" is used to
            trigger early stopping.
        """
        raise NotImplementedError

    @abstractmethod
    def get_next_obs(self, actions, obs, already_finished):
        """
        Produce the next observation and prior from the current observation and
        list of actions so far. Observations must be 1-D np.float32 vectors.

        Parameters
        ----------

        actions : np.ndarray (dtype=np.int32)
            Actions selected so far, shape (batch_size, current_length)

        obs : np.ndarray (dtype=np.float32)
            Previous observation, shape (batch_size, OBS_DIM).

        already_finished : np.ndarray (dtype=bool)
            Whether the object has *already* been completed.

        Returns
        -------

        next_obs : np.ndarray (dtype=np.float32)
            The next observation, shape (batch_size, OBS_DIM).

        prior : np.ndarray (dtype=np.float32)
            Prior for selecting the next token, shape (batch_size,
            self.library.L).

        finished : np.ndarray (dtype=bool)
            Whether the object has *ever* been completed.
        """
        pass

    @abstractmethod
    def reset_task(self):
        """
        Create the starting observation.

        Returns
        -------

        obs : np.ndarray (dtype=np.float32)
            Starting observation, shape (batch_size, OBS_DIM).
        """
        pass


class HierarchicalTask(Task):
    """
    A Task in which the search space is a binary tree. Observations include
    the previous action, the parent, the sibling, and/or the number of dangling
    (unselected) nodes.
    """

    OBS_DIM = 4 # action, parent, sibling, dangling

    def __init__(self):
        super(Task).__init__()

    def get_next_obs(self, actions, obs, already_finished):

        dangling = obs[:, 3] # Shape of obs: (?, 4)
        action = actions[:, -1] # Current action
        lib = self.library

        # Compute parents and siblings
        parent, sibling = parents_siblings(actions,
                                           arities=lib.arities,
                                           parent_adjust=lib.parent_adjust,
                                           empty_parent=lib.EMPTY_PARENT,
                                           empty_sibling=lib.EMPTY_SIBLING)

        # Compute dangling
        dangling += lib.arities[action] - 1

        # Compute finished
        just_finished = (dangling == 0) # Trees that completed _this_ time step
        # [batch_size]
        finished = np.logical_or(just_finished,
                                 already_finished)

        # Compute priors
        prior = self.prior(actions, parent, sibling, dangling, finished) # (?, n_choices)

        # Combine observation dimensions
        next_obs = np.stack([action, parent, sibling, dangling], axis=1) # (?, 4)
        next_obs = next_obs.astype(np.float32)

        return next_obs, prior, finished

    def reset_task(self, prior):
        """
        Returns the initial observation: empty action, parent, and sibling, and
        dangling is 1.
        """

        self.prior = prior

        # Order of observations: action, parent, sibling, dangling
        initial_obs = np.array([self.library.EMPTY_ACTION,
                                self.library.EMPTY_PARENT,
                                self.library.EMPTY_SIBLING,
                                1],
                               dtype=np.float32)
        return initial_obs


class SequentialTask(Task):
    """
    A Task in which the search space is a (possibly variable-length) sequence.
    The observation is simply the previous action.
    """

    pass


def make_task(task_type, **config_task):
    """
    Factory function for Task object.

    Parameters
    ----------

    task_type : str
        Type of task:
        "regression" : Symbolic regression task.
        "control" : Episodic reinforcement learning task.
        "binding": AbAg binding affinity optimization task.

    config_task : kwargs
        Task-specific arguments. See specifications of task_dict.

    Returns
    -------

    task : Task
        Task object.
    """

    # Lazy import of task factory functions
    if task_type == 'binding':
        from dso.task.binding.binding import BindingTask
        task_class = BindingTask
    elif task_type == "regression":
        from dso.task.regression.regression import RegressionTask
        task_class = RegressionTask
    elif task_type == "control":
        from dso.task.control.control import ControlTask
        task_class = ControlTask
    else:
        # Custom task import
        task_class = import_custom_source(task_type)
        assert issubclass(task_class, Task), \
            "Custom task {} must subclass dso.task.Task.".format(task_class)

    task = task_class(**config_task)
    return task


def set_task(config_task):
    """Helper function to make set the Program class Task and execute function
    from task config."""

    # Use of protected functions is the same for all tasks, so it's handled separately
    protected = config_task["protected"] if "protected" in config_task else False

    Program.set_execute(protected)
    task = make_task(**config_task)
    Program.set_task(task)

In [ ]:
import heapq
from collections import namedtuple

import numpy as np

In [ ]:
#memory

Batch = namedtuple(
    "Batch", ["actions", "obs", "priors", "lengths", "rewards", "on_policy"])


# TBD: This should be member function of Batch class
def save_batch(B, save_path):
    """Save Batch to file."""

    with open(save_path, "wb") as f:
        np.savez(f, **dict(B._asdict()))


# TBD: This should be class function of Batch class
def load_batch(save_path):
    """Load Batch from file."""

    B = np.load(save_path)
    B = Batch(**B)
    return B


def make_queue(policy=None, priority=False, capacity=np.inf, seed=0):
    """Factory function for various Queues.

    Parameters
    ----------
    policy : dso.policy.Policy
        Reference to the Policy, used to compute probabilities of items in
        the Queue.

    priority : bool
        If True, returns an object inheriting UniquePriorityQueue. Otherwise,
        returns an object inheriting from UniqueQueue.

    capacity : int
        Maximum queue length.

    seed : int
        RNG seed used for random sampling.

    Returns
    -------
    queue : ProgramQueue
        Dynamic class inheriting from ProgramQueueMixin and a Queue subclass.
    """

    if priority:
        Base = UniquePriorityQueue
    else:
        Base = UniqueQueue

    class ProgramQueue(ProgramQueueMixin, Base):
        def __init__(self, policy, capacity, seed):
            ProgramQueueMixin.__init__(self, policy)
            Base.__init__(self, capacity, seed)

    queue = ProgramQueue(policy, capacity, seed)
    return queue


def get_samples(batch, key):
    """
    Returns a sub-Batch with samples from the given indices.

    Parameters
    ----------
    key : int or slice
        Indices of samples to return.

    Returns
    -------
    batch : Batch
        Sub-Batch with samples from the given indices.
    """

    batch = Batch(
        actions=batch.actions[key],
        obs=batch.obs[key],
        priors=batch.priors[key],
        lengths=batch.lengths[key],
        rewards=batch.rewards[key],
        on_policy=batch.on_policy[key])
    return batch


# Adapted from https://github.com/tensorflow/models/blob/1af55e018eebce03fb61bba9959a04672536107d/research/brain_coder/common/utils.py
class ItemContainer(object):
    """Class for holding an item with its score.

    Defines a comparison function for use in the heap-queue.
    """

    def __init__(self, score, item, extra_data):
        self.item = item
        self.score = score
        self.extra_data = extra_data

    def __lt__(self, other):
        assert isinstance(other, type(self))
        return self.score < other.score

    def __eq__(self, other):
        assert isinstance(other, type(self))
        return self.item == other.item

    def __iter__(self):
        """Allows unpacking like a tuple."""
        yield self.score
        yield self.item
        yield self.extra_data

    def __repr__(self):
        """String representation of this item.

        `extra_data` is not included in the representation. We are assuming that
        `extra_data` is not easily interpreted by a human (if it was, it should be
        hashable, like a string or tuple).

        Returns:
            String representation of `self`.
        """
        return str((self.score, self.item))

    def __str__(self):
        return repr(self)


class Queue(object):
    """Abstract class for queue that must define a push and pop routine"""

    def __init__(self, capacity, seed=0):
        self.capacity = capacity
        self.rng = np.random.RandomState(seed)
        self.heap = []
        self.unique_items = set()

    def push(self, score, item, extra_data):
        raise NotImplementedError

    def pop(self):
        raise NotImplementedError

    def random_sample(self, sample_size):
        """Uniform randomly select items from the queue.

        Args:
            sample_size: Number of random samples to draw. The same item can be
                    sampled multiple times.

        Returns:
            List of sampled items (of length `sample_size`). Each element in the list
            is a tuple: (item, extra_data).
        """
        idx = self.rng.choice(len(self.heap), sample_size, )
        return [(self.heap[i].item, self.heap[i].extra_data) for i in idx]

    def __len__(self):
        return len(self.heap)

    def __iter__(self):
        for _, item, _ in self.heap:
            yield item

    def __repr__(self):
        return '[' + ', '.join(repr(c) for c in self.heap) + ']'

    def __str__(self):
        return repr(self)


class UniqueQueue(Queue):
    """A queue in which duplicates are not allowed. Instead, adding a duplicate
    moves that item to the back of the queue."""

    def push(self, score, item, extra_data=None):
        """Push an item onto the queue, or move it to the back if already
        present.

        Score is unused but included as an argument to follow the interface.
        """

        container = ItemContainer(None, item, extra_data)

        # If the item is already in the queue, move it to the back of the queue
        # and return
        if item in self.unique_items:
            self.heap.remove(container)
            self.heap.append(container)
            return

        # If the queue is at capacity, first pop the front of the queue
        if len(self.heap) >= self.capacity:
            self.pop()

        # Add the item
        self.heap.append(container)
        self.unique_items.add(item)

    def pop(self):
        """Pop the front of the queue (the oldest item)."""

        if not self.heap:
            return ()
        score, item, extra_data = self.heap.pop(0)
        self.unique_items.remove(item)
        return (score, item, extra_data)


# Adapted from https://github.com/tensorflow/models/blob/1af55e018eebce03fb61bba9959a04672536107d/research/brain_coder/common/utils.py
class UniquePriorityQueue(Queue):
    """A priority queue where duplicates are not added.

    The top items by score remain in the queue. When the capacity is reached,
    the lowest scored item in the queue will be dropped.
    """

    def push(self, score, item, extra_data=None):
        """Push an item onto the queue.

        If the queue is at capacity, the item with the smallest score will be
        dropped. Note that it is assumed each item has exactly one score. The same
        item with a different score will still be dropped.

        Args:
            score: Number used to prioritize items in the queue. Largest scores are
                    kept in the queue.
            item: A hashable item to be stored. Duplicates of this item will not be
                    added to the queue.
            extra_data: An extra (possible not hashable) data to store with the item.
        """
        if item in self.unique_items:
            return
        if len(self.heap) >= self.capacity:
            _, popped_item, _ = heapq.heappushpop(
                self.heap, ItemContainer(score, item, extra_data))
            self.unique_items.add(item)
            self.unique_items.remove(popped_item)
        else:
            heapq.heappush(self.heap, ItemContainer(score, item, extra_data))
            self.unique_items.add(item)

    def pop(self):
        """Pop the item with the lowest score.

        Returns:
            score: Item's score.
            item: The item that was popped.
            extra_data: Any extra data stored with the item.
        """
        if not self.heap:
            return ()
        score, item, extra_data = heapq.heappop(self.heap)
        self.unique_items.remove(item)
        return score, item, extra_data

    def get_max(self):
        """Peek at the item with the highest score.

        Returns:
            Same as `pop`.
        """
        if not self.heap:
            return ()
        score, item, extra_data = heapq.nlargest(1, self.heap)[0]
        return score, item, extra_data

    def get_min(self):
        """Peek at the item with the lowest score.

        Returns:
            Same as `pop`.
        """
        if not self.heap:
            return ()
        score, item, extra_data = heapq.nsmallest(1, self.heap)[0]
        return score, item, extra_data

    def iter_in_order(self):
        """Iterate over items in the queue from largest score to smallest.

        Yields:
            item: Hashable item.
            extra_data: Extra data stored with the item.
        """
        for _, item, extra_data in heapq.nlargest(len(self.heap), self.heap):
            yield item, extra_data


class ProgramQueueMixin():
    """A mixin for Queues with additional utilities specific to Batch and
    Program."""

    def __init__(self, policy=None):
        self.policy = policy

    def push_sample(self, sample, program):
        """
        Push a single sample corresponding to Program to the queue.

        Parameters
        ----------
        sample : Batch
            A Batch comprising a single sample.

        program : Program
            Program corresponding to the sample.
        """

        id_ = program.str
        score = sample.rewards
        self.push(score, id_, sample)

    def push_batch(self, batch, programs):
        """Push a Batch corresponding to Programs to the queue."""

        for i, program in enumerate(programs):
            sample = get_samples(batch, i)
            self.push_sample(sample, program)

    def push_best(self, batch, programs):
        """Push the single best sample from a Batch"""

        i = np.argmax(batch.rewards)
        sample = get_samples(batch, i)
        program = programs[i]
        self.push_sample(sample, program)

    def sample_batch(self, sample_size):
        """Randomly select items from the queue and return them as a Batch."""

        assert len(self.heap) > 0, "Cannot sample from an empty queue."
        samples = [sample for (id_, sample) in self.random_sample(sample_size)]
        batch = self._make_batch(samples)
        return batch

    def _make_batch(self, samples):
        """Turns a list of samples into a Batch."""

        # Pad up to max length across samples
        max_len = max([s.actions.shape[0] for s in samples])
        padded_aop = [pad_action_obs_priors(np.expand_dims(s.actions, axis=0),
                                            np.expand_dims(s.obs, axis=0),
                                            np.expand_dims(s.priors, axis=0),
                                            pad_length=max_len - s.actions.shape[0]) for s in samples]

        actions, obs, priors = zip(*padded_aop)
        actions = np.concatenate(actions, axis=0)
        obs = np.concatenate(obs, axis=0)
        priors = np.concatenate(priors, axis=0)
        lengths = np.array([s.lengths for s in samples], dtype=np.int32)
        rewards = np.array([s.rewards for s in samples], dtype=np.float32)
        on_policy = np.array([s.on_policy for s in samples], dtype=np.bool)
        batch = Batch(actions=actions, obs=obs, priors=priors,
                      lengths=lengths, rewards=rewards, on_policy=on_policy)
        return batch

    def to_batch(self):
        """Return the entire queue as a Batch."""

        samples = [container.extra_data for container in self.heap]
        batch = self._make_batch(samples)
        return batch

    def compute_probs(self):
        """Computes the probabilities of items in the queue according to the
        Policy."""

        if self.policy is None:
            raise RuntimeError("Cannot compute probabilities. This Queue does \
                not have a Policy.")
        return self.policy.compute_probs(self.to_batch())

    def get_rewards(self):
        """Returns the rewards"""

        r = [container.extra_data.rewards for container in self.heap]
        return r

    def save(self, save_path):
        """Save the contents of the queue to file."""

        B = self.to_batch()
        save_batch(B, save_path)

    def load(self, load_path):
        """Load the contents of the queue from file."""

        B = load_batch(load_path)
        programs = [from_tokens(np.array(tokens, dtype=np.int32)) for tokens in B.actions]
        for p, r in zip(programs, B.rewards):
            p.r = r
        self.push_batch(B, programs)

In [ ]:
#variance

def quantile_variance(memory_queue, policy, batch_size, epsilon, step,
                      n_experiments=1000, estimate_bias=True,
                      n_samples_bias=1e6):

    print("Running quantile variance/bias experiments...")
    empirical_quantiles = []
    memory_augmented_quantiles = []

    if len(memory_queue) < memory_queue.capacity:
        print("WARNING: Memory queue not yet at capacity.")

    memory_r = memory_queue.get_rewards()
    memory_w = memory_queue.compute_probs()
    for exp in range(n_experiments):
        actions, obs, priors = policy.sample(batch_size)
        programs = [from_tokens(a) for a in actions]
        r = np.array([p.r for p in programs])
        quantile = np.quantile(r, 1 - epsilon, interpolation="higher")
        empirical_quantiles.append(quantile)
        unique_programs = [p for p in programs if p.str not in memory_queue.unique_items]
        N = len(unique_programs)
        sample_r = [p.r for p in unique_programs]
        combined_r = np.concatenate([memory_r, sample_r])
        if N == 0:
            print("WARNING: Found no unique samples in batch!")
            combined_w = memory_w / memory_w.sum() # Renormalize
        else:
            sample_w = np.repeat((1 - memory_w.sum()) / N, N)
            combined_w = np.concatenate([memory_w, sample_w])

        # Compute the weighted quantile
        quantile = weighted_quantile(values=combined_r, weights=combined_w, q=1 - epsilon)
        memory_augmented_quantiles.append(quantile)

    empirical_quantiles = np.array(empirical_quantiles)
    memory_augmented_quantiles = np.array(memory_augmented_quantiles)
    print("Train step:", step)
    print("Memory weight:", memory_w.sum())
    print("Mean(empirical quantile):", np.mean(empirical_quantiles))
    print("Var(empirical quantile):", np.var(empirical_quantiles))
    print("Mean(Memory augmented quantile):", np.mean(memory_augmented_quantiles))
    print("Var(Memory augmented quantile):", np.var(memory_augmented_quantiles))
    if estimate_bias:
        actions, obs, priors = policy.sample(int(n_samples_bias))
        programs = [from_tokens(a) for a in actions]
        r = np.array([p.r for p in programs])
        true_quantile = np.quantile(r, 1 - epsilon, interpolation="higher")
        print("'True' empirical quantile:", true_quantile)
        print("Empirical quantile bias:", np.mean(np.abs(empirical_quantiles - true_quantile)))
        print("Memory-augmented quantile bias:", np.mean(np.abs(memory_augmented_quantiles - true_quantile)))
    exit()

In [ ]:
import os
import json
import time
from itertools import compress

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.random import set_seed


In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
#train
# Ignore TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # Enable v1 behavior
tf.set_random_seed(0)

# Work for multiprocessing pool: compute reward
def work(p):
    """Compute reward and return it with optimized constants"""
    r = p.r
    return p

class Trainer():
    def __init__(self, sess, policy, policy_optimizer, gp_controller, logger,
                 pool, n_samples=2000000, batch_size=1000, alpha=0.5,
                 epsilon=0.05, verbose=True, baseline="R_e",
                 b_jumpstart=False, early_stopping=True, debug=0,
                 use_memory=False, memory_capacity=1e3,  warm_start=None, memory_threshold=None,
                 complexity="token", const_optimizer="scipy", const_params=None,  n_cores_batch=1):

        """
        Initializes the main training loop.

        Parameters
        ----------
        sess : tf.Session
            TensorFlow Session object.

        policy : dso.policy.Policy
            Parametrized probability distribution over discrete objects.
            Used to generate programs and compute loglikelihoods.

        policy_optimizer : dso.policy_optimizer.policy_optimizer
            policy_optimizer object used to optimize the policy.

        gp_controller : dso.gp.gp_controller.GPController or None
            GP controller object used to generate Programs.

        logger : dso.train_stats.StatsLogger
            Logger to save results with

        pool : multiprocessing.Pool or None
            Pool to parallelize reward computation. For the control task, each
            worker should have its own TensorFlow model. If None, a Pool will be
            generated if n_cores_batch > 1.

        n_samples : int or None, optional
            Total number of objects to sample. This may be exceeded depending
            on batch size.

        batch_size : int, optional
            Number of sampled expressions per iteration.

        alpha : float, optional
            Coefficient of exponentially-weighted moving average of baseline.

        epsilon : float or None, optional
            Fraction of top expressions used for training. None (or
            equivalently, 1.0) turns off risk-seeking.

        verbose : bool, optional
            Whether to print progress.

        baseline : str, optional
            Type of baseline to use: grad J = (R - b) * grad-log-prob(expression).
            Choices:
            (1) "ewma_R" : b = EWMA(<R>)
            (2) "R_e" : b = R_e
            (3) "ewma_R_e" : b = EWMA(R_e)
            (4) "combined" : b = R_e + EWMA(<R> - R_e)
            In the above, <R> is the sample average _after_ epsilon sub-sampling and
            R_e is the (1-epsilon)-quantile estimate.

        b_jumpstart : bool, optional
            Whether EWMA part of the baseline starts at the average of the first
            iteration. If False, the EWMA starts at 0.0.

        early_stopping : bool, optional
            Whether to stop early if stopping criteria is reached.

        debug : int, optional
            Debug level, also passed to Controller. 0: No debug. 1: Print initial
            parameter means. 2: Print parameter means each step.

        use_memory : bool, optional
            If True, use memory queue for reward quantile estimation.

        memory_capacity : int
            Capacity of memory queue.

        warm_start : int or None
            Number of samples to warm start the memory queue. If None, uses
            batch_size.

        memory_threshold : float or None
            If not None, run quantile variance/bias estimate experiments after
            memory weight exceeds memory_threshold.

        complexity : str, optional
            Not used

        const_optimizer : str or None, optional
            Not used

        const_params : dict, optional
            Not used

        n_cores_batch : int, optional
            Not used


        """
        self.sess = sess
        # Initialize compute graph
        self.sess.run(tf.global_variables_initializer())

        self.policy = policy
        self.policy_optimizer = policy_optimizer
        self.gp_controller = gp_controller
        self.logger = logger
        self.pool = pool
        self.n_samples = n_samples
        self.batch_size = batch_size
        self.alpha = alpha
        self.epsilon = epsilon
        self.verbose = verbose
        self.baseline = baseline
        self.b_jumpstart = b_jumpstart
        self.early_stopping = early_stopping
        self.debug = debug
        self.use_memory = use_memory
        self.memory_threshold = memory_threshold

        if self.debug:
            tvars = tf.trainable_variables()
            def print_var_means():
                tvars_vals = self.sess.run(tvars)
                for var, val in zip(tvars, tvars_vals):
                    print(var.name, "mean:", val.mean(), "var:", val.var())
            self.print_var_means = print_var_means

        # Create the priority_queue if needed
        if hasattr(self.policy_optimizer, 'pqt_k'):
            from dso.policy_optimizer.pqt_policy_optimizer import PQTPolicyOptimizer
            assert type(self.policy_optimizer) == PQTPolicyOptimizer
            # Create the priority queue
            k = self.policy_optimizer.pqt_k
            if k is not None and k > 0:
                self.priority_queue = make_queue(priority=True, capacity=k)
        else:
            self.priority_queue = None

        # Create the memory queue
        if self.use_memory:
            assert self.epsilon is not None and self.epsilon < 1.0, \
                "Memory queue is only used with risk-seeking."
            self.memory_queue = make_queue(policy=self.policy, priority=False,
                                           capacity=int(memory_capacity))

            # Warm start the queue
            # TBD: Parallelize. Abstract sampling a Batch
            warm_start = warm_start if warm_start is not None else self.batch_size
            actions, obs, priors = policy.sample(warm_start)
            programs = [from_tokens(a) for a in actions]
            r = np.array([p.r for p in programs])
            l = np.array([len(p.traversal) for p in programs])
            on_policy = np.array([p.originally_on_policy for p in programs])
            sampled_batch = Batch(actions=actions, obs=obs, priors=priors,
                                  lengths=l, rewards=r, on_policy=on_policy)
            self.memory_queue.push_batch(sampled_batch, programs)
        else:
            self.memory_queue = None

        self.nevals = 0 # Total number of sampled expressions (from RL or GP)
        self.iteration = 0 # Iteration counter
        self.r_best = -np.inf
        self.p_r_best = None
        self.done = False

    def run_one_step(self, override=None):
        """
        Executes one step of main training loop. If override is given,
        train on that batch. Otherwise, sample the batch to train on.

        Parameters
        ----------
        override : tuple or None
            Tuple of (actions, obs, priors, programs) to train on offline
            samples instead of sampled
        """
        positional_entropy = None
        top_samples_per_batch = list()
        if self.debug >= 1:
            print("\nDEBUG: Policy parameter means:")
            self.print_var_means()

        ewma = None if self.b_jumpstart else 0.0 # EWMA portion of baseline

        start_time = time.time()
        if self.verbose:
            print("-- RUNNING ITERATIONS START -------------")


        # Number of extra samples generated during attempt to get
        # batch_size new samples
        n_extra = 0
        # Record previous cache before new samples are added by from_tokens
        s_history = list(Program.cache.keys())

        # Construct the actions, obs, priors, and programs
        # Shape of actions: (batch_size, max_length)
        # Shape of obs: (batch_size, obs_dim, max_length)
        # Shape of priors: (batch_size, max_length, n_choices)
        if override is None:
            # Sample batch of Programs from the Controller
            actions, obs, priors = self.policy.sample(self.batch_size)
            programs = [from_tokens(a) for a in actions]
        else:
            # Train on the given batch of Programs
            actions, obs, priors, programs = override
            for p in programs:
                Program.cache[p.str] = p

        # Extra samples, previously already contained in cache,
        # that were geneated during the attempt to get
        # batch_size new samples for expensive reward evaluation
        if self.policy.valid_extended_batch:
            self.policy.valid_extended_batch = False
            n_extra = self.policy.extended_batch[0]
            if n_extra > 0:
                extra_programs = [from_tokens(a) for a in
                                  self.policy.extended_batch[1]]
                # Concatenation is fine because rnn_policy.sample_novel()
                # already made sure that offline batch and extended batch
                # are padded to the same trajectory length
                actions = np.concatenate([actions, self.policy.extended_batch[1]])
                obs = np.concatenate([obs, self.policy.extended_batch[2]])
                priors = np.concatenate([priors, self.policy.extended_batch[3]])
                programs = programs + extra_programs

        self.nevals += self.batch_size + n_extra

        # Run GP seeded with the current batch, returning elite samples
        if self.gp_controller is not None:
            deap_programs, deap_actions, deap_obs, deap_priors = self.gp_controller(actions)
            self.nevals += self.gp_controller.nevals

            # Pad AOP if different sized
            if actions.shape[1] < deap_actions.shape[1]:
                # If RL shape is smaller than GP then pad
                pad_length = deap_actions.shape[1] - actions.shape[1]
                actions, obs, priors = pad_action_obs_priors(actions, obs, priors, pad_length)
            elif actions.shape[1] > deap_actions.shape[1]:
                # If GP shape is smaller than RL then pad
                pad_length = actions.shape[1] - deap_actions.shape[1]
                deap_actions, deap_obs, deap_priors = pad_action_obs_priors(deap_actions, deap_obs, deap_priors, pad_length)

            # Combine RNN and deap programs, actions, obs, and priors
            programs = programs + deap_programs
            actions = np.append(actions, deap_actions, axis=0)
            obs = np.append(obs, deap_obs, axis=0)
            priors = np.append(priors, deap_priors, axis=0)

        # Compute rewards in parallel
        if self.pool is not None:
            # Filter programs that need reward computing
            programs_to_optimize = list(set([p for p in programs if "r" not in p.__dict__]))
            pool_p_dict = { p.str : p for p in self.pool.map(work, programs_to_optimize) }
            programs = [pool_p_dict[p.str] if "r" not in p.__dict__  else p for p in programs]
            # Make sure to update cache with new programs
            Program.cache.update(pool_p_dict)

        # Compute rewards (or retrieve cached rewards)
        r = np.array([p.r for p in programs])

        # Back up programs to save them properly later
        controller_programs = programs.copy() if self.logger.save_token_count else None

        # Need for Vanilla Policy Gradient (epsilon = null)
        l           = np.array([len(p.traversal) for p in programs])
        s           = [p.str for p in programs] # Str representations of Programs
        on_policy   = np.array([p.originally_on_policy for p in programs])
        invalid     = np.array([p.invalid for p in programs], dtype=bool)

        if self.logger.save_positional_entropy:
            positional_entropy = np.apply_along_axis(empirical_entropy, 0, actions)

        if self.logger.save_top_samples_per_batch > 0:
            # sort in descending order: larger rewards -> better solutions
            sorted_idx = np.argsort(r)[::-1]
            top_perc = int(len(programs) * float(self.logger.save_top_samples_per_batch))
            for idx in sorted_idx[:top_perc]:
                top_samples_per_batch.append([self.iteration, r[idx], repr(programs[idx])])

        # Store in variables the values for the whole batch (those variables will be modified below)
        r_full = r
        l_full = l
        s_full = s
        actions_full = actions
        invalid_full = invalid
        r_max = np.max(r)

        """
        Apply risk-seeking policy gradient: compute the empirical quantile of
        rewards and filter out programs with lesser reward.
        """
        if self.epsilon is not None and self.epsilon < 1.0:
            # Compute reward quantile estimate
            if self.use_memory: # Memory-augmented quantile
                # Get subset of Programs not in buffer
                unique_programs = [p for p in programs \
                                   if p.str not in self.memory_queue.unique_items]
                N = len(unique_programs)

                # Get rewards
                memory_r = self.memory_queue.get_rewards()
                sample_r = [p.r for p in unique_programs]
                combined_r = np.concatenate([memory_r, sample_r])

                # Compute quantile weights
                memory_w = self.memory_queue.compute_probs()
                if N == 0:
                    print("WARNING: Found no unique samples in batch!")
                    combined_w = memory_w / memory_w.sum() # Renormalize
                else:
                    sample_w = np.repeat((1 - memory_w.sum()) / N, N)
                    combined_w = np.concatenate([memory_w, sample_w])

                # Quantile variance/bias estimates
                if self.memory_threshold is not None:
                    print("Memory weight:", memory_w.sum())
                    if memory_w.sum() > self.memory_threshold:
                        quantile_variance(self.memory_queue, self.policy, self.batch_size, self.epsilon, self.iteration)

                # Compute the weighted quantile
                quantile = weighted_quantile(values=combined_r, weights=combined_w, q=1 - self.epsilon)

            else: # Empirical quantile
                quantile = np.quantile(r, 1 - self.epsilon, interpolation="higher")

            # Filter quantities whose reward >= quantile
            keep = r >= quantile
            l = l[keep]
            s = list(compress(s, keep))
            invalid = invalid[keep]
            r = r[keep]
            programs  = list(compress(programs, keep))
            actions = actions[keep, :]
            obs = obs[keep, :, :]
            priors = priors[keep, :, :]
            on_policy = on_policy[keep]

        # Clip bounds of rewards to prevent NaNs in gradient descent
        r = np.clip(r, -1e6, 1e6)

        # Compute baseline
        # NOTE: pg_loss = tf.reduce_mean((self.r - self.baseline) * neglogp, name="pg_loss")
        if self.baseline == "ewma_R":
            ewma = np.mean(r) if ewma is None else self.alpha*np.mean(r) + (1 - self.alpha)*ewma
            b = ewma
        elif self.baseline == "R_e": # Default
            ewma = -1
            b = quantile
        elif self.baseline == "ewma_R_e":
            ewma = np.min(r) if ewma is None else self.alpha*quantile + (1 - self.alpha)*ewma
            b = ewma
        elif self.baseline == "combined":
            ewma = np.mean(r) - quantile if ewma is None else self.alpha*(np.mean(r) - quantile) + (1 - self.alpha)*ewma
            b = quantile + ewma

        # Compute sequence lengths
        lengths = np.array([min(len(p.traversal), self.policy.max_length)
                            for p in programs], dtype=np.int32)

        # Create the Batch
        sampled_batch = Batch(actions=actions, obs=obs, priors=priors,
                              lengths=lengths, rewards=r, on_policy=on_policy)

        # Update and sample from the priority queue
        if self.priority_queue is not None:
            self.priority_queue.push_best(sampled_batch, programs)
            pqt_batch = self.priority_queue.sample_batch(self.policy_optimizer.pqt_batch_size)
            # Train the policy
            summaries = self.policy_optimizer.train_step(b, sampled_batch, pqt_batch)
        else:
            pqt_batch = None
            # Train the policy
            summaries = self.policy_optimizer.train_step(b, sampled_batch)

        # Walltime calculation for the iteration
        iteration_walltime = time.time() - start_time

        # Update the memory queue
        if self.memory_queue is not None:
            self.memory_queue.push_batch(sampled_batch, programs)

        # Update new best expression
        if r_max > self.r_best:
            self.r_best = r_max
            self.p_r_best = programs[np.argmax(r)]

            # Print new best expression
            if self.verbose or self.debug:
                print("[{}] Training iteration {}, current best R: {:.4f}".format(get_duration(start_time), self.iteration + 1, self.r_best))
                print("\n\t** New best")
                self.p_r_best.print_stats()

        # Collect sub-batch statistics and write output
        self.logger.save_stats(r_full, l_full, actions_full, s_full,
                               invalid_full, r, l, actions, s, s_history,
                               invalid, self.r_best, r_max, ewma, summaries,
                               self.iteration, b, iteration_walltime,
                               self.nevals, controller_programs,
                               positional_entropy, top_samples_per_batch)


        # Stop if early stopping criteria is met
        if self.early_stopping and self.p_r_best.evaluate.get("success"):
            print("[{}] Early stopping criteria met; breaking early.".format(get_duration(start_time)))
            self.done = True

        if self.verbose and (self.iteration + 1) % 10 == 0:
            print("[{}] Training iteration {}, current best R: {:.4f}".format(get_duration(start_time), self.iteration + 1, self.r_best))

        if self.debug >= 2:
            print("\nParameter means after iteration {}:".format(self.iteration + 1))
            self.print_var_means()

        if self.nevals >= self.n_samples:
            self.done = True

        # Increment the iteration counter
        self.iteration += 1

    def save(self, save_path):
        """
        Save the state of the Trainer.
        """

        state_dict = {
            "nevals" : self.nevals,
            "iteration" : self.iteration,
            "r_best" : self.r_best,
            "p_r_best_tokens" : self.p_r_best.tokens.tolist() if self.p_r_best is not None else None
        }
        with open(save_path, 'w') as f:
            json.dump(state_dict, f)

        print("Saved Trainer state to {}.".format(save_path))

    def load(self, load_path):
        """
        Load the state of the Trainer.
        """

        with open(load_path, 'r') as f:
            state_dict = json.load(f)

        # Load nevals and iteration from savestate
        self.nevals = state_dict["nevals"]
        self.iteration = state_dict["iteration"]

        # Load r_best and p_r_best
        if state_dict["p_r_best_tokens"] is not None:
            tokens = np.array(state_dict["p_r_best_tokens"], dtype=np.int32)
            self.p_r_best = from_tokens(tokens)
        else:
            self.p_r_best = None

        print("Loaded Trainer state from {}.".format(load_path))

In [ ]:
#checkpoint

class Checkpoint():
    """
    A helper class to checkpoint models.

    Methods
    -------
    save
        Save a checkpoint.

    load
        Load from a given checkpoint.

    update
        Maybe save a checkpoint depending on frequency configuration.
    """

    def __init__(self, model, load_path=None, save_freq=23, units="hours",
                 save_on_done=False):
        """
        model : dso.DeepSymbolicOptimizer
            The model to checkpoint.

        load_path : str or None
            Path to initial checkpoint directory to load. If None, do not start from
            checkpoint.

        save_freq : float or None
            The frequency at which to save a checkpoint. If None, non-final checkpoints
            will not be automatically saved.

        units : str
            The units of save_freq. Supports "hours", "minutes", "seconds", "iterations".

        save_on_done : bool
            Whether to save a final checkpoint upon reaching model.trainer.done.
        """

        self.model = model
        if model.save_path is not None:
            self.checkpoint_dir = os.path.join(model.save_path, "checkpoint")
            os.makedirs(self.checkpoint_dir, exist_ok=True)
        else:
            self.checkpoint_dir = None

        # Create the Saver
        self.saver = tf.train.Saver()

        # Load from existing checkpoint, if given
        if load_path is not None:
            self.load(load_path)

        # Setup time-based checkpointing
        if save_freq is not None and units in ["hours", "minutes", "seconds"]:
            if units == "hours":
                self.dt = timedelta(hours=save_freq)
            elif units == "minutes":
                self.dt = timedelta(minutes=save_freq)
            elif units == "seconds":
                self.dt = timedelta(seconds=save_freq)
            self.next_save_time = datetime.now() + self.dt
        else:
            self.next_save_time = None
            self.dt = None

        # Setup iteration-based checkpointing
        if save_freq is not None and units == "iterations":
            self.save_freq_iters = save_freq
        else:
            self.save_freq_iters = None

        self.save_on_done = save_on_done

    def update(self):
        """
        Maybe a save a checkpoint, depending on configuration. This should be called
        each iteration, i.e. after model.trainer.run_one_step().
        """

        # Save final checkpoint if done
        if self.save_on_done and self.model.trainer.done:
            self.save()

        # Save if time-based frequency is met
        elif self.next_save_time is not None and datetime.now() > self.next_save_time:
            self.save()
            self.next_save_time = datetime.now() + self.dt

        # Save if iteration-based frequency is met
        elif self.save_freq_iters is not None and (self.model.trainer.iteration % self.save_freq_iters) == 0:
            self.save()

    def save(self, save_path=None):
        """
        Save a checkpoint.

        Parameters
        ----------
        save_path : str or None
            Directory in which to save checkpoint. If None, save in:
            <self.model.save_path>/checkpoint/checkpoint_<timestamp>.
        """

        # Determine the save path
        if save_path is None:
            assert self.checkpoint_dir is not None, "Cannot support automated checkpointing with model.save_dir=None."
            timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            save_path = os.path.join(self.checkpoint_dir,
                                    "checkpoint_{}".format(timestamp))
        if os.path.exists(save_path):
            paths = os.listdir(os.path.dirname(save_path))
            paths = [path for path in paths if path.startswith(os.path.basename(save_path))]
            save_path += "_{}".format(len(paths))
        os.makedirs(save_path, exist_ok=False)

        # Save the TensorFlow graph
        # print("Saving TensorFlow graph...")
        tf_save_path = os.path.join(save_path, "tf")
        self.saver.save(self.model.sess, tf_save_path)

        # Save the Trainer
        # print("Saving Trainer...")
        trainer_save_path = os.path.join(save_path, "trainer.json")
        self.model.trainer.save(trainer_save_path)

        # Save the priority queue, if applicable
        # TBD: This should be in self.model.trainer.save or self.model.trainer.policy_optimizer.save after refactoring PolicyOptimizers to handle their own bookkeeping
        if self.model.trainer.priority_queue is not None:
            priority_queue_save_path = os.path.join(save_path, "priority_queue.npz")
            self.model.trainer.priority_queue.save(priority_queue_save_path)

        # Save the cache
        # print("Saving cache...")
        # TBD: Abstract into cache saving function
        cache_save_path = os.path.join(save_path, "cache.csv")
        cache_programs = Program.cache.values()
        cache_tokens = [",".join(map(str, p.tokens.tolist())) for p in cache_programs]
        cache_rewards = [p.r for p in cache_programs]
        cache_data = { "tokens" : cache_tokens, "rewards" : cache_rewards }
        cache_df = pd.DataFrame(cache_data)
        cache_df.to_csv(cache_save_path, index=False)

        # Save the extra samples that were produced while attempting to
        # generate a batch of new and unique samples
        if self.model.trainer.policy.valid_extended_batch:
            self.model.trainer.policy.valid_extended_batch = False
            batch_save_path = os.path.join(save_path, "batch.npz")
            with open(batch_save_path, 'wb') as f:
                np.savez(f, self.model.trainer.policy.extended_batch)

    def load(self, load_path):
        """
        Load model state from checkpoint.

        Parameters
        ----------
        load_path : str
            Checkpoint directory to load model state.
        """

        # Load the TensorFlow graph
        if self.model.sess is None:
            self.model.setup()
        tf_load_path = os.path.join(load_path, "tf")
        self.saver.restore(self.model.sess, tf_load_path)

        # Load the Trainer
        # print("Loading Trainer...")
        trainer_load_path = os.path.join(load_path, "trainer.json")
        self.model.trainer.load(trainer_load_path)

        # Load the priority queue, if applicable
        # TBD: This should be in self.model.trainer.load or self.model.trainer.policy_optimizer.load after refactoring PolicyOptimizers to handle their own bookkeeping
        if self.model.trainer.priority_queue is not None:
            priority_queue_load_path = os.path.join(load_path, "priority_queue.npz")
            self.model.trainer.priority_queue.load(priority_queue_load_path)

        # Load the cache
        # print("Loading cache...")
        cache_load_path = os.path.join(load_path, "cache.csv")
        cache_df = pd.read_csv(cache_load_path)
        cache_df["tokens"] = cache_df["tokens"].str.split(",")
        programs = [from_tokens(np.array(tokens, dtype=np.int32)) for tokens in cache_df["tokens"]]
        for p, r in zip(programs, cache_df["rewards"]):
            p.r = r

        # Load the extra samples
        batch_save_path = os.path.join(load_path, "batch.npz")
        if os.path.isfile(batch_save_path):
            npzfile = np.load(batch_save_path, allow_pickle=True)
            self.model.trainer.policy.extended_batch = npzfile['arr_0']
            self.model.trainer.policy.valid_extended_batch = True

In [ ]:
from io import StringIO, BytesIO
import shutil
from collections import defaultdict

In [ ]:
#train_stats
#These functions are defined globally so they are pickleable and can be used by Pool.map
def hof_work(p):
    return [p.r, p.on_policy_count, p.off_policy_count, repr(p.sympy_expr), repr(p), p.evaluate]

def pf_work(p):
    return [p.complexity, p.r, p.on_policy_count, p.off_policy_count, repr(p.sympy_expr), repr(p), p.evaluate]


class StatsLogger():
    """ Class responsible for dealing with output files of training statistics.
        It encapsulates all outputs to files."""

    def __init__(self, sess, output_file, save_summary=False, save_all_iterations=False, hof=100,
                 save_pareto_front=True, save_positional_entropy=False, save_top_samples_per_batch=0,
                 save_cache=False, save_cache_r_min=0.9, save_freq=1, save_token_count=False):

        """"
        sess : tf.Session
            TenorFlow Session object (used for generating summary files)

        output_file : str
            Filename to write results for each iteration.

        save_summary : bool, optional
            Whether to write TensorFlow summaries.

        save_all_iterations : bool, optional
            Whether to save statistics for all programs for each iteration.

        hof : int or None, optional
            Number of top Programs to evaluate after training.

        save_pareto_front : bool, optional
            If True, compute and save the Pareto front at the end of training.

        save_positional_entropy : bool, optional
            Whether to save evolution of positional entropy for each iteration.

        save_top_samples_per_batch : float, optional
            Whether to store the X% (float) top-performer samples in every batch to a csv file.

        save_cache : bool
            Whether to save the str, count, and r of each Program in the cache.

        save_cache_r_min : float or None
            If not None, only keep Programs with r >= r_min when saving cache.

        save_freq : int or None
            Statistics are flushed to file every save_freq iterations (default == 1). If < 0, uses save_freq = inf

        save_token_count : bool
            Wether to count used tokens in each iteration
        """
        self.sess = sess
        self.output_file = output_file
        self.save_summary = save_summary
        self.save_all_iterations = save_all_iterations
        self.hof = hof
        self.save_pareto_front = save_pareto_front
        self.save_positional_entropy = save_positional_entropy
        self.save_top_samples_per_batch = save_top_samples_per_batch
        self.save_cache = save_cache
        self.save_cache_r_min = save_cache_r_min
        self.save_token_count = save_token_count
        self.all_r = []   # save all R separately to keep backward compatibility with a generated file.

        if save_freq is None:
            self.buffer_frequency = 1
        elif save_freq < 1:
            self.buffer_frequency = float('inf')
        else:
            self.buffer_frequency = save_freq

        self.buffer_iteration_stats = StringIO() # Buffer for iteration statistics
        self.buffer_all_programs = StringIO() # Buffer for the statistics for all programs.
        self.buffer_token_stats = StringIO() # Buffer for iteration statistics
        self.buffer_top_samples = StringIO() # Buffer for top samples per batch
        self.buffer_pos_entropy = BytesIO()  # Buffer for positional entropy

        self.setup_output_files()

    def setup_output_files(self):
        """
        Opens and prepares all output log files controlled by this class.
        """
        if self.output_file is not None:
            os.makedirs(os.path.dirname(self.output_file), exist_ok=True)
            prefix, _ = os.path.splitext(self.output_file)
            self.all_r_output_file = "{}_all_r.npy".format(prefix)
            self.all_info_output_file = "{}_all_info.csv".format(prefix)
            self.hof_output_file = "{}_hof.csv".format(prefix)
            self.pf_output_file = "{}_pf.csv".format(prefix)
            self.positional_entropy_output_file = "{}_positional_entropy.npy".format(prefix)
            self.top_samples_per_batch_output_file = "{}_top_samples_per_batch.csv".format(prefix)
            self.cache_output_file = "{}_cache.csv".format(prefix)
            self.token_counter_output_file = "{}_token_count.csv".format(prefix)
            with open(self.output_file, 'w') as f:
                # r_best : Maximum across all iterations so far
                # r_max : Maximum across this iteration's batch
                # r_avg_full : Average across this iteration's full batch (before taking epsilon subset)
                # r_avg_sub : Average across this iteration's epsilon-subset batch
                # n_unique_* : Number of unique Programs in batch
                # n_novel_* : Number of never-before-seen Programs per batch
                # a_ent_* : Empirical positional entropy across sequences averaged over positions
                # invalid_avg_* : Fraction of invalid Programs per batch
                # baseline: Baseline value used for training
                # time: time used to learn in this iteration (in seconds)
                headers = ["r_best",
                           "r_max",
                           "r_avg_full",
                           "r_avg_sub",
                           "l_avg_full",
                           "l_avg_sub",
                           "ewma",
                           "n_unique_full",
                           "n_unique_sub",
                           "n_novel_full",
                           "n_novel_sub",
                           "a_ent_full",
                           "a_ent_sub",
                           "invalid_avg_full",
                           "invalid_avg_sub",
                           "baseline",
                           "time",
                           "nevals"]
                f.write("{}\n".format(",".join(headers)))
            if self.save_all_iterations:
                with open(self.all_info_output_file, 'w') as f:
                    # iteration : The iteration in which this line was saved
                    # r : reward for this program
                    # l : length of the program
                    # invalid : if the program is invalid
                    headers = ["iteration",
                                "r",
                                "l",
                                "invalid"]
                    f.write("{}\n".format(",".join(headers)))
            if self.save_token_count:
                with open(self.token_counter_output_file, 'w') as f:
                    headers = [str(token) for token in Program.library.tokens]
                    f.write("{}\n".format(",".join(headers)))

        else:
            self.all_r_output_file = None
            self.all_info_output_file = None
            self.hof_output_file = None
            self.pf_output_file = None
            self.positional_entropy_output_file = None
            self.top_samples_per_batch_output_file = None
            self.cache_output_file = None
            self.token_counter_output_file = None

        # Create summary writer
        if self.save_summary:
            if self.output_file is not None:
                summary_dir = "{}_summary".format(prefix)
            else:
                timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
                summary_dir = os.path.join("summary", timestamp)
            self.summary_writer = tf.summary.FileWriter(summary_dir, self.sess.graph)
        else:
            self.summary_writer = None

    def save_stats(self, r_full, l_full, actions_full, s_full, invalid_full, r,
                   l, actions, s, s_history, invalid, r_best, r_max, ewma,
                   summaries, iteration, baseline, iteration_walltime, nevals,
                   programs, positional_entropy, top_samples_per_batch):

        """
        Computes and saves all statistics that are computed for every time step. Depending on the value of
            self.buffer_frequency, the statistics might be instead saved in a buffer before going to disk.
        :param r_full: The reward of all programs
        :param l_full: The length of all programs
        :param actions_full: all actions sampled this step
        :param s_full: String representation of all programs sampled this step.
        :param invalid_full: boolean for all programs sampled showing if they are invalid
        :param r: r_full excluding the ones where keep=false
        :param l: l_full excluding the ones where keep=false
        :param actions: actions_full excluding the ones where keep=false
        :param s: s_full excluding the ones where keep=false
        :param s_history: String representation of programs in cache excluding s_full
        :param invalid: invalid_full excluding the ones where keep=false
        :param r_best: reward from the all time best program so far
        :param r_max: reward from the best program in this iteration
        :param ewma: Exponentially Weighted Moving Average weight that might be used for baseline computation
        :param summaries: Sumarries returned by the Controller this step
        :param iteration: This iteration id
        :param baseline: baseline value used for training
        :param iteration_walltime: time taken to process this iteration
        :param nevals: number of function evaluations
        :param programs: A batch of controller programs
        :param positional_entropy: current positional_entropy
        :param top_samples_per_batch: list containing top solutions on the last batch
        """
        iteration = iteration + 1 # Change from 0- to 1-based indexing
        if self.output_file is not None:
            r_avg_full = np.mean(r_full)

            l_avg_full = np.mean(l_full)
            a_ent_full = np.mean(np.apply_along_axis(empirical_entropy, 0, actions_full))
            n_unique_full = len(set(s_full))
            n_novel_full = len(set(s_full).difference(s_history))
            invalid_avg_full = np.mean(invalid_full)

            r_avg_sub = np.mean(r)
            l_avg_sub = np.mean(l)
            a_ent_sub = np.mean(np.apply_along_axis(empirical_entropy, 0, actions))
            n_unique_sub = len(set(s))
            n_novel_sub = len(set(s).difference(s_history))
            invalid_avg_sub = np.mean(invalid)
            stats = np.array([[
                r_best,
                r_max,
                r_avg_full,
                r_avg_sub,
                l_avg_full,
                l_avg_sub,
                ewma,
                n_unique_full,
                n_unique_sub,
                n_novel_full,
                n_novel_sub,
                a_ent_full,
                a_ent_sub,
                invalid_avg_full,
                invalid_avg_sub,
                baseline,
                iteration_walltime,
                nevals
            ]], dtype=np.float32)
            np.savetxt(self.buffer_iteration_stats, stats, delimiter=',')
        if self.save_all_iterations:
            all_iteration_stats = np.array([
                              [iteration] * len(r_full),
                              r_full,
                              l_full,
                              invalid_full
                              ]).transpose()
            df = pd.DataFrame(all_iteration_stats)
            df.to_csv(self.buffer_all_programs, mode='a', header=False, index=False, line_terminator='\n')

        # Collect stats about used tokens and write to buffer
        if self.save_token_count:
            self.write_token_count(programs)

        # summary writers have their own buffer
        if self.save_summary:
            self.summary_writer.add_summary(summaries, iteration)

        # Should the buffer be saved now?
        if iteration % self.buffer_frequency == 0:
            self.flush_buffers()

        #Backwards compatibility of all_r numpy file
        if self.save_all_iterations:
            self.all_r.append(r_full)

        if self.save_positional_entropy:
            #with open(self.buffer_pos_entropy, 'ab') as f:
                # saves only the last positional entropy
            np.save(self.buffer_pos_entropy, positional_entropy)

        if self.save_top_samples_per_batch > 0:
            df_topsamples = pd.DataFrame(top_samples_per_batch,
                                         columns=['Iteration', 'Reward', 'Sequence'])
            # only save to the file the current iteration
            header = iteration == 1
            df_topsamples.to_csv(self.buffer_top_samples, mode='a', index=False, header=header)

    def save_results(self, pool, n_samples):
        """
        Saves stats that are available only after all iterations are finished
        :param pool: Pool used to parallelize reward computation
        :param n_samples: Total number of samples
        """
        # First of all, saves any pending buffer
        self.flush_buffers()

        if self.save_all_iterations:
            # Check if all batches are the same size
            batch_sizes = np.array([batch.shape[0] for batch in self.all_r])
            if np.all(batch_sizes == batch_sizes[0]):
                all_r_padded = self.all_r
            else:
                max_batch_size = np.max([batch.shape[0] for batch in self.all_r])
                all_r_padded = []
                for batch in self.all_r:
                    n_pad = max_batch_size - batch.shape[0]
                    all_r_padded.append(np.pad(
                        batch, pad_width=(0,n_pad), mode='constant',
                        constant_values=-np.inf))
            #Kept all_r numpy file for backwards compatibility.
            with open(self.all_r_output_file, 'ab') as f:
                all_r = np.array(all_r_padded, dtype=np.float32)
                np.save(f, all_r)

        result = {}

        # Save the hall of fame
        if self.hof is not None and self.hof > 0:
            assert not Program.task.stochastic, "HOF only supported for deterministic Tasks."
            programs = list(Program.cache.values())  # All unique Programs found during training
            r = [p.r for p in programs]
            i_hof = np.argsort(r)[-self.hof:][::-1]  # Indices of top hof Programs
            hof = [programs[i] for i in i_hof]

            if pool is not None:
                results = pool.map(hof_work, hof)
            else:
                results = list(map(hof_work, hof))

            eval_keys = list(results[0][-1].keys())
            columns = ["r", "count_on_policy", "count_off_policy", "expression", "traversal"] + eval_keys
            hof_results = [result[:-1] + [result[-1][k] for k in eval_keys] for result in results]
            df = pd.DataFrame(hof_results, columns=columns)
            if self.hof_output_file is not None:
                print("Saving Hall of Fame to {}".format(self.hof_output_file))
                df.to_csv(self.hof_output_file, header=True, index=False)

        # Save cache
        if self.save_cache and Program.cache:
            print("Saving cache to {}".format(self.cache_output_file))
            cache_data = [(repr(p), p.on_policy_count, p.off_policy_count, p.r) for p in Program.cache.values()]
            df_cache = pd.DataFrame(cache_data)
            df_cache.columns = ["str", "count_on_policy", "count_off_policy", "r"]
            if self.save_cache_r_min is not None:
                df_cache = df_cache[df_cache["r"] >= self.save_cache_r_min]
            df_cache.to_csv(self.cache_output_file, header=True, index=False)

        # Compute the pareto front
        if self.save_pareto_front:

            assert not Program.task.stochastic, "Pareto front only supported for deterministic Tasks."

            all_programs = list(Program.cache.values())
            costs = np.array([(p.complexity, -p.r) for p in all_programs])
            pareto_efficient_mask = is_pareto_efficient(costs)  # List of bool
            pf = list(compress(all_programs, pareto_efficient_mask))
            pf.sort(key=lambda p: p.complexity) # Sort by complexity

            if pool is not None:
                results = pool.map(pf_work, pf)
            else:
                results = list(map(pf_work, pf))

            eval_keys = list(results[0][-1].keys())
            columns = ["complexity", "r", "count_on_policy", "count_off_policy", "expression", "traversal"] + eval_keys
            pf_results = [result[:-1] + [result[-1][k] for k in eval_keys] for result in results]
            df = pd.DataFrame(pf_results, columns=columns)
            if self.pf_output_file is not None:
                print("Saving Pareto Front to {}".format(self.pf_output_file))
                df.to_csv(self.pf_output_file, header=True, index=False)

            # Look for a success=True case within the Pareto front
            for p in pf:
                if p.evaluate.get("success"):
                    p_final = p
                    break

        #Save error summaries
        # Print error statistics of the cache
        n_invalid = 0
        error_types = defaultdict(lambda: 0)
        error_nodes = defaultdict(lambda: 0)

        result = {}
        for p in Program.cache.values():
            if p.invalid:
                count = p.off_policy_count + p.on_policy_count
                n_invalid += count
                error_types[p.error_type] += count
                error_nodes[p.error_node] += count

        if n_invalid > 0:
            print("Invalid expressions: {} of {} ({:.1%}).".format(n_invalid, n_samples,
                                                                    n_invalid / n_samples))
            print("Error type counts:")
            for error_type, count in error_types.items():
                print("  {}: {} ({:.1%})".format(error_type, count, count / n_invalid))
                result["error_"+str(error_type)] = count
            print("Error node counts:")
            for error_node, count in error_nodes.items():
                print("  {}: {} ({:.1%})".format(error_node, count, count / n_invalid))
                result["error_node_" + str(error_node)] = count

        result['n_samples'] = n_samples
        result['n_cached'] = len(Program.cache)
        return result

    def write_token_count(self, programs):
        token_counter = {token: 0 for token in Program.library.names}
        for program in programs:
            for token in program.traversal:
                token_counter[token.name] += 1
        stats = np.array([[
            token_counter[token] for token in token_counter.keys()
        ]], dtype=np.int)
        np.savetxt(self.buffer_token_stats, stats, fmt='%i', delimiter=',')

    def flush_buffers(self):
        """Write all available buffers to file."""
        if self.output_file is not None:
            self.buffer_iteration_stats = self.flush_buffer(
                self.buffer_iteration_stats, self.output_file)
        if self.save_all_iterations:
            self.buffer_all_programs = self.flush_buffer(
                self.buffer_all_programs, self.all_info_output_file)
        if self.save_token_count:
            self.buffer_token_stats = self.flush_buffer(
                self.buffer_token_stats, self.token_counter_output_file)
        if self.save_top_samples_per_batch > 0 and self.output_file is not None:
            self.buffer_top_samples = self.flush_buffer(
                self.buffer_top_samples, self.top_samples_per_batch_output_file)
        if self.save_positional_entropy:
            self.buffer_pos_entropy = self.flush_buffer(
                self.buffer_pos_entropy, self.positional_entropy_output_file, byte_buffer=True)
        if self.summary_writer:
            self.summary_writer.flush()

    def flush_buffer(self, buffer, output_file, byte_buffer=False):
        """
        Write specific buffer to corresponding output file.
        :param buffer: Buffer that will be flushed
        :param output_file: File to which the buffer will be flushed
        :param byte_buffer: If true, assume the buffer is in String format,
            otherwise, that it is in bytes format
        """
        mode = 'ab' if byte_buffer else 'a'
        with open(output_file, mode) as f:
            buffer.seek(0)
            shutil.copyfileobj(buffer, f, -1)
        # clear buffer
        return BytesIO() if byte_buffer else StringIO()

In [ ]:
#dyn_rnn
import tensorflow as tf
from tensorflow.keras.layers import RNN, LSTM, Dropout, Dense
from tensorflow.compat.v1.nn.rnn_cell import BasicRNNCell

from tensorflow.keras.losses import SparseCategoricalCrossentropy

class LanguageModel(object):
    def __init__(self, vocabulary_size, embedding_size, num_layers, num_hidden, mode='train'):
        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.num_hidden = num_hidden

        self.x = tf.compat.v1.placeholder(tf.int32, [None, None], name="x") # whole seq + seq len
        self.keep_prob = tf.compat.v1.placeholder(tf.float32, [], name="keep_prob")
        self.batch_size = tf.compat.v1.shape(self.x)[0]

        if mode == 'train':
            self.lm_input = self.x[:, :-2]
            self.seq_len = self.x[:, -1]
        elif mode == 'predict':
            self.lm_input = self.x[:,:]
            self.seq_len = tf.reduce_sum(tf.sign(self.lm_input), 1)

        # Embedding (optional)
        with tf.name_scope("embedding"):
            init_embeddings = tf.random.uniform([vocabulary_size, self.embedding_size])
            embeddings = tf.compat.v1.get_variable("embeddings", initializer=init_embeddings)
            lm_input_emb = tf.nn.embedding_lookup(embeddings, self.lm_input)

        with tf.compat.v1.variable_scope("rnn"):
            def make_cell():
                cell = BasicRNNCell(self.num_hidden)  # Or use LSTM(...) for better performance
                cell = Dropout(1 - self.keep_prob)(cell)
                return cell

            cell = RNN([make_cell() for _ in range(self.num_layers)])

            self.initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)
            rnn_outputs, self.last_state = tf.nn.dynamic_rnn(
                cell=cell,
                initial_state=self.initial_state,
                inputs=lm_input_emb,  # Assuming you're using embeddings
                sequence_length=self.seq_len,
                dtype=tf.float32)

        with tf.name_scope("output"):
            self.logits = Dense(vocabulary_size)(rnn_outputs)  # Use tf.layers.dense for TF < 2.0

        with tf.name_scope("loss"):
            if mode == "train":
                target = self.x[:, 1:-1]
            elif mode == "predict":
                target = self.x[:, :]

            self.loss = SparseCategoricalCrossentropy(from_logits=True)(target, self.logits)


In [ ]:
#language_model_prior

class LanguageModelPrior(object):
    """
    Language model to get prior for DSO, given token.

    History of tokens of a sequence is holded as a state of language model.
    Usage: LanguageModelPrior.get_lm_prior(token)

    Parameters
    ----------
    dso_library: dso.library.Library
        Library used in main DSO model

    model_path: str
        Path to separately trained mathematical language model to use as prior

    lib_path: str
        Path to token library of mathematical language model

    embedding_size: int
    num_layers: int
    num_hidden: int
        Model architecture of loaded mathematical language model

    prob_sharing: bool
        Share probabilities among terminal tokens?
    """

    def __init__(self, dso_library,
                model_path="./language_model/model/saved_model",
                lib_path="./language_model/model/saved_model/word_dict.pkl",
                embedding_size=32, num_layers=1, num_hidden=256,
                prob_sharing=True
                ):

        self.dso_n_input_var = len(dso_library.input_tokens)
        self.prob_sharing = prob_sharing

        with open(lib_path, 'rb') as f:
            self.lm_token2idx = pickle.load(f)
        self.dso2lm, self.lm2dso = self.set_lib_to_lib(dso_library)

        self.language_model = LanguageModel(len(self.lm_token2idx), embedding_size, num_layers, num_hidden, mode='predict')
        self.lsess = self.load_model(model_path)
        self.next_state = None
        self._zero_state = np.zeros(num_hidden, dtype=np.float32)

    def load_model(self, saved_language_model_path):
        sess = tf.compat.v1.Session()
        saver = tf.train.Saver()
        saver.restore(sess,tf.train.latest_checkpoint(saved_language_model_path))
        return sess

    def set_lib_to_lib(self, dso_library):
        """match token libraries of DSO and lm (LanguageModel)"""

        # dso token -> lm token
        dso2lm = [self.lm_token2idx['TERMINAL']] * self.dso_n_input_var
        dso2lm += [self.lm_token2idx[t.name.lower()] for t in dso_library.tokens if t.input_var is None] # ex) [1,1,1,2,3,4,5,6,7,8,9], len(dso2lm) = len(library of dso)

        # lm token -> DSO token
        lm2dso = {lm_idx: i for i, lm_idx in enumerate(dso2lm)}

        # TODO: if DSO token missing in lm token library

        return dso2lm, lm2dso

    def get_lm_prior(self, next_input):
        """return language model prior based on given current token"""

        # set feed_dict
        next_input = np.array(self.dso2lm)[next_input]  # match library with DSO
        next_input = np.array([next_input])

        if self.next_state is None: # first input of a sequence
            # For dynamic_rnn, not passing language_model.initial_state == passing zero_state.
            # Here, explicitly passing zero_state
            self.next_state = np.atleast_2d(self._zero_state) # initialize the cell

        feed_dict = {self.language_model.x: next_input, self.language_model.keep_prob: 1.0, self.language_model.initial_state: self.next_state}

        # get language model prior
        self.next_state, lm_logit = self.lsess.run([self.language_model.last_state, self.language_model.logits], feed_dict=feed_dict)

        if self.prob_sharing is True:
            # sharing probability among tokens in same group (e.g., TERMINAL to multiple variables)
            lm_logit[:, :, self.lm_token2idx['TERMINAL']] = lm_logit[:, :, self.lm_token2idx['TERMINAL']] - np.log(self.dso_n_input_var)
        lm_prior = lm_logit[0, :, self.dso2lm]
        lm_prior = np.transpose(lm_prior) # make its shape to (batch size, dso size)

        return lm_prior

In [ ]:
#prior
def make_prior(library, config_prior):
    """Factory function for JointPrior object."""

    config_prior = deepcopy(config_prior)

    prior_dict = {
        "relational" : RelationalConstraint,
        "length" : LengthConstraint,
        "repeat" : RepeatConstraint,
        "inverse" : InverseUnaryConstraint,
        "trig" : TrigConstraint,
        "const" : ConstConstraint,
        "no_inputs" : NoInputsConstraint,
        "soft_length" : SoftLengthPrior,
        "uniform_arity" : UniformArityPrior,
        "domain_range" : DomainRangeConstraint,
        "language_model" : LanguageModelPrior,
        "multi_discrete" : MultiDiscreteConstraint
    }

    count_constraints = config_prior.pop("count_constraints", False)

    priors = []
    warn_messages = []
    for prior_type, prior_args in config_prior.items():
        if prior_type in prior_dict:
            prior_class = prior_dict[prior_type]
        else:
            # Tries to import custom priors
            prior_class = import_custom_source(prior_type)

        if isinstance(prior_args, dict):
            prior_args = [prior_args]
        for single_prior_args in prior_args:
            # Attempt to build the Prior. Any Prior can fail if it references a
            # Token not in the Library.
            prior_is_enabled = single_prior_args.pop('on', False)
            if prior_is_enabled:
                try:
                    prior = prior_class(library, **single_prior_args)
                    warn_message = prior.validate()
                except TokenNotFoundError:
                    prior = None
                    warn_message = "Uses Tokens not in the Library."
            else:
                prior = None
                warn_message = "Prior disabled."

            # Add warning context
            if warn_message is not None:
                warn_message = "Skipping invalid '{}' with arguments {}. " \
                    "Reason: {}" \
                    .format(prior_class.__name__, single_prior_args, warn_message)
                warn_messages.append(warn_message)

            # Add the Prior if there are no warnings
            if warn_message is None:
                priors.append(prior)

    # Turn PolyConstraint "on" if and only if "poly" token is in library
    if "poly" in library.names:
        priors.append(PolyConstraint(library))

    # Turn StateCheckerConstraint "on" if and only if StateChecker is in library
    if len(library.state_checker_tokens) > 0:
        priors.append(StateCheckerConstraint(library))

    joint_prior = JointPrior(library, priors, count_constraints)

    print("-- BUILDING PRIOR START -------------")
    print("\n".join(["WARNING: " + message for message in warn_messages]))
    print(joint_prior.describe())
    print("-- BUILDING PRIOR END ---------------\n")

    return joint_prior


class JointPrior():
    """A collection of joint Priors."""

    def __init__(self, library, priors, count_constraints=False):
        """
        Parameters
        ----------
        library : Library
            The Library associated with the Priors.

        priors : list of Prior
            The individual Priors to be joined.

        count_constraints : bool
            Whether to count the number of constrained tokens.
        """

        self.library = library
        self.L = self.library.L
        self.priors = priors
        assert all([prior.library is library for prior in priors]), \
            "All Libraries must be identical."

        # Name the priors, e.g. RepeatConstraint-0
        counts = defaultdict(lambda : -1)
        self.names = []
        for prior in self.priors:
            name = prior.__class__.__name__
            counts[name] += 1
            self.names.append("{}-{}".format(name, counts[name]))

        # Initialize variables for constraint count report
        self.do_count = count_constraints
        self.constraint_counts = [0] * len(self.priors)
        self.total_constraints = 0
        self.total_tokens = 0

        self.requires_parents_siblings = True # TBD: Determine

        # SPECIAL CASE: Set DomainRangeConstraint.max_length if constraining max length
        length_prior = None
        domain_range_prior = None
        for prior in self.priors:
            if isinstance(prior, LengthConstraint):
                length_prior = prior
            elif isinstance(prior, DomainRangeConstraint):
                domain_range_prior = prior
        if length_prior is not None and domain_range_prior is not None:
            domain_range_prior.max_length = length_prior.max

        self.describe()

    def initial_prior(self):
        combined_prior = np.zeros((self.L,), dtype=np.float32)
        for prior in self.priors:
            combined_prior += prior.initial_prior()
        return combined_prior

    def __call__(self, actions, parent, sibling, dangling, finished):

        # Filter out finished sequences to save compute
        unfinished = ~finished
        final_combined_prior = np.zeros((actions.shape[0], self.L), dtype=np.float32)
        unfinished_actions = actions[unfinished]
        unfinished_parent = parent[unfinished]
        unfinished_sibling = sibling[unfinished]
        unfinished_dangling = dangling[unfinished]

        # Sum the individual priors
        zero_prior = np.zeros((unfinished_actions.shape[0], self.L), dtype=np.float32)
        ind_priors = [zero_prior.copy() for _ in range(len(self.priors))]
        for i in range(len(self.priors)):
            ind_priors[i] += self.priors[i](unfinished_actions, unfinished_parent, unfinished_sibling, unfinished_dangling)
        combined_prior = sum(ind_priors) + zero_prior # TBD FIX HACK

        # Count number of constrained tokens per prior
        if self.do_count:
            self.total_tokens += unfinished_actions.shape[0] * self.library.L
            for i in range(len(self.constraint_counts)):
                self.constraint_counts[i] += np.count_nonzero(ind_priors[i] == -np.inf)
            self.total_constraints += np.count_nonzero(combined_prior == -np.inf)

        # Give status report if a prior contains all -inf
        collision_mask = np.all(np.isneginf(combined_prior), axis=1)
        if np.any(collision_mask):
            collisions = collision_mask.nonzero()[0] # Indices of collisions
            msg = []
            msg.append("ERROR in {}:".format(__file__))
            msg.append("Encountered collision(s) in prior. This occurs when a prior contains all -inf values. " +
                       "This typically indicates a logic error in a prior formulation, or a 'collision' when " +
                       "configuring multiple priors that are not always compatible. See the table(s) below " +
                       "for which priors caused each collision. X's indicate a value of -inf.")
            for i, collision in enumerate(collisions):
                msg.append("\n----- Collision {} of {} -----".format(i + 1, len(collisions)))
                msg.append("Traversal: {}".format(self.library.tokenize(unfinished_actions[collision])))
                table = PrettyTable(["Prior"] + self.library.names)
                for j, ind_prior in enumerate(ind_priors):
                    if np.any(np.isneginf(ind_prior[collision])):
                        name = self.priors[j].__class__.__name__
                        row = ["X" if x == -np.inf else "" for x in ind_prior[collision]]
                        table.add_row([name] + row)
                msg.append(repr(table))
            # msg.append("\nApplication will now exit.")
            print("\n".join(msg))
            # os._exit(1) # Bypass tensorflow exception-handling

        final_combined_prior[unfinished] = combined_prior

        return final_combined_prior

    def report_constraint_counts(self):
        if not self.do_count:
            return
        print("Constraint counts per prior:")
        for i, count in enumerate(self.constraint_counts):
            print("{}: {} ({:%})".format(self.names[i], count, count / self.total_tokens))
        print("All priors: {} ({:%})".format(self.total_constraints, self.total_constraints / self.total_tokens))

    def describe(self):
        message = "\n".join(prior.describe() for prior in self.priors)
        return message

    def is_violated(self, actions, parent, sibling):
        for prior in self.priors:
            if isinstance(prior, Constraint):
                if prior.is_violated(actions, parent, sibling):
                    return True
        return False

    def at_once(self, actions, parent, sibling):
        """
        Given a full sequence of actions, parents, and siblings, each of shape
        (batch, time), *retrospectively* compute what was the joint prior at all
        time steps. The combined prior has shape (batch, time, L).
        """

        B, T = actions.shape
        zero_prior = np.zeros((B, T, self.L), dtype=np.float32) # (batch, time, L)
        ind_priors = [zero_prior.copy() for _ in range(len(self.priors))] # i x (batch, time, L)

        if len(self.priors) == 0:
            return zero_prior

        # Set initial prior
        # Note: intial_prior() is already a combined prior, so we just set the
        # first individual prior, ind_priors[0].
        initial_prior = self.initial_prior() # Shape (L,)
        ind_priors[0][:, 0, :] = initial_prior # Broadcast to (batch, L)

        dangling = np.ones(B)
        for t in range(1, T): # For each time step
            # Update dangling based on previously sampled token
            dangling += self.library.arities[actions[:, (t - 1)]] - 1
            for i in range(len(self.priors)): # For each Prior
                # Compute the ith Prior at time step t
                prior = self.priors[i](actions[:, :t],
                                       parent[:, t],
                                       sibling[:, t],
                                       dangling) # Shape (batch, L)
                ind_priors[i][:, t, :] += prior

        # Combine all Priors
        combined_prior = sum(ind_priors) + zero_prior

        return combined_prior


class Prior():
    """Abstract class whose call method return logits."""

    def __init__(self, library):
        self.library = library
        self.L = library.L
        self.mask_val = -np.inf

    def validate(self):
        """
        Determine whether the Prior has a valid configuration. This is useful
        when other algorithmic parameters may render the Prior degenerate. For
        example, having a TrigConstraint with no trig Tokens.

        Returns
        -------
        message : str or None
            Error message if Prior is invalid, or None if it is valid.
        """

        return None

    def init_zeros(self, actions):
        """Helper function to generate a starting prior of zeros."""

        batch_size = actions.shape[0]
        prior = np.zeros((batch_size, self.L), dtype=np.float32)
        return prior

    def initial_prior(self):
        """
        Compute the initial prior, before any actions are selected.

        Returns
        -------
        initial_prior : array
            Initial logit adjustment before actions are selected. Shape is
            (self.L,) as it will be broadcast to batch size later.
        """

        return np.zeros((self.L,), dtype=np.float32)

    def __call__(self, actions, parent, sibling, dangling):
        """
        Compute the prior (logit adjustment) given the history of actions
        and the current observation.

        actions : np.ndarray (dtype=np.int32)
            History of all actions so far. Each action is a Library index.
            Shape is (batch_size, current_time).

        parent : np.ndarray (dtype=np.in32)
            Adjusted Library indices of parent of the token being selected.
            Shape is (batch_size,).

        sibling : np.ndarray (dtype=np.int32)
            Library indices of sibling of the token being selected.
            Shape is (batch_size,).

        dangling : np.ndarray (dtype=np.int32)
            Current number of dangling/unselected nodes.
            Shape is (batch_size,).

        Returns
        -------
        prior : np.ndarray (dtype=np.float32)
            Logit adjustment for selecting next action.
            Shape is (batch_size, self.L).
        """

        raise NotImplementedError

    def describe(self):
        """Describe the Prior."""

        return "{}: No description available.".format(self.__class__.__name__)


class Constraint(Prior):
    def __init__(self, library):
        Prior.__init__(self, library)

    def make_constraint(self, mask, tokens):
        """
        Generate the prior for a batch of constraints and the corresponding
        Tokens to constrain.

        For example, with L=5 and tokens=[1,2], a constrained row of the prior
        will be: [0.0, -np.inf, -np.inf, 0.0, 0.0].

        Parameters
        __________

        mask : np.ndarray, shape=(?,), dtype=np.bool_
            Boolean mask of samples to constrain.

        tokens : np.ndarray, dtype=np.int32
            Tokens to constrain.

        Returns
        _______

        prior : np.ndarray, shape=(?, L), dtype=np.float32
            Logit adjustment. Since these are hard constraints, each element is
            either 0.0 or -np.inf.
        """
        prior = np.zeros((mask.shape[0], self.L), dtype=np.float32)

        for t in tokens:
            prior[mask, t] = self.mask_val
        return prior

    def is_violated(self, actions, parent, sibling):
        """
        Given a set of actions, tells us if a prior constraint has been violated
        post hoc.

        This is a generic version that will run using the __call__ function so that one
        does not have to write a function twice for both DSO and Deap.

        >>>HOWEVER<<<

        Using this function is less optimal than writing a variant for Deap. So...

        If you create a constraint and find you use if often with Deap, you should go ahead and
        write the optimal version.

        Returns
        -------
        violated : Bool
        """

        caller          = inspect.getframeinfo(inspect.stack()[1][0])

        warnings.warn("{} ({}) {} : Using a slower version of constraint for Deap. You should write your own.".format(caller.filename, caller.lineno, type(self).__name__))

        assert len(actions.shape) == 2, "Only takes in one action at a time since this is how Deap will use it."
        assert actions.shape[0] == 1, "Only takes in one action at a time since this is how Deap will use it."
        dangling        = np.ones((1), dtype=np.int32)

        # For each step in time, get the prior
        for t in range(actions.shape[1]):
            dangling    += self.library.arities[actions[:,t]] - 1
            priors      = self.__call__(actions[:,:t], parent[:,t], sibling[:,t], dangling)
            # Does our action conflict with this prior?
            if priors[0, actions[0,t]] == -np.inf:
                return True

        return False

    def test_is_violated(self, actions, parent, sibling):
        r"""
            This allows one to call the generic version of "is_violated" for testing purposes
            from the derived classes even if they have an optimized version.
        """
        return Constraint.is_violated(self, actions, parent, sibling)


class RelationalConstraint(Constraint):
    """
    Class that constrains the following:

        Constrain (any of) `targets` from being the `relationship` of (any of)
        `effectors`.

    Parameters
    ----------
    targets : list of Tokens
        List of Tokens, all of which will be constrained if any of effectors
        are the given relationship.

    effectors : list of Tokens
        List of Tokens, any of which will cause all targets to be constrained
        if they are the given relationship.

    relationship : str
        The type of relationship to constrain.
        Supported options: "child", "descendant", "sibling", "uchild",
                           "lchild", "rchild"
    """

    def __init__(self, library, targets, effectors, relationship):
        Prior.__init__(self, library)
        self.targets = library.actionize(targets)
        self.effectors = library.actionize(effectors)
        self.relationship = relationship
        if self.relationship == "uchild":
            assert len(self.targets) == 1, "uchild RelationalConstraint" \
                "cannot be applied correctly if len(self.targets) > 1"
            unary_effectors = np.intersect1d(self.effectors,
                                             self.library.unary_tokens)
            self.adj_unary_effectors = library.parent_adjust[unary_effectors]
            self.adj_effectors = library.parent_adjust[self.effectors]

    def __call__(self, actions, parent, sibling, dangling):

        if self.relationship == "descendant":
            mask = ancestors(actions=actions,
                             arities=self.library.arities,
                             ancestor_tokens=self.effectors)
            prior = self.make_constraint(mask, self.targets)

        elif self.relationship == "child":
            parents = self.effectors
            adj_parents = self.library.parent_adjust[parents]
            mask = np.isin(parent, adj_parents)
            prior = self.make_constraint(mask, self.targets)

        elif self.relationship == "sibling":
            # The sibling relationship is reflexive: if A is a sibling of B,
            # then B is also a sibling of A. Thus, we combine two priors, where
            # targets and effectors are swapped.
            mask = np.isin(sibling, self.effectors)
            prior = self.make_constraint(mask, self.targets)
            mask = np.isin(sibling, self.targets)
            prior += self.make_constraint(mask, self.effectors)

        elif self.relationship == "uchild":
            # Case 1: parent is a unary effector
            mask = np.isin(parent, self.adj_unary_effectors)
            # Case 2: sibling is a target and parent is an effector
            mask += np.logical_and(np.isin(sibling, self.targets),
                                   np.isin(parent, self.adj_effectors))
            prior = self.make_constraint(mask, self.targets)

        elif self.relationship == "lchild":
            parents = self.effectors
            adj_parents = self.library.parent_adjust[parents]
            mask = np.logical_and(np.isin(parent, adj_parents),
                                  np.equal(sibling, self.library.EMPTY_SIBLING))
            prior = self.make_constraint(mask, self.targets)

        elif self.relationship == "rchild":
            parents = self.effectors
            adj_parents = self.library.parent_adjust[parents]
            mask = np.logical_and(np.isin(parent, adj_parents),
                                  np.not_equal(sibling, self.library.EMPTY_SIBLING))
            prior = self.make_constraint(mask, self.targets)

        return prior

    def is_violated(self, actions, parent, sibling):

        if self.relationship == "descendant":
            violated = jit_check_constraint_violation_descendant_with_target_tokens(
                actions, self.targets, self.effectors, self.library.binary_tokens, self.library.unary_tokens)

        elif self.relationship == "child":
            parents = self.effectors
            adj_parents = self.library.parent_adjust[parents]
            violated = jit_check_constraint_violation(actions, self.targets, parent, adj_parents)

        elif self.relationship == "sibling":
            violated = jit_check_constraint_violation(actions, self.targets, sibling, self.effectors)
            if not violated:
                violated = jit_check_constraint_violation(actions, self.effectors, sibling, self.targets)

        elif self.relationship == "uchild":
            unary_effectors = np.intersect1d(self.effectors, self.library.unary_tokens)
            adj_unary_effectors = self.library.parent_adjust[unary_effectors]
            adj_effectors = self.library.parent_adjust[self.effectors]
            violated = jit_check_constraint_violation_uchild(actions, parent, sibling, self.targets,
                                                     adj_unary_effectors, adj_effectors)

        return violated

    def validate(self):
        message = []
        if self.relationship in ["child", "descendant", "uchild", "lchild", "rchild"]:
            if np.isin(self.effectors, self.library.terminal_tokens).any():
                message = "{} relationship cannot have terminal effectors." \
                          .format(self.relationship.capitalize())
                return message
        if len(self.targets) == 0:
            message = "There are no target Tokens."
            return message
        if len(self.effectors) == 0:
            message = "There are no effector Tokens."
            return message
        return None

    def describe(self):

        targets = ", ".join([self.library.names[t] for t in self.targets])
        effectors = ", ".join([self.library.names[t] for t in self.effectors])
        relationship = {
            "child" : "a child",
            "sibling" : "a sibling",
            "descendant" : "a descendant",
            "uchild" : "the only unique child",
            "lchild" : "the left child",
            "rchild" : "the right child"
        }[self.relationship]
        message = "{}: [{}] cannot be {} of [{}]." \
                  .format(self.__class__.__name__, targets, relationship, effectors)
        return message


class TrigConstraint(RelationalConstraint):
    """Class that constrains trig Tokens from being the descendants of trig
    Tokens."""

    def __init__(self, library):
        targets = library.trig_tokens
        effectors = library.trig_tokens
        super(TrigConstraint, self).__init__(library=library,
                                             targets=targets,
                                             effectors=effectors,
                                             relationship="descendant")

    def is_violated(self, actions, parent, sibling):

        # Call a slightly faster descendant computation since target is the same as effectors
        return jit_check_constraint_violation_descendant_no_target_tokens(\
                actions, self.effectors, self.library.binary_tokens, self.library.unary_tokens)


class ConstConstraint(RelationalConstraint):
    """Class that constrains the const Token from being the only unique child
    of all non-terminal Tokens."""

    def __init__(self, library):
        targets = library.const_token
        effectors = np.concatenate([library.unary_tokens,
                                    library.binary_tokens])

        super(ConstConstraint, self).__init__(library=library,
                                              targets=targets,
                                              effectors=effectors,
                                              relationship="uchild")


class NoInputsConstraint(Constraint):
    """Class that constrains sequences without input variables.

    NOTE: This *should* be a special case of RepeatConstraint, but is not yet
    supported."""

    def __init__(self, library):
        Prior.__init__(self, library)

    def validate(self):
        if len(self.library.float_tokens) == 0:
            message = "All terminal tokens are input variables, so all" \
                "sequences will have an input variable."
            return message
        return None

    def __call__(self, actions, parent, sibling, dangling):
        # Constrain when:
        # 1) the expression would end if a terminal is chosen and
        # 2) there are no input variables
        mask = (dangling == 1) & \
               (np.sum(np.isin(actions, self.library.input_tokens), axis=1) == 0)
        prior = self.make_constraint(mask, self.library.float_tokens)
        return prior

    def is_violated(self, actions, parent, sibling):
        # Violated if no input tokens are found in actions
        tokens = self.library.input_tokens
        return bool(np.isin(tokens, actions).sum() == 0)

    def describe(self):
        message = "{}: Sequences contain at least one input variable Token.".format(self.__class__.__name__)
        return message


class InverseUnaryConstraint(Constraint):
    """Class that constrains each unary Token from being the child of its
    corresponding inverse unary Tokens."""

    def __init__(self, library):
        Prior.__init__(self, library)
        self.priors = []

        for target, effector in library.inverse_tokens.items():
            targets = [target]
            effectors = [effector]
            prior = RelationalConstraint(library=library,
                                         targets=targets,
                                         effectors=effectors,
                                         relationship="child")
            self.priors.append(prior)

    def validate(self):
        if len(self.priors) == 0:
            message = "There are no inverse unary Token pairs in the Library."
            return message
        return None

    def __call__(self, actions, parent, sibling, dangling):
        prior = sum([prior(actions, parent, sibling, dangling) for prior in self.priors])
        return prior

    def is_violated(self, actions, parent, sibling):

        for p in self.priors:
            if p.is_violated(actions, parent, sibling):
                return True

        return False

    def describe(self):
        message = [prior.describe() for prior in self.priors]
        return "\n{}: ".format(self.__class__.__name__).join(message)


class RepeatConstraint(Constraint):
    """Class that constrains Tokens to appear between a minimum and/or maximum
    number of times."""

    def __init__(self, library, tokens, min_=None, max_=None):
        """
        Parameters
        ----------
        tokens : Token or list of Tokens
            Token(s) which should, in total, occur between min_ and max_ times.

        min_ : int or None
            Minimum number of times tokens should occur.

        max_ : int or None
            Maximum number of times tokens should occur.
        """

        Prior.__init__(self, library)
        assert min_ is not None or max_ is not None, \
            "{}: At least one of (min_, max_) must not be None.".format(self.__class__.__name__)
        self.min = min_
        self.max = max_
        self.tokens = library.actionize(tokens)

        assert min_ is None, "{}: Repeat minimum constraints are not yet " \
            "supported. This requires knowledge of length constraints.".format(self.__class__.__name__)

    def __call__(self, actions, parent, sibling, dangling):
        counts = np.sum(np.isin(actions, self.tokens), axis=1)
        prior = self.init_zeros(actions)
        if self.min is not None:
            raise NotImplementedError
        if self.max is not None:
            mask = counts >= self.max
            prior += self.make_constraint(mask, self.tokens)

        return prior

    def is_violated(self, actions, parent, sibling):
        return bool(np.isin(actions, self.tokens).sum() > self.max)

    def describe(self):
        names = ", ".join([self.library.names[t] for t in self.tokens])
        if self.min is None:
            message = "{}: [{}] cannot occur more than {} times."\
                .format(self.__class__.__name__, names, self.max)
        elif self.max is None:
            message = "{}: [{}] must occur at least {} times."\
                .format(self.__class__.__name__, names, self.min)
        else:
            message = "{}: [{}] must occur between {} and {} times."\
                .format(self.__class__.__name__, names, self.min, self.max)
        return message


class LengthConstraint(Constraint):
    """Class that constrains the Program from falling within a minimum and/or
    maximum length"""

    def __init__(self, library, min_=None, max_=None):
        """
        Parameters
        ----------
        min_ : int or None
            Minimum length of the Program.

        max_ : int or None
            Maximum length of the Program.
        """

        Prior.__init__(self, library)
        self.min = min_
        self.max = max_

        assert min_ is not None or max_ is not None, \
            "At least one of (min_, max_) must not be None."

    def initial_prior(self):
        prior = Prior.initial_prior(self)
        for t in self.library.terminal_tokens:
            prior[t] = -np.inf
        return prior

    def __call__(self, actions, parent, sibling, dangling):

        # Initialize the prior
        prior = self.init_zeros(actions)
        i = actions.shape[1] - 1 # Current time
        # Never need to constrain max length for first half of expression
        if self.max is not None and (i + 2) >= self.max // 2:
            remaining = self.max - (i + 1)
            # assert sum(dangling > remaining) == 0, (dangling, remaining)
            # TBD: For loop over arities
            mask = dangling >= remaining - 1 # Constrain binary
            prior += self.make_constraint(mask, self.library.binary_tokens)
            mask = dangling == remaining # Constrain unary
            prior += self.make_constraint(mask, self.library.unary_tokens)

        # Constrain terminals when dangling == 1 until selecting the
        # (min_length)th token
        if self.min is not None and (i + 2) < self.min:
            mask = dangling == 1 # Constrain terminals
            prior += self.make_constraint(mask, self.library.terminal_tokens)


        return prior

    def is_violated(self, actions, parent, sibling):
        l = len(actions[0])
        if self.min is not None and l < self.min:
            return True
        if self.max is not None and l > self.max:
            return True

        return False

    def describe(self):
        message = []
        indent = " " * len(self.__class__.__name__) + "  "
        if self.min is not None:
            message.append("{}: Sequences have minimum length {}.".format(self.__class__.__name__, self.min))
        if self.max is not None:
            message.append(indent + "Sequences have maximum length {}.".format(self.max))
        message = "\n".join(message)
        return message


class DomainRangeConstraint(Constraint):
    """
    Class that constrains two special cases regarding the domain and range of
    (X, y) data and tokens. First, for the first position, it constrains tokens
    when its range does not contain the interval [min(y), max(y)]. For example,
    if min(y) = -2, the first token cannot be sin. Second, input variable x is
    constrained if their parent would be unary and the domain of the parent
    does not contain the interval [min(x), max(x)]. For example, if min(x) = -2
    and the parent is sqrt, then x is constrained.
    """

    def __init__(self, library):
        Constraint.__init__(self, library)

        # Min/max of dataset domain
        X_min = Program.task.X_train.min(axis=0) # (n_input_var,)
        X_max = Program.task.X_train.max(axis=0) # (n_input_var,)

        # Min/max of dataset range
        y_min = Program.task.y_train.min() # scalar
        y_max = Program.task.y_train.max() # scalar

        # TBD: Add domain/range to Token
        # Define domains of possible tokens
        domains = defaultdict(lambda : (-np.inf, np.inf))
        domains.update({
            "sqrt" : (0, np.inf),
            "log" : (0, np.inf)
        })

        # TBD: Add domain/range to Token
        # Define ranges of possible tokens
        ranges = defaultdict(lambda : (-np.inf, np.inf))
        ranges.update({
            "sin" : (-1.0, 1.0),
            "cos" : (-1.0, 1.0),
            "exp" : (0, np.inf),
            "abs" : (0, np.inf),
            "sqrt" : (0, np.inf),
            "n2" : (0, np.inf),
            "n4" : (0, np.inf),
            "n6" : (0, np.inf),
        })

        # Convert to np.ndarray of shape (L, 2)
        domains = np.array([domains[t.name] for t in self.library.tokens], dtype=np.float32)
        ranges = np.array([ranges[t.name] for t in self.library.tokens], dtype=np.float32)

        # Pre-compute the initial prior
        self.p0 = Prior.initial_prior(self) # Zeros of shape (L,)
        for i, R in enumerate(ranges):
            if y_min < R[0] or y_max > R[1]:
                self.p0[i] = -np.inf

        # Pre-compute constraining unary parents for each input variable
        self.constraining_parents = [] # List of np.ndarray
        for i in range(self.library.n_input_tokens):
            self.constraining_parents.append([])
            for t in self.library.unary_tokens:
                D = domains[t]
                if X_min[i] < D[0] or X_max[i] > D[1]:
                    adj_parent = self.library.parent_adjust[t]
                    self.constraining_parents[i].append(adj_parent)
            self.constraining_parents[i] = np.array(self.constraining_parents[i], dtype=np.int32)

        # Pre-compute logits to add when constraining maximum length and this is
        # the last chance to select a unary token (before being constrained to
        # select only terminals). This will only be used if self.max_length is later
        # set to not be None
        self.max_length = None
        self.last_chance_unary = Prior.initial_prior(self)
        for t in self.library.unary_tokens:
            parent = self.library.parent_adjust[t]
            if all([parent in arr for arr in self.constraining_parents]):
                self.last_chance_unary[t] = -np.inf

    def initial_prior(self):
        return self.p0

    def __call__(self, actions, parent, sibling, dangling):

        prior = self.init_zeros(actions)

        # SPECIAL CASE:
        # If max length is constrained and this is the last chance to select a
        # unary token, make sure that a unary can only be chosen if it is not a
        # constraining parent of all input variables. Without this, there can be
        # a collision with LengthConstraint.
        if self.max_length is not None:
            remaining = self.max_length - actions.shape[1]
            mask = dangling == remaining - 1 # Last chance to constrain unary
            prior[mask] += self.last_chance_unary

        # For each input variable, see if the parent constrains that input
        for i, x in enumerate(self.library.input_tokens):
            logit_adjust = Prior.initial_prior(self) # Zeros of shape (L,)
            logit_adjust[x] = -np.inf
            mask = np.isin(parent, self.constraining_parents[i])
            prior[mask] += logit_adjust

        return prior

    def describe(self):
        message = []
        indent = " " * len(self.__class__.__name__) + "  "
        message.append("{}: First token's range must contain [min(y), max(y)].".format(self.__class__.__name__))
        message.append(indent + "Input variable domains must be contained in unary parent domains.")
        message = "\n".join(message)
        return message

    def validate(self):
        if self.p0.sum() == 0 and all([len(x) == 0 for x in self.constraining_parents]):
            return "All token ranges contain [min(y), max(y)] and all token domains contain [min(x), max(x)]."
        return None


class UniformArityPrior(Prior):
    """Class that puts a fixed prior on arities by transforming the initial
    distribution from uniform over tokens to uniform over arities."""

    def __init__(self, library):

        Prior.__init__(self, library)

        # For each token, subtract log(n), where n is the total number of tokens
        # in the library with the same arity as that token. This is equivalent
        # to... For each arity, subtract log(n) from tokens of that arity, where
        # n is the total number of tokens of that arity
        self.logit_adjust = np.zeros((self.L,), dtype=np.float32)
        for arity, tokens in self.library.tokens_of_arity.items():
            self.logit_adjust[tokens] -= np.log(len(tokens))

    def initial_prior(self):
        return self.logit_adjust

    def __call__(self, actions, parent, sibling, dangling):

        # This will be broadcast when added to the joint prior
        prior = self.logit_adjust
        return prior

    def describe(self):
        """Describe the Prior."""

        return "{}: Activated.".format(self.__class__.__name__)


class SoftLengthPrior(Prior):
    """Class that puts a soft prior on length. Before loc, terminal probabilities
    are scaled by exp(-(t - loc) ** 2 / (2 * scale)) where dangling == 1. After
    loc, non-terminal probabilities are scaled by that number."""

    def __init__(self, library, loc, scale):

        Prior.__init__(self, library)

        self.loc = loc
        self.scale = scale

        self.terminal_mask = np.zeros((self.L,), dtype=np.bool)
        self.terminal_mask[self.library.terminal_tokens] = True

        self.nonterminal_mask = ~self.terminal_mask

    def __call__(self, actions, parent, sibling, dangling):

        # Initialize the prior
        prior = self.init_zeros(actions)
        t = actions.shape[1] # Current time

        # Adjustment to terminal or non-terminal logits
        logit_adjust = -(t - self.loc) ** 2 / (2 * self.scale)

        # Before loc, decrease p(terminal) where dangling == 1
        if t < self.loc:
            prior[dangling == 1] += self.terminal_mask * logit_adjust

        # After loc, decrease p(non-terminal)
        else:
            prior += self.nonterminal_mask * logit_adjust

        return prior

    def validate(self):
        if self.loc is None or self.scale is None:
            message = "'scale' and 'loc' arguments must be specified!"
            return message
        return None



class LanguageModelPrior(Prior):
    """Class that applies a prior based on a pre-trained language model."""

    def __init__(self, library, weight=1.0, **kwargs):

        Prior.__init__(self, library)

        self.lm = LM(library, **kwargs)
        self.weight = weight

    def initial_prior(self):

        # TBD: Get initial prior from language model
        return np.zeros((self.L,), dtype=np.float32)

    def __call__(self, actions, parent, sibling, dangling):

        """
        NOTE: This assumes that the prior is always called sequentially during
        sampling. This may break if calling the prior arbitrarily.
        """
        if actions.shape[1] == 1:
            self.lm.next_state = None

        action = actions[:, -1] # Current action
        prior = self.lm.get_lm_prior(action)
        prior *= self.weight

        return prior

    def validate(self):
        if self.weight is None:
            message = "Need to specify language model arguments."
            return message
        return None


class StateCheckerConstraint(Constraint):
    """Class that impose constraints on StateChecker Tokens to avoid degenerate
    situations (e.g., checking if xi < 6 when we know xi < 3).

    Consider StateCheckers 'xi < tj' that is associated with the
    i-th state variable xi and threshold tj, and 'xl < tk' that is
    associated with the l-th state variable xl and threshold tk.

    The constraints include:
        1. 'xl < tk' cannot be the left child of 'xi < tj' if l < i or if l == i and tk >= tj
        2. 'xl < tk' cannot be the right child of 'xi < tj' if l <= i
        3. a StateChecker cannot be a child of a non-StateChecker."""

    def __init__(self, library):
        Prior.__init__(self, library)
        self.priors = []

        # Loop over StateChecker 'xi < tj'
        for parent in library.state_checker_tokens:
            effectors = [parent]
            targets = []

            # Add StateChecker 'xl < tk' to targets if l < i
            for child in library.state_checker_tokens:
                if self.library[child].state_index < self.library[parent].state_index:
                    targets.append(child)

            # Add StateChecker 'xl < tk' to targets if l == i and tk >= tj
            for child in library.state_checker_tokens:
                if self.library[child].state_index == self.library[parent].state_index:
                    if self.library[child].threshold >= self.library[parent].threshold:
                        targets.append(child)

            if len(targets) > 0:
                # Add prior that constraints targets (containing 'xl < tk' with l < i, and
                # 'xl < tk' with l == i and tk >= tj) from being the left child of 'xi < tj'
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=effectors,
                                             relationship="lchild")
                self.priors.append(prior)

            # Add StateChecker 'xl < tk' to targets if l == i and tk < tj
            # (targets now contains all StateChecker 'xl < tk' with l <= i)
            for child in library.state_checker_tokens:
                if self.library[child].state_index == self.library[parent].state_index:
                    if self.library[child].threshold < self.library[parent].threshold:
                        targets.append(child)

            if len(targets) > 0:
                # Add prior that constraints targets (containing 'xl < tk' with l <= i)
                # from being the right child of 'xi < tj'
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=effectors,
                                             relationship="rchild")
                self.priors.append(prior)

        # Add priors that constraint any StateChecker from being a child of any non-StateChecker
        if len(library.state_checker_tokens) > 0:
            non_state_checker_tokens = np.setdiff1d(np.arange(self.L), library.state_checker_tokens,
                                                    assume_unique=True)
            for parent in non_state_checker_tokens:
                if self.library[parent].arity > 0:
                    effectors = [parent]
                    prior = RelationalConstraint(library,
                                                 targets=library.state_checker_tokens,
                                                 effectors=effectors,
                                                 relationship="child")
                    self.priors.append(prior)

            for terminal_token in library.terminal_tokens:
                prior = RelationalConstraint(library,
                                             targets=[terminal_token],
                                             effectors=library.state_checker_tokens,
                                             relationship="uchild")
                self.priors.append(prior)


    def validate(self):
        if len(self.library.state_checker_tokens) == 0:
            return "There are no StateChecker tokens in the library."
        return None

    def __call__(self, actions, parent, sibling, dangling):
        prior = sum([prior(actions, parent, sibling, dangling)
                     for prior in self.priors])
        return prior

    def describe(self):
        indent = " " * len(self.__class__.__name__) + "  "
        message = ["StateCheckerConstraint: Sequences containing StateChecker tokens cannot produce degenerate logic."]
        message.append(indent + "'xl < tk' cannot be the left child of 'xi < tj' if l < i or if l == i and tk >= tj.")
        message.append(indent + "'xl < tk' cannot be the right child of 'xi < tj' if l <= i.")
        message.append(indent + "A StateChecker cannot be a child of a non-StateChecker.")
        return "\n".join(message)


class MutuallyExclusiveConstraint(Constraint):
    """Class that constrains the program from having two or more distinct tokens
    in a given set of tokens. Mathematically, this constrains the intersection
    of set(tokens) and set(actions) to have a resulting size of 0 or 1.

    For example, if the given set of tokens = ["poly", "const"], then this
    constraint prevents actions = ["add", "const", "poly"] to be sampled.
    Note, however, that it does not prevents the same token to appear multiple
    times. So, e.g., actions = ["add", "poly", "poly"] is allowed."""

    def __init__(self, library, tokens):
        super().__init__(library)
        self.tokens = tokens

    def __call__(self, actions, parent, sibling, dangling):
        prior = self.init_zeros(actions)

        # For each mutually exclusive token, see if it exists.
        # If so, constrain all other mutually exclusive tokens.
        for token in self.tokens:
            mask = np.any(actions == token, axis=1)
            others = self.tokens[self.tokens != token]
            prior += self.make_constraint(mask, others)
        return prior

    def validate(self):
        if len(self.tokens) < 2:
            return "Length of {} must be at least 2".format(self.tokens)
        return None

    def is_violated(self, actions, parent, sibling):
        return np.intersect1d(self.tokens, actions).size > 1

    def describe(self):
        tokens = ", ".join([self.library.names[t] for t in self.tokens])
        message = self.__class__.__name__
        message += ": Two or more distinct tokens in [{}] cannot appear in the same sequence.".format(tokens)
        return message

class PolyConstraint(Constraint):
    """Class that impose constraints such that polynomial fitting problems can be
    constructed and well-defined when Polynomial Token is in library."""
    def __init__(self, library):
        Prior.__init__(self, library)
        # poly token cannot appear more than 1 time
        self.priors = [RepeatConstraint(library, "poly", None, 1)]

        # any function whose inverse is multi-valued cannot be an ancestor of "poly"
        # because it may lead to wrong data for the polynomial fitting problem.
        invalid_ancestors = ["sin", "cos", "tan", "n2", "n4", "n6", "abs"]
        invalid_ancestors = np.intersect1d(invalid_ancestors, library.names, assume_unique=True)
        if len(invalid_ancestors) > 0:
            descendant_prior = RelationalConstraint(library,
                                                    targets=["poly"],
                                                    effectors=invalid_ancestors,
                                                    relationship="descendant")
            self.priors.append(descendant_prior)

        # poly and const cannot appear in the same traversal
        if library.const_token is not None:
            mutually_exclusive_tokens = np.array([library.poly_token, library.const_token])
            self.priors.append(MutuallyExclusiveConstraint(library, mutually_exclusive_tokens))

    def __call__(self, actions, parent, sibling, dangling):
        prior = sum([prior(actions, parent, sibling, dangling)
                     for prior in self.priors])
        return prior

    def validate(self):
        if "poly" not in self.library.names:
            return "There is no 'poly' token in the Library"
        return None

    def is_violated(self, actions, parent, sibling):
        for prior in self.priors:
            if prior.is_violated(actions, parent, sibling):
                return True
        return False

    def describe(self):
        return "\n".join([prior.describe() for prior in self.priors])


class MultiDiscreteConstraint(Constraint):
    """
    Class that imposes the constraint that once one MultiDiscreteAction is
    sampled, only MultiDiscreteAction with a different action dimension can be
    sampled, until "STOP" is sampled.

    Additional constraints may apply based on the value of dense and ordered.
    """
    def __init__(self, library, dense, ordered):
        """
        Parameters
        ----------
        dense : bool
            If True, once one MultiDiscreteAction is sampled, "STOP" cannot be
            sampled until MultiDiscreteAction for all action dimensions are sampled.

        ordered : bool
            If True, action dimensions of adjacent MultiDiscreteAction tokens
            constrained to be in ascending order, until "STOP" is sampled.
        """
        super().__init__(library)
        self.dense = dense
        self.ordered = ordered
        self.priors = []
        self.special_prior = None
        non_multi_discrete = np.setdiff1d(np.arange(self.L), library.multi_discrete_tokens,
                                          assume_unique=True).tolist()

        self.unary_multi_discrete = np.intersect1d(library.multi_discrete_tokens,
                                                   library.unary_tokens, assume_unique=True)

        if dense and ordered:
            targets = [t for t in library.multi_discrete_tokens if
                    self.library[t].action_dim is None or
                    self.library[t].action_dim != 0]
            prior = RelationalConstraint(library,
                                         targets=targets,
                                         effectors=non_multi_discrete,
                                         relationship="child")
            self.priors.append(prior)

            for parent in self.unary_multi_discrete:
                if self.library[parent].action_dim < MultiDiscreteAction.n_dims-1:
                    targets = [t for t in library.multi_discrete_tokens if
                            self.library[t].action_dim is None or
                            self.library[t].action_dim != self.library[parent].action_dim+1]
                else:
                    targets = self.unary_multi_discrete
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=[parent],
                                             relationship="child")
                self.priors.append(prior)

        elif not dense and ordered:
            for parent in self.unary_multi_discrete:
                targets = [t for t in self.unary_multi_discrete if
                        self.library[t].action_dim <= self.library[parent].action_dim]
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=[parent],
                                             relationship="child")
                self.priors.append(prior)

        elif dense and not ordered:
            targets = [t for t in library.multi_discrete_tokens if
                        self.library[t].action_dim is None]
            prior = RelationalConstraint(library,
                                         targets=targets,
                                         effectors=non_multi_discrete,
                                         relationship="child")
            self.priors.append(prior)
            for ancestor in self.unary_multi_discrete:
                targets = [t for t in library.multi_discrete_tokens if
                        self.library[t].action_dim is None or
                        self.library[t].action_dim == self.library[ancestor].action_dim]
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=[ancestor],
                                             relationship="descendant")
                self.priors.append(prior)

            self.special_prior = RelationalConstraint(library,
                                                      targets=self.unary_multi_discrete,
                                                      effectors=self.unary_multi_discrete,
                                                      relationship="child")
        else: # not dense and not ordered
            for ancestor in self.unary_multi_discrete:
                targets = [t for t in self.unary_multi_discrete if
                        self.library[t].action_dim == self.library[ancestor].action_dim]
                prior = RelationalConstraint(library,
                                             targets=targets,
                                             effectors=[ancestor],
                                             relationship="descendant")
                self.priors.append(prior)

    def __call__(self, actions, parent, sibling, dangling):
        if self.special_prior is None:
            prior = sum([prior(actions, parent, sibling, dangling)
                        for prior in self.priors])
        else:
            mask = np.full(len(actions), False)
            for i in range(len(actions)):
                for action in actions[i][-MultiDiscreteAction.n_dims:]:
                    if action not in self.unary_multi_discrete:
                        mask[i] = True
                        break

            prior = self.init_zeros(actions)
            prior[~mask] = self.special_prior(actions[~mask], parent[~mask],
                                              sibling[~mask], dangling[~mask])
            prior[mask] = sum([prior(actions[mask], parent[mask],
                                     sibling[mask], dangling[mask]) for prior in self.priors])
        return prior

    def validate(self):
        if len(self.library.multi_discrete_tokens) == 0:
            return "There are no MultiDiscreteAction tokens in the library."
        return None

    def describe(self):
        indent = " " * len(self.__class__.__name__) + "  "
        message = "MultiDiscreteConstraint: Child of a MultiDiscreteAction must"
        message += " be a MultiDiscreteAction with a different action_dim or STOP."
        message = [message]
        if self.dense:
            message.append(indent + "Each action branch must contain all action_dim.")
        if self.ordered:
            message.append(indent + "action_dim in each action branch must be ascending.")
        return "\n".join(message)

In [ ]:
#config
def get_base_config(task, language_prior):
    # Load base config
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_common.json"), encoding='utf-8') as f:
        base_config = json.load(f)

    # Load task specific config
    task_config_file = None
    if task in ["regression", None]:
        task_config_file = "config_regression.json"
    elif task in ["control"]:
        task_config_file = "config_control.json"
    else:
        # Custom tasks use config_common.json.
        task_config_file = "config_common.json"
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), task_config_file), encoding='utf-8') as f:
        task_config = json.load(f)

    # Load language prior config
    if language_prior:
        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_language.json"), encoding='utf-8') as f:
            language_config = json.load(f)
        task_config = safe_merge_dicts(task_config, language_config)

    return safe_merge_dicts(base_config, task_config)


def load_config(config=None):
    # Load user config
    if isinstance(config, str):
        with open(config, encoding='utf-8') as f:
            user_config = json.load(f)
    elif isinstance(config, dict):
        user_config = config
    else:
        assert config is None, "Config must be None, str, or dict."
        user_config = {}

    # Determine the task and language prior
    try:
        task = user_config["task"]["task_type"]
    except KeyError:
        task = "regression"
        print("WARNING: Task type not specified. Falling back to default task type '{}' to load config.".format(task))
    try:
        language_prior = user_config["prior"]["language_model"]["on"]
    except KeyError:
        language_prior = False

    # Load task-specific base config
    base_config = get_base_config(task, language_prior)

    # Return combined configs
    return safe_merge_dicts(base_config, user_config)

In [ ]:
#tf_state_manager

class StateManager(ABC):
    """
    An interface for handling the tf.Tensor inputs to the Policy.
    """

    def setup_manager(self, policy):
        """
        Function called inside the policy to perform the needed initializations (e.g., if the tf context is needed)
        :param policy the policy class
        """
        self.policy = policy
        self.max_length = policy.max_length

    @abstractmethod
    def get_tensor_input(self, obs):
        """
        Convert an observation from a Task into a Tesnor input for the
        Policy, e.g. by performing one-hot encoding or embedding lookup.

        Parameters
        ----------
        obs : np.ndarray (dtype=np.float32)
            Observation coming from the Task.

        Returns
        --------
        input_ : tf.Tensor (dtype=tf.float32)
            Tensor to be used as input to the Policy.
        """
        return

    def process_state(self, obs):
        """
        Entry point for adding information to the state tuple.
        If not overwritten, this functions does nothing
        """
        return obs


def make_state_manager(config):
    """
    Parameters
    ----------
    config : dict
        Parameters for this StateManager.

    Returns
    -------
    state_manager : StateManager
        The StateManager to be used by the policy.
    """
    manager_dict = {
        "hierarchical": HierarchicalStateManager
    }

    if config is None:
        config = {}

    # Use HierarchicalStateManager by default
    manager_type = config.pop("type", "hierarchical")

    manager_class = manager_dict[manager_type]
    state_manager = manager_class(**config)

    return state_manager


class HierarchicalStateManager(StateManager):
    """
    Class that uses the previous action, parent, sibling, and/or dangling as
    observations.
    """

    def __init__(self, observe_parent=True, observe_sibling=True,
                 observe_action=False, observe_dangling=False, embedding=False,
                 embedding_size=8):
        """
        Parameters
        ----------
        observe_parent : bool
            Observe the parent of the Token being selected?

        observe_sibling : bool
            Observe the sibling of the Token being selected?

        observe_action : bool
            Observe the previously selected Token?

        observe_dangling : bool
            Observe the number of dangling nodes?

        embedding : bool
            Use embeddings for categorical inputs?

        embedding_size : int
            Size of embeddings for each categorical input if embedding=True.
        """
        self.observe_parent = observe_parent
        self.observe_sibling = observe_sibling
        self.observe_action = observe_action
        self.observe_dangling = observe_dangling
        self.library = Program.library

        # Parameter assertions/warnings
        assert self.observe_action + self.observe_parent + self.observe_sibling + self.observe_dangling > 0, \
            "Must include at least one observation."

        self.embedding = embedding
        self.embedding_size = embedding_size

    def setup_manager(self, policy):
        super().setup_manager(policy)
        # Create embeddings if needed
        if self.embedding:
            initializer = tf.random_uniform_initializer(minval=-1.0,
                                                        maxval=1.0,
                                                        seed=0)
            with tf.variable_scope("embeddings", initializer=initializer):
                if self.observe_action:
                    self.action_embeddings = tf.get_variable("action_embeddings",
                                                             [self.library.n_action_inputs, self.embedding_size],
                                                             trainable=True)
                if self.observe_parent:
                    self.parent_embeddings = tf.get_variable("parent_embeddings",
                                                             [self.library.n_parent_inputs, self.embedding_size],
                                                             trainable=True)
                if self.observe_sibling:
                    self.sibling_embeddings = tf.get_variable("sibling_embeddings",
                                                              [self.library.n_sibling_inputs, self.embedding_size],
                                                              trainable=True)

    def get_tensor_input(self, obs):
        observations = []
        unstacked_obs = tf.unstack(obs, axis=1)
        action, parent, sibling, dangling = unstacked_obs[:4]

        # Cast action, parent, sibling to int for embedding_lookup or one_hot
        action = tf.cast(action, tf.int32)
        parent = tf.cast(parent, tf.int32)
        sibling = tf.cast(sibling, tf.int32)

        # Action, parent, and sibling inputs are either one-hot or embeddings
        if self.observe_action:
            if self.embedding:
                x = tf.nn.embedding_lookup(self.action_embeddings, action)
            else:
                x = tf.one_hot(action, depth=self.library.n_action_inputs)
            observations.append(x)
        if self.observe_parent:
            if self.embedding:
                x = tf.nn.embedding_lookup(self.parent_embeddings, parent)
            else:
                x = tf.one_hot(parent, depth=self.library.n_parent_inputs)
            observations.append(x)
        if self.observe_sibling:
            if self.embedding:
                x = tf.nn.embedding_lookup(self.sibling_embeddings, sibling)
            else:
                x = tf.one_hot(sibling, depth=self.library.n_sibling_inputs)
            observations.append(x)

        # Dangling input is just the value of dangling
        if self.observe_dangling:
            x = tf.expand_dims(dangling, axis=-1)
            observations.append(x)

        input_ = tf.concat(observations, -1)
        # possibly concatenates additional observations (e.g., bert embeddings)
        if len(unstacked_obs) > 4:
            input_ = tf.concat([input_, tf.stack(unstacked_obs[4:], axis=-1)], axis=-1)
        return input_

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
#policy
from typing import Tuple, TypeVar
from abc import ABC, abstractmethod

actions = tf.TensorArray
obs     = tf.TensorArray
priors  = tf.TensorArray
neglogp = tf.TensorArray
entropy = tf.TensorArray

def make_policy(sess, prior, state_manager, policy_type, **config_policy):
    """Factory function for Policy object."""

    if policy_type == "rnn":
        from dso.policy.rnn_policy import RNNPolicy
        policy_class = RNNPolicy
    else:
        # Custom policy import
        policy_class = import_custom_source(policy_type)
        assert issubclass(policy_class, Policy), \
                "Custom policy {} must subclass dso.policy.Policy.".format(policy_class)

    policy = policy_class(sess,
                          prior,
                          state_manager,
                          **config_policy)

    return policy

class Policy(ABC):
    """Abstract class for a policy. A policy is a parametrized probability
    distribution over discrete objects. DSO algorithms optimize the parameters
    of this distribution to generate discrete objects with high rewards.
    """

    def __init__(self,
            sess : tf.Session,
            prior : JointPrior,
            state_manager : StateManager,
            debug : int = 0,
            max_length : int = 30) -> None:
        '''Parameters
        ----------
        sess : tf.Session
            TenorFlow Session object.

        prior : dso.prior.JointPrior
            JointPrior object used to adjust probabilities during sampling.

        state_manager: dso.tf_state_manager.StateManager
            Object that handles the state features to be used

        debug : int
            Debug level, also used in learn(). 0: No debug. 1: Print shapes and
            number of parameters for each variable.

        max_length : int or None
            Maximum sequence length. This will be overridden if a LengthConstraint
            with a maximum length is part of the prior.
        '''
        self.sess = sess
        self.prior = prior
        self.state_manager = state_manager
        self.debug = debug

        # Set self.max_length depending on the Prior
        self._set_max_length(max_length)

        # Samples produced during attempt to get novel samples.
        # Will be combined with checkpoint-loaded samples for next training step
        self.extended_batch = None
        self.valid_extended_batch = False

    def _set_max_length(self, max_length : int) -> None:
        """Set the max legnth depending on the Prior
        """
        # Find max_length from the LengthConstraint prior, if it exists
        # For binding task, max_length is # of allowed mutations or master-seq length
        # Both priors will never happen in the same experiment
        prior_max_length = None
        for single_prior in self.prior.priors:
            if isinstance(single_prior, LengthConstraint):
                if single_prior.max is not None:
                    prior_max_length = single_prior.max
                    self.max_length = prior_max_length
                break

        if prior_max_length is None:
            assert max_length is not None, "max_length must be specified if "\
                "there is no LengthConstraint."
            self.max_length = max_length
            print("WARNING: Maximum length not constrained. Sequences will "
                  "stop at {} and complete by repeating the first input "
                  "variable.".format(self.max_length))
        elif max_length is not None and max_length != self.max_length:
            print("WARNING: max_length ({}) will be overridden by value from "
                  "LengthConstraint ({}).".format(max_length, self.max_length))

    @abstractmethod
    def _setup_tf_model(self, **kwargs) -> None:
        """"Setup the TensorFlow graph(s).

        Returns
        -------
            None
        """
        raise NotImplementedError

    @abstractmethod
    def make_neglogp_and_entropy(self,
            B : Batch,
            entropy_gamma : float
            ) -> Tuple[neglogp, entropy]:
        """Computes the negative log-probabilities for a given
        batch of actions, observations and priors
        under the current policy.

        Returns
        -------
        neglogp, entropy :
            Tensorflow tensors
        """
        raise NotImplementedError

    @abstractmethod
    def sample(self, n : int) -> Tuple[actions, obs, priors]:
        """Sample batch of n expressions.

        Returns
        -------
        actions, obs, priors :
            Or a batch
        """
        raise NotImplementedError

    @abstractmethod
    def compute_probs(self, memory_batch, log=False):
        """Compute the probabilities of a Batch.

        Returns
        -------
        probs :
            Or a batch
        """
        raise NotImplementedError

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
from abc import ABC, abstractmethod

import tensorflow as tf
import numpy as np



# Used for function annotations using the type system
summaries = tf.TensorArray

def make_policy_optimizer(sess, policy, policy_optimizer_type, **config_policy_optimizer):
    """Factory function for policy optimizer object."""

    if policy_optimizer_type == "pg":
        from dso.policy_optimizer.pg_policy_optimizer import PGPolicyOptimizer
        policy_optimizer_class = PGPolicyOptimizer
    elif policy_optimizer_type == "pqt":
        from dso.policy_optimizer.pqt_policy_optimizer import PQTPolicyOptimizer
        policy_optimizer_class = PQTPolicyOptimizer
    elif policy_optimizer_type == "ppo":
        from dso.policy_optimizer.ppo_policy_optimizer import PPOPolicyOptimizer
        policy_optimizer_class = PPOPolicyOptimizer
    else:
        # Custom policy import
        policy_optimizer_class = import_custom_source(policy_optimizer_type)
        assert issubclass(policy_optimizer_class, Policy), \
                "Custom policy {} must subclass dso.policy.Policy.".format(policy_optimizer_class)

    policy_optimizer = policy_optimizer_class(sess,
                                              policy,
                                              **config_policy_optimizer)

    return policy_optimizer

class PolicyOptimizer(ABC):
    """Abstract class for a policy optimizer. A policy optimizer is an
    algorithm for optimizing the parameters of a parametrized policy.

    To define a new optimizer, inherit from this class and add the following
    methods (look in _setup_policy_optimizer below):

        _set_loss() : Define the \propto \log(p(\tau|\theta)) loss for the method
        _preppend_to_summary() : Add additional fields for the tensorflow summary

    """

    def _init(self,
            sess : tf.compat.v1.Session(),
            policy : Policy,
            debug : int = 0,
            summary : bool = False,
            # Optimizer hyperparameters
            optimizer : str = 'adam',
            learning_rate : float = 0.001,
            # Loss hyperparameters
            entropy_weight : float = 0.005,
            entropy_gamma : float = 1.0) -> None:
        '''Parameters
        ----------
        sess : tf.Session
            TensorFlow Session object.

        policy : dso.policy.Policy
            Parametrized probability distribution over discrete objects

        debug : int
            Debug level, also used in learn(). 0: No debug. 1: Print shapes and
            number of parameters for each variable.

        summary : bool
            Write tensorboard summaries?

        optimizer : str
            Optimizer to use. Supports 'adam', 'rmsprop', and 'sgd'.

        learning_rate : float
            Learning rate for optimizer.

        entropy_weight : float
            Coefficient for entropy bonus.

        entropy_gamma : float or None
            Gamma in entropy decay. None (or
            equivalently, 1.0) turns off entropy decay.
        '''
        self.sess = sess
        self.policy = policy

        # Needed in _setup_optimizer
        self.debug = debug
        self.optimizer = optimizer
        self.learning_rate = learning_rate

        # Need in self.summary
        self.summary = summary

        # Needed for make_batch_ph calls
        self.n_choices = Program.library.L

        # Placeholders, computed after instantiating expressions
        self.batch_size = tf.placeholder(dtype=tf.int32, shape=(), name="batch_size")
        self.baseline = tf.placeholder(dtype=tf.float32, shape=(), name="baseline")

        # On policy batch
        self.sampled_batch_ph = make_batch_ph("sampled_batch", self.n_choices)

        # Need in _init_loss_with_entropy
        self.entropy_weight = entropy_weight
        self.entropy_gamma = entropy_gamma


    def _init_loss_with_entropy(self) -> None:
        # Add entropy contribution to loss. The entropy regularizer does not
        # depend on the particular policy optimizer
        with tf.name_scope("losses"):

            self.neglogp, entropy = self.policy.make_neglogp_and_entropy(self.sampled_batch_ph, self.entropy_gamma)

            # Entropy loss
            self.entropy_loss = -self.entropy_weight * tf.reduce_mean(entropy, name="entropy_loss")
            loss = self.entropy_loss

            # self.loss is modified in the child object
            self.loss = loss


    @abstractmethod
    def _set_loss(self) -> None:
        """Define the \propto \log(p(\tau|\theta)) loss for the method

        Returns
        -------
            None
        """
        raise NotImplementedError


    def _setup_optimizer(self):
        """ Setup the optimizer
        """
        def make_optimizer(name, learning_rate):
            if name == "adam":
                return tf.train.AdamOptimizer(learning_rate=learning_rate)
            if name == "rmsprop":
                return tf.train.RMSPropOptimizer(learning_rate=learning_rate, decay=0.99)
            if name == "sgd":
                return tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            raise ValueError("Did not recognize optimizer '{}'".format(name))

        # Create training op
        optimizer = make_optimizer(name=self.optimizer, learning_rate=self.learning_rate)
        with tf.name_scope("train"):
            self.grads_and_vars = optimizer.compute_gradients(self.loss)
            self.train_op = optimizer.apply_gradients(self.grads_and_vars)
            # The two lines above are equivalent to:
            # self.train_op = optimizer.minimize(self.loss)
        with tf.name_scope("grad_norm"):
            self.grads, _ = list(zip(*self.grads_and_vars))
            self.norms = tf.global_norm(self.grads)

        if self.debug >= 1:
            total_parameters = 0
            print("")
            for variable in tf.trainable_variables():
                shape = variable.get_shape()
                n_parameters = np.product(shape)
                total_parameters += n_parameters
                print("Variable:    ", variable.name)
                print("  Shape:     ", shape)
                print("  Parameters:", n_parameters)
            print("Total parameters:", total_parameters)


    # abstractmethod (override if needed)
    def _preppend_to_summary(self) -> None:
        """Add particular fields to the summary log.
        Override if needed.
        """
        pass


    def _setup_summary(self) -> None:
        """ Setup tensor flow summary
        """
        with tf.name_scope("summary"):
            tf.summary.scalar("entropy_loss", self.entropy_loss)
            tf.summary.scalar("total_loss", self.loss)
            tf.summary.scalar("reward", tf.reduce_mean(self.sampled_batch_ph.rewards))
            tf.summary.scalar("baseline", self.baseline)
            tf.summary.histogram("reward", self.sampled_batch_ph.rewards)
            tf.summary.histogram("length", self.sampled_batch_ph.lengths)
            for g, v in self.grads_and_vars:
                tf.summary.histogram(v.name, v)
                tf.summary.scalar(v.name + '_norm', tf.norm(v))
                tf.summary.histogram(v.name + '_grad', g)
                tf.summary.scalar(v.name + '_grad_norm', tf.norm(g))
            tf.summary.scalar('gradient norm', self.norms)
            self.summaries = tf.summary.merge_all()


    def _setup_policy_optimizer(self,
            sess : tf.compat.v1.Session(),
            policy : Policy,
            debug : int = 0,
            summary : bool = False,
            # Optimizer hyperparameters
            optimizer : str = 'adam',
            learning_rate : float = 0.001,
            # Loss hyperparameters
            entropy_weight : float = 0.005,
            entropy_gamma : float = 1.0) -> None:
        """Setup of the policy optimizer.
        """
        self._init(sess, policy, debug, summary, optimizer, learning_rate, entropy_weight, entropy_gamma)
        self._init_loss_with_entropy()
        self._set_loss() # Abstract method defined in derived class
        self._setup_optimizer()
        if self.summary:
            self._preppend_to_summary() # Abstract method defined in derived class
            self._setup_summary()
        else:
            self.summaries = tf.no_op()


    @abstractmethod
    def train_step(self,
            baseline : np.ndarray,
            sampled_batch : Batch) -> summaries:
        """Computes loss, trains model, and returns summaries.

        Returns
        -------
            None
        """
        raise NotImplementedError




In [ ]:
#core.py
#DSO


class DeepSymbolicOptimizer():
    """
    Deep symbolic optimization model. Includes model hyperparameters and
    training configuration.

    Parameters
    ----------
    config : dict or str
        Config dictionary or path to JSON.

    Attributes
    ----------
    config : dict
        Configuration parameters for training.

    Methods
    -------
    train
        Builds and trains the model according to config.
    """

    def __init__(self, config=None):
        self.set_config(config)
        self.sess = None

    def setup(self):

        # Clear the cache and reset the compute graph
        Program.clear_cache()
        tf.reset_default_graph()

        # Generate objects needed for training and set seeds
        self.pool = self.make_pool_and_set_task()
        self.set_seeds() # Must be called _after_ resetting graph and _after_ setting task

        # Limit TF to single thread to prevent "resource not available" errors in parallelized runs
        session_config = tf.ConfigProto(intra_op_parallelism_threads=1,
                                        inter_op_parallelism_threads=1)
        self.sess = tf.Session(config=session_config)

        # Setup logdirs and output files
        self.output_file = self.make_output_file()
        self.save_config()

        # Prepare training parameters
        self.prior = self.make_prior()
        self.state_manager = self.make_state_manager()
        self.policy = self.make_policy()
        self.policy_optimizer = self.make_policy_optimizer()
        self.gp_controller = self.make_gp_controller()
        self.logger = self.make_logger()
        self.trainer = self.make_trainer()
        self.checkpoint = self.make_checkpoint()

    def train_one_step(self, override=None):
        """
        Train one iteration.
        """

        # Setup the model
        if self.sess is None:
            self.setup()

        # Run one step
        assert not self.trainer.done, "Training has already completed!"
        self.trainer.run_one_step(override)

        # Maybe save next checkpoint
        self.checkpoint.update()

        # If complete, return summary
        if self.trainer.done:
            return self.finish()

    def train(self):
        """
        Train the model until completion.
        """

        # Setup the model
        self.setup()

        # Train the model until done
        while not self.trainer.done:
            result = self.train_one_step()

        return result

    def finish(self):
        """
        After training completes, finish up and return summary dict.
        """

        # Return statistics of best Program
        p = self.trainer.p_r_best
        result = {"seed" : self.config_experiment["seed"]} # Seed listed first
        result.update({"r" : p.r})
        result.update(p.evaluate)
        result.update({
            "expression" : repr(p.sympy_expr),
            "traversal" : repr(p),
            "program" : p
        })

        # Save all results available only after all iterations are finished. Also return metrics to be added to the summary file
        results_add = self.logger.save_results(self.pool, self.trainer.nevals)
        result.update(results_add)

        # Close the pool
        if self.pool is not None:
            self.pool.close()

        return result

    def set_config(self, config):
        config = load_config(config)

        self.config = defaultdict(dict, config)
        self.config_task = self.config["task"]
        self.config_prior = self.config["prior"]
        self.config_logger = self.config["logging"]
        self.config_training = self.config["training"]
        self.config_state_manager = self.config["state_manager"]
        self.config_policy = self.config["policy"]
        self.config_policy_optimizer = self.config["policy_optimizer"]
        self.config_gp_meld = self.config["gp_meld"]
        self.config_experiment = self.config["experiment"]
        self.config_checkpoint = self.config["checkpoint"]

    def save_config(self):
        # Save the config file
        if self.output_file is not None:
            path = os.path.join(self.config_experiment["save_path"],
                                "config.json")
            # With run.py, config.json may already exist. To avoid race
            # conditions, only record the starting seed. Use a backup seed
            # in case this worker's seed differs.
            backup_seed = self.config_experiment["seed"]
            if not os.path.exists(path):
                if "starting_seed" in self.config_experiment:
                    self.config_experiment["seed"] = self.config_experiment["starting_seed"]
                    del self.config_experiment["starting_seed"]
                with open(path, 'w') as f:
                    json.dump(self.config, f, indent=3)
            self.config_experiment["seed"] = backup_seed

    def set_seeds(self):
        """
        Set the tensorflow, numpy, and random module seeds based on the seed
        specified in config. If there is no seed or it is None, a time-based
        seed is used instead and is written to config.
        """

        seed = self.config_experiment.get("seed")

        # Default uses current time in milliseconds, modulo 1e9
        if seed is None:
            seed = round(time() * 1000) % int(1e9)
            self.config_experiment["seed"] = seed

        # Shift the seed based on task name
        # This ensures a specified seed doesn't have similarities across different task names
        task_name = Program.task.name
        shifted_seed = seed + zlib.adler32(task_name.encode("utf-8"))

        # Set the seeds using the shifted seed
        tf.set_random_seed(shifted_seed)
        np.random.seed(shifted_seed)
        random.seed(shifted_seed)

    def make_prior(self):
        prior = make_prior(Program.library, self.config_prior)
        return prior

    def make_state_manager(self):
        state_manager = make_state_manager(self.config_state_manager)
        return state_manager

    def make_trainer(self):
        trainer = Trainer(self.sess,
                          self.policy,
                          self.policy_optimizer,
                          self.gp_controller,
                          self.logger,
                          self.pool,
                          **self.config_training)
        return trainer

    def make_logger(self):
        logger = StatsLogger(self.sess,
                             self.output_file,
                             **self.config_logger)
        return logger

    def make_checkpoint(self):
        checkpoint = Checkpoint(self,
                                **self.config_checkpoint)
        return checkpoint

    def make_policy_optimizer(self):
        policy_optimizer = make_policy_optimizer(self.sess,
                                                 self.policy,
                                                 **self.config_policy_optimizer)
        return policy_optimizer

    def make_policy(self):
        policy = make_policy(self.sess,
                             self.prior,
                             self.state_manager,
                             **self.config_policy)
        return policy

    def make_gp_controller(self):
        if self.config_gp_meld.pop("run_gp_meld", False):
            from dso.gp.gp_controller import GPController
            gp_controller = GPController(self.prior,
                                         self.config_prior,
                                         **self.config_gp_meld)
        else:
            gp_controller = None
        return gp_controller

    def make_pool_and_set_task(self):
        # Create the pool and set the Task for each worker

        # Set complexity and const optimizer here so pool can access them
        # Set the complexity function
        complexity = self.config_training["complexity"]
        Program.set_complexity(complexity)

        # Set the constant optimizer
        const_optimizer = self.config_training["const_optimizer"]
        const_params = self.config_training["const_params"]
        const_params = const_params if const_params is not None else {}
        Program.set_const_optimizer(const_optimizer, **const_params)

        pool = None
        n_cores_batch = self.config_training.get("n_cores_batch")
        if n_cores_batch is not None:
            if n_cores_batch == -1:
                n_cores_batch = cpu_count()
            if n_cores_batch > 1:
                pool = Pool(n_cores_batch,
                            initializer=set_task,
                            initargs=(self.config_task,))

        # Set the Task for the parent process
        set_task(self.config_task)

        return pool

    def make_output_file(self):
        """Generates an output filename"""

        # If logdir is not provided (e.g. for pytest), results are not saved
        if self.config_experiment.get("logdir") is None:
            self.save_path = None
            print("WARNING: logdir not provided. Results will not be saved to file.")
            return None

        # When using run.py, timestamp is already generated
        timestamp = self.config_experiment.get("timestamp")
        if timestamp is None:
            timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            self.config_experiment["timestamp"] = timestamp

        # Generate save path
        task_name = Program.task.name
        if self.config_experiment["exp_name"] is None:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                '_'.join([task_name, timestamp]))
        else:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                self.config_experiment["exp_name"])

        self.config_experiment["task_name"] = task_name
        self.config_experiment["save_path"] = save_path
        os.makedirs(save_path, exist_ok=True)

        seed = self.config_experiment["seed"]
        output_file = os.path.join(save_path,
                                   "dso_{}_{}.csv".format(task_name, seed))

        self.save_path = save_path

        return output_file

    def save(self, save_path=None):
        self.checkpoint.save(save_path)

    def load(self, load_path):
        self.checkpoint.load(load_path)

In [ ]:
#load_config

import os

import commentjson as json

def get_base_config(task, language_prior):
    # Load base config
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_common.json"), encoding='utf-8') as f:
        base_config = json.load(f)

    # Load task specific config
    task_config_file = None
    if task in ["regression", None]:
        task_config_file = "config_regression.json"
    elif task in ["control"]:
        task_config_file = "config_control.json"
    else:
        # Custom tasks use config_common.json.
        task_config_file = "config_common.json"
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), task_config_file), encoding='utf-8') as f:
        task_config = json.load(f)

    # Load language prior config
    if language_prior:
        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_language.json"), encoding='utf-8') as f:
            language_config = json.load(f)
        task_config = safe_merge_dicts(task_config, language_config)

    return safe_merge_dicts(base_config, task_config)


def load_config(config=None):
    # Load user config
    if isinstance(config, str):
        with open(config, encoding='utf-8') as f:
            user_config = json.load(f)
    elif isinstance(config, dict):
        user_config = config
    else:
        assert config is None, "Config must be None, str, or dict."
        user_config = {}

    # Determine the task and language prior
    try:
        task = user_config["task"]["task_type"]
    except KeyError:
        task = "regression"
        print("WARNING: Task type not specified. Falling back to default task type '{}' to load config.".format(task))
    try:
        language_prior = user_config["prior"]["language_model"]["on"]
    except KeyError:
        language_prior = False

    # Load task-specific base config
    base_config = get_base_config(task, language_prior)

    # Return combined configs
    return safe_merge_dicts(base_config, user_config)

In [ ]:
import click
import pandas as pd

In [ ]:
#int_it
import os

import commentjson as json

def get_base_config(task, language_prior):
    # Load base config
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_common.json"), encoding='utf-8') as f:
        base_config = json.load(f)

    # Load task specific config
    task_config_file = None
    if task in ["regression", None]:
        task_config_file = "config_regression.json"
    elif task in ["control"]:
        task_config_file = "config_control.json"
    else:
        # Custom tasks use config_common.json.
        task_config_file = "config_common.json"
    with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), task_config_file), encoding='utf-8') as f:
        task_config = json.load(f)

    # Load language prior config
    if language_prior:
        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), "config_language.json"), encoding='utf-8') as f:
            language_config = json.load(f)
        task_config = safe_merge_dicts(task_config, language_config)

    return safe_merge_dicts(base_config, task_config)


def load_config(config=None):
    # Load user config
    if isinstance(config, str):
        with open(config, encoding='utf-8') as f:
            user_config = json.load(f)
    elif isinstance(config, dict):
        user_config = config
    else:
        assert config is None, "Config must be None, str, or dict."
        user_config = {}

    # Determine the task and language prior
    try:
        task = user_config["task"]["task_type"]
    except KeyError:
        task = "regression"
        print("WARNING: Task type not specified. Falling back to default task type '{}' to load config.".format(task))
    try:
        language_prior = user_config["prior"]["language_model"]["on"]
    except KeyError:
        language_prior = False

    # Load task-specific base config
    base_config = get_base_config(task, language_prior)

    # Return combined configs
    return safe_merge_dicts(base_config, user_config)


In [ ]:
import os

# Specify the path to the directory you want to create
save_path = "/content/output"

# Create the directory
os.makedirs(save_path, exist_ok=True)


In [ ]:
def main():
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/path/to/your/save/directory",
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",
            "dataset": "path/to/your/dataset.csv",
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }


In [ ]:
from copy import deepcopy

In [ ]:
def train_dso(config):
    """Trains DSO and returns dict of reward, expression, and traversal"""

    print("\n== TRAINING SEED {} START ============".format(config["experiment"]["seed"]))

    # For some reason, for the control task, the environment needs to be instantiated
    # before creating the pool. Otherwise, gym.make() hangs during the pool initializer
    if config["task"]["task_type"] == "control" and config["training"]["n_cores_batch"] > 1:
        import gym
        import dso.task.control # Registers custom and third-party environments
        gym.make(config["task"]["env"])

    # Train the model
    model = DeepSymbolicOptimizer(deepcopy(config))
    start = time.time()
    result = model.train()
    result["t"] = time.time() - start
    result.pop("program")

    save_path = model.config_experiment["save_path"]
    summary_path = os.path.join(save_path, "summary.csv")
    print("== TRAINING SEED {} END ==============".format(config["experiment"]["seed"]))

    return result, summary_path

In [ ]:
def print_summary(config, runs, messages):
    text = '\n== EXPERIMENT SETUP START ===========\n'
    text += 'Task type            : {}\n'.format(config["task"]["task_type"])
    if config["task"]["task_type"] == "regression":
        text += 'Dataset              : {}\n'.format(config["task"]["dataset"])
    elif config["task"]["task_type"] == "control":
        text += 'Environment          : {}\n'.format(config["task"]["env"])
    text += 'Starting seed        : {}\n'.format(config["experiment"]["seed"])
    text += 'Runs                 : {}\n'.format(runs)
    if len(messages) > 0:
        text += 'Additional context   :\n'
        for message in messages:
            text += "      {}\n".format(message)
    text += '== EXPERIMENT SETUP END ============='
    print(text)

In [ ]:
def main():
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/path/to/your/save/directory",
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",
            "dataset": "/content/train_feature.csv",
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }
    print("\n== TRAINING START ============")  # Optional starting message
    results = []
    for i in range(runs):
        config["experiment"]["seed"] = seed + i  # Increment seed

        result = train_dso(deepcopy(config))  # Deepcopy avoids overwriting

        # Assuming 'result' already includes a performance metric (e.g., 'reward')
        best_reward = result.get("reward", -1)  # Or your relevant metric's key

        results.append(result)
        print(f"Run {i} Best Reward: {best_reward}")

    print("== TRAINING END ==============")
    print("Training complete!")

def train_dso(config):
  model = DeepSymbolicOptimizer(config)
  start = time.time()
  result = model.train()
  result["t"] = time.time() - start
  return result


In [ ]:
import time
from datetime import datetime
from copy import deepcopy


def main():
    print("Script started.")
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/path/to/your/save/directory",
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",  # Ensure task type matches your problem
            "dataset": "/content/train_feature.csv",  # Updated dataset path
            "validation_dataset": "/content/train_feature.csv",  # Added validation dataset path
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",  # Confirm the evaluation metric
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }

    print("\n== TRAINING START ============")
    results = []
    for i in range(runs):
        config["experiment"]["seed"] = seed + i  # Increment seed

        result = train_dso(deepcopy(config))  # Deepcopy avoids overwriting

        # Assuming 'result' includes a performance metric
        best_reward = result.get("reward", -1)

        results.append(result)
        print(f"Run {i} Best Reward: {best_reward}")

    print("== TRAINING END ==============")
    print("Training complete!")

def train_dso(config):
    model = DeepSymbolicOptimizer(config)
    start = time.time()
    result = model.train()
    result["t"] = time.time() - start
    return result


In [ ]:
import time
from datetime import datetime
from copy import deepcopy
# Ensure you have imported the DeepSymbolicOptimizer class from its respective module

class DeepSymbolicOptimizer:
    # Dummy class definition - Replace with actual implementation
    def __init__(self, config):
        self.config = config

    def train(self):
        # Dummy train method - Replace with actual implementation
        print("Training model...")
        # Simulate training process
        time.sleep(5)
        print("Training complete.")
        return {"reward": 20 }  # Replace with actual reward

def main():
    print("Script started.")
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/mnt/data/your_save_directory",  # Update to your actual save path
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",
            "dataset": "/content/train_feature.csv",  # Updated dataset path
            "validation_dataset": "/content/validation_feature.csv",  # Updated validation dataset path
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }

    print("\n== TRAINING START ============")
    results = []
    for i in range(runs):
        print(f"Starting run {i}...")  # Indicates which run is starting.
        config["experiment"]["seed"] = seed + i  # Increment seed

        result = train_dso(deepcopy(config))  # Deepcopy avoids overwriting

        # Assuming 'result' includes a performance metric
        best_reward = result.get("reward", -1)
        results.append(result)
        print(f"Run {i} Best Reward: {best_reward}")

    print("== TRAINING END ==============")
    print("Training complete!")

def train_dso(config):
    print("Training DSO...")  # Indicates the start of the training process.
    model = DeepSymbolicOptimizer(config)
    start = time.time()
    result = model.train()
    result["t"] = time.time() - start
    return result

if __name__ == "__main__":
    main()


Script started.

== TRAINING START ============
Starting run 0...
Training DSO...
Training model...
Training complete.
Run 0 Best Reward: 20
Starting run 1...
Training DSO...
Training model...
Training complete.
Run 1 Best Reward: 20
Starting run 2...
Training DSO...
Training model...
Training complete.
Run 2 Best Reward: 20
Starting run 3...
Training DSO...
Training model...
Training complete.
Run 3 Best Reward: 20
Starting run 4...
Training DSO...
Training model...
Training complete.
Run 4 Best Reward: 20
== TRAINING END ==============
Training complete!


In [ ]:
import time
from datetime import datetime
from copy import deepcopy
class DeepSymbolicOptimizer:
  def __init__(self, config):
        self.config = config
  def train(self):
        print("Training model...")
        self.optimizer.fit()
        print("Training complete.")
        return {"reward": 42}  # Placeholder for the reward

  def get_equation(self):
        return self.optimizer.best_expression

   # Get the best expression from DSO

def train_dso(config):
    print("Training DSO...")
    model = DeepSymbolicOptimizer(config)
    start = time.time()
    result = model.train()
    equation = model.get_equation()  # Extract the equation
    result["equation"] = equation  # Store the equation in the result dictionary
    result["t"] = time.time() - start
    return result
def get_equation(self):
        return self.optimizer.best_expression
def main():
    print("Script started.")
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/mnt/data/your_save_directory",  # Update to your actual save path
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",
            "dataset": "/content/train_feature.csv",  # Updated dataset path
            "validation_dataset": "/content/validation_feature.csv",  # Updated validation dataset path
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }

    print("\n== TRAINING START ============")
    print("Script started.")
    # Configuration and other setup
    results = []
    for i in range(runs):
        print(f"Starting run {i}...")
        config["experiment"]["seed"] = seed + i
        result = train_dso(deepcopy(config))
        best_reward = result.get("reward", -1)
        equation = result.get("equation", "No equation found")
        print(f"Run {i} Best Reward: {best_reward}, Equation: {equation}")
        results.append(result)
    print("== TRAINING END ==============")
    print("Training complete!")

if __name__ == "__main__":
    main()


Script started.

== TRAINING START ============
Script started.
Starting run 0...
Training DSO...
Training model...


AttributeError: 'DeepSymbolicOptimizer' object has no attribute 'optimizer'

In [ ]:
def main():
    print("Script started.")
    # Hardcoded parameters
    runs = 5
    n_cores_task = 4
    seed = 42
    exp_name = "DSO_Example"

    # Define the configuration directly within the script
    config = {
        "experiment": {
            "seed": seed,
            "save_path": "/mnt/data/your_save_directory",  # Update to your actual save path
            "exp_name": exp_name,
            "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
        },
        "task": {
            "task_type": "regression",
            "dataset": "/content/train_feature.csv",  # Updated dataset path
            "validation_dataset": "/content/validation_feature.csv",  # Updated validation dataset path
            "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
            "metric": "inv_nrmse",
            "metric_params": [1.0],
            "threshold": 1e-12,
            "protected": False,
            "reward_noise": 0.0,
            "reward_noise_type": "r",
            "normalize_variance": False
        },
        "training": {
            "n_samples": 2000000,
            "batch_size": 1000,
            "epsilon": 0.05,
            "n_cores_batch": 1
        },
        "policy_optimizer": {
            "learning_rate": 0.0005,
            "entropy_weight": 0.03,
            "entropy_gamma": 0.7
        },
        "prior": {
            "length": {"min_": 4, "max_": 64, "on": True},
            "repeat": {"tokens": "const", "max_": 3, "on": True},
            "inverse": {"on": True},
            "trig": {"on": True},
            "const": {"on": True},
            "no_inputs": {"on": True},
            "uniform_arity": {"on": True},
            "soft_length": {"loc": 10, "scale": 5, "on": True},
            "domain_range": {"on": False}
        }
    }


In [ ]:
import time
from datetime import datetime
from copy import deepcopy

# Dummy classes for illustration - replace with actual imports from DSO or your implementation
class Task(ABC):
    """
    Object specifying a symbolic search task.

    Attributes
    ----------
    library : Library
        Library of Tokens.

    stochastic : bool
        Whether the reward function of the task is stochastic.

    task_type : str
        Task type: regression, control, or binding.

    name : str
        Unique name for instance of this task.
    """

    task_type = None

    @abstractmethod
    def reward_function(self, program, optimizing=False):
        """
        The reward function for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to compute reward of.

        optimizing : bool

            Whether the reward is computed for PlaceholderConstant optimization.

        Returns
        -------
        reward : float

            Fitness/reward of the program.
        """
        raise NotImplementedError

    @abstractmethod
    def evaluate(self, program):
        """
        The evaluation metric for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to evaluate.

        Returns
        -------

        info : dict

            Dictionary of evaluation metrics. Special key "success" is used to
            trigger early stopping.
        """
        raise NotImplementedError

    @abstractmethod
    def get_next_obs(self, actions, obs, already_finished):
        """
        Produce the next observation and prior from the current observation and
        list of actions so far. Observations must be 1-D np.float32 vectors.

        Parameters
        ----------

        actions : np.ndarray (dtype=np.int32)
            Actions selected so far, shape (batch_size, current_length)

        obs : np.ndarray (dtype=np.float32)
            Previous observation, shape (batch_size, OBS_DIM).

        already_finished : np.ndarray (dtype=bool)
            Whether the object has *already* been completed.

        Returns
        -------

        next_obs : np.ndarray (dtype=np.float32)
            The next observation, shape (batch_size, OBS_DIM).

        prior : np.ndarray (dtype=np.float32)
            Prior for selecting the next token, shape (batch_size,
            self.library.L).

        finished : np.ndarray (dtype=bool)
            Whether the object has *ever* been completed.
        """
        pass

    @abstractmethod
    def reset_task(self):
        """
        Create the starting observation.

        Returns
        -------

        obs : np.ndarray (dtype=np.float32)
            Starting observation, shape (batch_size, OBS_DIM).
        """
        pass


class HierarchicalTask(Task):
    """
    A Task in which the search space is a binary tree. Observations include
    the previous action, the parent, the sibling, and/or the number of dangling
    (unselected) nodes.
    """

    OBS_DIM = 4 # action, parent, sibling, dangling

    def __init__(self):
        super(Task).__init__()

    def get_next_obs(self, actions, obs, already_finished):

        dangling = obs[:, 3] # Shape of obs: (?, 4)
        action = actions[:, -1] # Current action
        lib = self.library

        # Compute parents and siblings
        parent, sibling = parents_siblings(actions,
                                           arities=lib.arities,
                                           parent_adjust=lib.parent_adjust,
                                           empty_parent=lib.EMPTY_PARENT,
                                           empty_sibling=lib.EMPTY_SIBLING)

        # Compute dangling
        dangling += lib.arities[action] - 1

        # Compute finished
        just_finished = (dangling == 0) # Trees that completed _this_ time step
        # [batch_size]
        finished = np.logical_or(just_finished,
                                 already_finished)

        # Compute priors
        prior = self.prior(actions, parent, sibling, dangling, finished) # (?, n_choices)

        # Combine observation dimensions
        next_obs = np.stack([action, parent, sibling, dangling], axis=1) # (?, 4)
        next_obs = next_obs.astype(np.float32)

        return next_obs, prior, finished

    def reset_task(self, prior):
        """
        Returns the initial observation: empty action, parent, and sibling, and
        dangling is 1.
        """

        self.prior = prior

        # Order of observations: action, parent, sibling, dangling
        initial_obs = np.array([self.library.EMPTY_ACTION,
                                self.library.EMPTY_PARENT,
                                self.library.EMPTY_SIBLING,
                                1],
                               dtype=np.float32)
        return initial_obs


class SequentialTask(Task):
    """
    A Task in which the search space is a (possibly variable-length) sequence.
    The observation is simply the previous action.
    """

    pass


def make_task(task_type, **config_task):
    """
    Factory function for Task object.

    Parameters
    ----------

    task_type : str
        Type of task:
        "regression" : Symbolic regression task.
        "control" : Episodic reinforcement learning task.
        "binding": AbAg binding affinity optimization task.

    config_task : kwargs
        Task-specific arguments. See specifications of task_dict.

    Returns
    -------

    task : Task
        Task object.
    """

    # Lazy import of task factory functions
    if task_type == 'binding':
        from dso.task.binding.binding import BindingTask
        task_class = BindingTask
    elif task_type == "regression":
        from dso.task.regression.regression import RegressionTask
        task_class = RegressionTask
    elif task_type == "control":
        from dso.task.control.control import ControlTask
        task_class = ControlTask
    else:
        # Custom task import
        task_class = import_custom_source(task_type)
        assert issubclass(task_class, Task), \
            "Custom task {} must subclass dso.task.Task.".format(task_class)

    task = task_class(**config_task)
    return task


def set_task(config_task):
    """Helper function to make set the Program class Task and execute function
    from task config."""

    # Use of protected functions is the same for all tasks, so it's handled separately
    protected = config_task["protected"] if "protected" in config_task else False

    Program.set_execute(protected)
    task = make_task(**config_task)
    Program.set_task(task)

class LanguageModel(object):
    def __init__(self, vocabulary_size, embedding_size, num_layers, num_hidden, mode='train'):
        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.num_hidden = num_hidden

        self.x = tf.compat.v1.placeholder(tf.int32, [None, None], name="x") # whole seq + seq len
        self.keep_prob = tf.compat.v1.placeholder(tf.float32, [], name="keep_prob")
        self.batch_size = tf.compat.v1.shape(self.x)[0]

        if mode == 'train':
            self.lm_input = self.x[:, :-2]
            self.seq_len = self.x[:, -1]
        elif mode == 'predict':
            self.lm_input = self.x[:,:]
            self.seq_len = tf.reduce_sum(tf.sign(self.lm_input), 1)

        # Embedding (optional)
        with tf.name_scope("embedding"):
            init_embeddings = tf.random.uniform([vocabulary_size, self.embedding_size])
            embeddings = tf.compat.v1.get_variable("embeddings", initializer=init_embeddings)
            lm_input_emb = tf.nn.embedding_lookup(embeddings, self.lm_input)

        with tf.compat.v1.variable_scope("rnn"):
            def make_cell():
                cell = BasicRNNCell(self.num_hidden)  # Or use LSTM(...) for better performance
                cell = Dropout(1 - self.keep_prob)(cell)
                return cell

            cell = RNN([make_cell() for _ in range(self.num_layers)])

            self.initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)
            rnn_outputs, self.last_state = tf.nn.dynamic_rnn(
                cell=cell,
                initial_state=self.initial_state,
                inputs=lm_input_emb,  # Assuming you're using embeddings
                sequence_length=self.seq_len,
                dtype=tf.float32)

        with tf.name_scope("output"):
            self.logits = Dense(vocabulary_size)(rnn_outputs)  # Use tf.layers.dense for TF < 2.0

        with tf.name_scope("loss"):
            if mode == "train":
                target = self.x[:, 1:-1]
            elif mode == "predict":
                target = self.x[:, :]

            self.loss = SparseCategoricalCrossentropy(from_logits=True)(target, self.logits)

class DeepSymbolicOptimizer():
    """
    Deep symbolic optimization model. Includes model hyperparameters and
    training configuration.

    Parameters
    ----------
    config : dict or str
        Config dictionary or path to JSON.

    Attributes
    ----------
    config : dict
        Configuration parameters for training.

    Methods
    -------
    train
        Builds and trains the model according to config.
    """

    def __init__(self, config=None):
        self.set_config(config)
        self.sess = None

    def setup(self):

        # Clear the cache and reset the compute graph
        Program.clear_cache()
        tf.reset_default_graph()

        # Generate objects needed for training and set seeds
        self.pool = self.make_pool_and_set_task()
        self.set_seeds() # Must be called _after_ resetting graph and _after_ setting task

        # Limit TF to single thread to prevent "resource not available" errors in parallelized runs
        session_config = tf.ConfigProto(intra_op_parallelism_threads=1,
                                        inter_op_parallelism_threads=1)
        self.sess = tf.Session(config=session_config)

        # Setup logdirs and output files
        self.output_file = self.make_output_file()
        self.save_config()

        # Prepare training parameters
        self.prior = self.make_prior()
        self.state_manager = self.make_state_manager()
        self.policy = self.make_policy()
        self.policy_optimizer = self.make_policy_optimizer()
        self.gp_controller = self.make_gp_controller()
        self.logger = self.make_logger()
        self.trainer = self.make_trainer()
        self.checkpoint = self.make_checkpoint()

    def train_one_step(self, override=None):
        """
        Train one iteration.
        """

        # Setup the model
        if self.sess is None:
            self.setup()

        # Run one step
        assert not self.trainer.done, "Training has already completed!"
        self.trainer.run_one_step(override)

        # Maybe save next checkpoint
        self.checkpoint.update()

        # If complete, return summary
        if self.trainer.done:
            return self.finish()

    def train(self):
        """
        Train the model until completion.
        """

        # Setup the model
        self.setup()

        # Train the model until done
        while not self.trainer.done:
            result = self.train_one_step()

        return result

    def finish(self):
        """
        After training completes, finish up and return summary dict.
        """

        # Return statistics of best Program
        p = self.trainer.p_r_best
        result = {"seed" : self.config_experiment["seed"]} # Seed listed first
        result.update({"r" : p.r})
        result.update(p.evaluate)
        result.update({
            "expression" : repr(p.sympy_expr),
            "traversal" : repr(p),
            "program" : p
        })

        # Save all results available only after all iterations are finished. Also return metrics to be added to the summary file
        results_add = self.logger.save_results(self.pool, self.trainer.nevals)
        result.update(results_add)

        # Close the pool
        if self.pool is not None:
            self.pool.close()

        return result

    def set_config(self, config):
        config = load_config(config)

        self.config = defaultdict(dict, config)
        self.config_task = self.config["task"]
        self.config_prior = self.config["prior"]
        self.config_logger = self.config["logging"]
        self.config_training = self.config["training"]
        self.config_state_manager = self.config["state_manager"]
        self.config_policy = self.config["policy"]
        self.config_policy_optimizer = self.config["policy_optimizer"]
        self.config_gp_meld = self.config["gp_meld"]
        self.config_experiment = self.config["experiment"]
        self.config_checkpoint = self.config["checkpoint"]

    def save_config(self):
        # Save the config file
        if self.output_file is not None:
            path = os.path.join(self.config_experiment["save_path"],
                                "config.json")
            # With run.py, config.json may already exist. To avoid race
            # conditions, only record the starting seed. Use a backup seed
            # in case this worker's seed differs.
            backup_seed = self.config_experiment["seed"]
            if not os.path.exists(path):
                if "starting_seed" in self.config_experiment:
                    self.config_experiment["seed"] = self.config_experiment["starting_seed"]
                    del self.config_experiment["starting_seed"]
                with open(path, 'w') as f:
                    json.dump(self.config, f, indent=3)
            self.config_experiment["seed"] = backup_seed

    def set_seeds(self):
        """
        Set the tensorflow, numpy, and random module seeds based on the seed
        specified in config. If there is no seed or it is None, a time-based
        seed is used instead and is written to config.
        """

        seed = self.config_experiment.get("seed")

        # Default uses current time in milliseconds, modulo 1e9
        if seed is None:
            seed = round(time() * 1000) % int(1e9)
            self.config_experiment["seed"] = seed

        # Shift the seed based on task name
        # This ensures a specified seed doesn't have similarities across different task names
        task_name = Program.task.name
        shifted_seed = seed + zlib.adler32(task_name.encode("utf-8"))

        # Set the seeds using the shifted seed
        tf.set_random_seed(shifted_seed)
        np.random.seed(shifted_seed)
        random.seed(shifted_seed)

    def make_prior(self):
        prior = make_prior(Program.library, self.config_prior)
        return prior

    def make_state_manager(self):
        state_manager = make_state_manager(self.config_state_manager)
        return state_manager

    def make_trainer(self):
        trainer = Trainer(self.sess,
                          self.policy,
                          self.policy_optimizer,
                          self.gp_controller,
                          self.logger,
                          self.pool,
                          **self.config_training)
        return trainer

    def make_logger(self):
        logger = StatsLogger(self.sess,
                             self.output_file,
                             **self.config_logger)
        return logger

    def make_checkpoint(self):
        checkpoint = Checkpoint(self,
                                **self.config_checkpoint)
        return checkpoint

    def make_policy_optimizer(self):
        policy_optimizer = make_policy_optimizer(self.sess,
                                                 self.policy,
                                                 **self.config_policy_optimizer)
        return policy_optimizer

    def make_policy(self):
        policy = make_policy(self.sess,
                             self.prior,
                             self.state_manager,
                             **self.config_policy)
        return policy

    def make_gp_controller(self):
        if self.config_gp_meld.pop("run_gp_meld", False):
            from dso.gp.gp_controller import GPController
            gp_controller = GPController(self.prior,
                                         self.config_prior,
                                         **self.config_gp_meld)
        else:
            gp_controller = None
        return gp_controller

    def make_pool_and_set_task(self):
        # Create the pool and set the Task for each worker

        # Set complexity and const optimizer here so pool can access them
        # Set the complexity function
        complexity = self.config_training["complexity"]
        Program.set_complexity(complexity)

        # Set the constant optimizer
        const_optimizer = self.config_training["const_optimizer"]
        const_params = self.config_training["const_params"]
        const_params = const_params if const_params is not None else {}
        Program.set_const_optimizer(const_optimizer, **const_params)

        pool = None
        n_cores_batch = self.config_training.get("n_cores_batch")
        if n_cores_batch is not None:
            if n_cores_batch == -1:
                n_cores_batch = cpu_count()
            if n_cores_batch > 1:
                pool = Pool(n_cores_batch,
                            initializer=set_task,
                            initargs=(self.config_task,))

        # Set the Task for the parent process
        set_task(self.config_task)

        return pool

    def make_output_file(self):
        """Generates an output filename"""

        # If logdir is not provided (e.g. for pytest), results are not saved
        if self.config_experiment.get("logdir") is None:
            self.save_path = None
            print("WARNING: logdir not provided. Results will not be saved to file.")
            return None

        # When using run.py, timestamp is already generated
        timestamp = self.config_experiment.get("timestamp")
        if timestamp is None:
            timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            self.config_experiment["timestamp"] = timestamp

        # Generate save path
        task_name = Program.task.name
        if self.config_experiment["exp_name"] is None:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                '_'.join([task_name, timestamp]))
        else:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                self.config_experiment["exp_name"])

        self.config_experiment["task_name"] = task_name
        self.config_experiment["save_path"] = save_path
        os.makedirs(save_path, exist_ok=True)

        seed = self.config_experiment["seed"]
        output_file = os.path.join(save_path,
                                   "dso_{}_{}.csv".format(task_name, seed))

        self.save_path = save_path

        return output_file

    def save(self, save_path=None):
        self.checkpoint.save(save_path)

    def load(self, load_path):
        self.checkpoint.load(load_path)

# Configuration for DSO
config = {
    "experiment": {
        "seed": 42,
        "save_path": "/mnt/data/your_save_directory",
        "exp_name": "DSO_Example",
        "timestamp": datetime.now().strftime("%Y-%m-%d-%H%M%S")
    },
    "task": {
        "task_type": "regression",
        "dataset": "/content/train_feature.csv",
        "validation_dataset": "/content/validation_feature.csv",
        "function_set": ["add", "sub", "mul", "div", "sin", "cos", "exp", "log"],
        "metric": "inv_nrmse",
        "metric_params": [1.0],
        "threshold": 1e-12,
        "protected": False,
        "reward_noise": 0.0,
        "reward_noise_type": "r",
        "normalize_variance": False
    },
    "training": {
        "n_samples": 2000000,
        "batch_size": 1000,
        "epsilon": 0.05,
        "n_cores_batch": 1
    },
    "policy_optimizer": {
        "learning_rate": 0.0005,
        "entropy_weight": 0.03,
        "entropy_gamma": 0.7
    },
    "prior": {
        "length": {"min_": 4, "max_": 64, "on": True},
        "repeat": {"tokens": "const", "max_": 3, "on": True},
        "inverse": {"on": True},
        "trig": {"on": True},
        "const": {"on": True},
        "no_inputs": {"on": True},
        "uniform_arity": {"on": True},
        "soft_length": {"loc": 10, "scale": 5, "on": True},
        "domain_range": {"on": False}
    }
    "rnn": {
        "vocabulary_size": 10000,
        "embedding_size": 300,
        "num_layers": 2,
        "num_hidden": 128,
        "mode": 'train'
    }
}

def main():
    # Data loading logic
    train_features = pd.read_csv("/mnt/data/train_feature.csv").values
    train_labels = pd.read_csv("/mnt/data/train_label.csv").values

    # Instantiate the RNN model
    rnn_model = LanguageModel(**config["rnn"])

    # Create the task instance
    if config["task"]["task_type"] == "regression":
        task = RegressionTask(rnn_model, (train_features, train_labels))
    else:
        raise ValueError("Unsupported task type")

    # Initialize the controller and optimizer
    optimizer_config = {}  # Placeholder for actual optimizer config
    optimizer = DeepSymbolicOptimizer(config=optimizer_config)
    optimizer.setup()  # Setup all components and dependencies

    # Run the training process
    result = optimizer.train()

    # Output the results
    print("Training completed with result:", result)

if __name__ == "__main__":
    main()


TypeError: Task() takes no arguments

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import RNN, LSTM, Dropout, Dense
from tensorflow.compat.v1.nn.rnn_cell import BasicRNNCell

from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
pip install tensorflow==1.16  # This installs the last TensorFlow 1.x version


ERROR: Could not find a version that satisfies the requirement tensorflow==1.16 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==1.16


In [ ]:
#polyfit
import numpy as np
import scipy
from scipy import linalg, optimize, stats
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from itertools import compress

class PolyRegressorMixin:
    """
    Defines auxiliary functions to be used by DSO's specialized regressors
    """
    def np_array_signature(self, X):
        """
        Computes simplified hash of matrix X (m rows, n columns, m > n) for polynomial fitting purposes.
        Parameters
        ==========
        X : ndarray
            X data

        Returns
        =======
        result : int
            Simplified hash of X.
        """
        return hash((X.shape,                                               # array shape
                     X.diagonal().tobytes(),                                # main (top) diagonal
                     X.diagonal(offset=X.shape[1]-X.shape[0]).tobytes()))   # lowest diagonal

    def delete_oldest_pair(self, dictionary):
        """
        Deletes oldest (key, value) pair from dictionary.
        Takes advantage of ordered dictionaries in Python 3.6 and newer.
        """
        dictionary.pop(next(iter(dictionary)))

    def zero_out_ls_terms(self, cLS, XtX_inv, zero_out_indices):
        """
        Fast recomputation of least-squares fit when zeroing-out terms in the regression
        Solves:  [ XtX_inv   indexing^T ] [ c ] == [ Xt * y ]
                 [ indexing      0      ] [ z ]    [    0   ]
        which corresponds to the optimality conditions of:
                max_c || X c - y || : indexing * c = 0
        """
        # 1. form D = XtX_inv * indexing^T = (indexing * XtX_inv)^T = XtX_inv[indexing,:]^T
        D = np.ascontiguousarray(XtX_inv[zero_out_indices,:].transpose())
        # 2. form E = indexing * D = D[indexing,:]
        E = D[zero_out_indices,:]
        # NOTE: E is just a minor of XtX_inv, hence it is symmetric and positive definite
        # 3. solve linear system E * z = cLS[indexing]
        z = scipy.linalg.solve(E, cLS[zero_out_indices], assume_a="pos")    # take advantage D is PD
        # 4. compute solution with zero-ed out components and return
        return cLS - np.matmul(D, z)

    def regression_p_values(self, X, XtX_inv, y, c):
        """
        Computes p-values using t-Test (null hyphotesis: c_i == 0)
        """
        yhat = np.matmul(X, c)
        df = len(X) - X.shape[1]
        mse = sum((y - yhat)**2)/df
        sd_err = np.sqrt(mse * XtX_inv.diagonal())
        t_vals = c/sd_err
        return 2 * (1 - stats.t.cdf(np.abs(t_vals), df))


class DSOLeastSquaresData:
    """
    Holds Gram inverse and pseudo-inverse
    """
    def __init__(self, X, compute_inv=False):
        if X.shape[0] < X.shape[1]:
            raise AssertionError("X should have more rows than columns.")
        self.X_pinv = scipy.linalg.pinv(X)
        if compute_inv:
            XtX = np.matmul(X.transpose(), X)
            if not np.isfinite(np.linalg.cond(XtX)):
                raise AssertionError("X^t * X should always be invertible.")
            self.XtX_inv = scipy.linalg.inv(XtX)
        else:
            self.XtX_inv = None

class DSOLeastSquaresRegressor(PolyRegressorMixin):
    """
    Solve the problem min_{c} || X*c - y || by applying the psuedo-inverse
            c = (X^T*X)^{-1} * X^T * y
    """
    def __init__(self, cutoff_p_value=1.0, n_max_terms=None, coef_tol=1E-12):
        # include intercept_ just to match with sklearn regressors
        self.intercept_ = 0.0
        self.coef_ = None
        self.n_max_records = 10
        self.data_dict = {}
        if isinstance(cutoff_p_value, float) and \
           cutoff_p_value > 0.0 and cutoff_p_value <= 1.0:
            self.cutoff_p_value_ = cutoff_p_value
        else:
            raise ValueError("cutoff p-value should be in (0., 1.]")
        if (isinstance(n_max_terms, int) and n_max_terms >= 2) or n_max_terms is None:
           # 2 terms: constant + term
            self.n_max_terms_ = n_max_terms
        elif isinstance(n_max_terms, int):
            raise ValueError("maximum number of terms should be >= 2")
        else:
            raise TypeError("n_max_terms should be int or None")
        self.coef_tol_ = coef_tol

    def fit(self, X, y, X_signature=None):
        """
        Linear fit between X (data) and y (observations)
        """
        # check if data is cached, if not add it to the data_dict
        if X_signature is None:
            X_signature = self.np_array_signature(X)
        if X_signature not in self.data_dict.keys():
            while len(self.data_dict) >= self.n_max_records:
                self.delete_oldest_pair(self.data_dict)
            self.data_dict[X_signature] = DSOLeastSquaresData(X, self.cutoff_p_value_ < 1.0 or
                                                                 self.n_max_terms_ is not None)
        # perform regression
        lsd = self.data_dict[X_signature]
        self.coef_ = np.matmul(lsd.X_pinv, y)
        # if necessary, enforce p-value cutoff and maximum number of terms equal to zero
        if self.cutoff_p_value_ < 1.0 or \
           (self.n_max_terms_ is not None and \
            np.count_nonzero(np.abs(self.coef_) > self.coef_tol_) > self.n_max_terms_):
            # compute p-values for all coefficients
            p_values = self.regression_p_values(X, lsd.XtX_inv, y, self.coef_)
            # sort coefficients from smallets to largest p-value
            perm = np.argsort(p_values)
            # compute number of terms to keep
            n_terms = next((x[0] for x in enumerate(perm) if p_values[x[1]] > self.cutoff_p_value_),
                           len(p_values))
            if self.n_max_terms_ is not None:
                n_terms = np.minimum(n_terms, self.n_max_terms_)
            # zero-out coefficients with largest p-values
            if n_terms < len(self.coef_):
                zero_out_indices = np.sort(perm[n_terms:])
                self.coef_ = self.zero_out_ls_terms(self.coef_, lsd.XtX_inv, zero_out_indices)

    def clear(self):
        """
        Reset memory allocated to pseudo-inverses
        """
        self.data_dict.clear()


class DSOLassoRegressorData:
    """
    Holds information useful for multiple calls to DSOLassoRegressor
    """
    def __init__(self, X):
        self.XtX_inv = scipy.linalg.inv(np.matmul(X.transpose(), X))
        self.X_pinv = np.matmul(self.XtX_inv, X.transpose())
        self.n_obs = X.shape[0]
        self.n_params = X.shape[1]


class DSOLassoRegressor(PolyRegressorMixin):
    """
    Computes Lasso for X, y with gamma weighted L1 regularization, i.e. finds optimum beta for
        min_{beta} (1/2 * 1/var(y) * 1/n_obs * || y - X * beta ||^2_2 + gamma * 1/n_params * || beta ||_1)

    Implementation solves dual Lasso problem.
    """
    def __init__(self, gamma=0.1, comp_tol=1E-4, rtrn_constrnd_ls=True):
        # include intercept_ just to match with sklearn regressors
        self.intercept_ = 0.0
        self.coef_ = None
        self.gamma_ = gamma         # L1 weight -- standarized
        self.comp_tol_ = comp_tol   # tolerance for complementarity slackness
        self.rtrn_constrnd_ls_ = rtrn_constrnd_ls       # return re-optimized sparse least-squares
        self.data_dict = {}
        self.n_max_records = 10

    def fit(self, X, y, X_signature=None):
        # check if signature is provided, compute signature otherwise
        if X_signature is None:
            X_signature = self.np_array_signature(X)
        # if signature does not appear in dict, compute and store lasso regressor data
        if X_signature not in self.data_dict.keys():
            while len(self.data_dict) >= self.n_max_records:
                self.delete_oldest_pair(self.data_dict)
            self.data_dict[X_signature] = DSOLassoRegressorData(X)
        # perform lasso fit
        ldata = self.data_dict[X_signature]
        self.coef_ = self.dual_lasso(ldata.XtX_inv, ldata.X_pinv,
                                     ldata.n_obs, ldata.n_params, y)

    def dual_lasso(self, XtX_inv, X_pinv, n_obs, n_params, y):
        # compute program parameters
        beta_LS = np.matmul(X_pinv, y)  # least squares solution
        rho_bnd = n_obs/n_params * np.var(y) * self.gamma_

        # currently only scipy.minimize is supported as the solver
        # objective function and derivatives
        f_obj = lambda rho : 1/2 * np.dot(rho, np.matmul(XtX_inv, rho)) - np.dot(beta_LS, rho)
        g_obj = lambda rho : np.matmul(XtX_inv, rho) - beta_LS
        # define initial guess
        rho_init = rho_bnd * np.ones(n_params)
        rho_init[beta_LS > 0.0] *= -1.0
        # call scipy minimize
        bnds = scipy.optimize.Bounds(-rho_bnd * np.ones(n_params), rho_bnd * np.ones(n_params))
        res = scipy.optimize.minimize(f_obj, rho_init, jac=g_obj, bounds=bnds)
        if not res.success:
            raise Exception("failed to solve dual lasso problem.")
        rho_opt = res.x

        if self.rtrn_constrnd_ls_:
            # determine indexes to zero-out
            zero_out_indices = [i for i in range(n_params) if
                                0.25 * (1 + rho_opt[i]/rho_bnd) * (1 - rho_opt[i]/rho_bnd) > self.comp_tol_]
            # recompute least squares with zero-ed out coefficients
            beta_cLS = self.zero_out_ls_terms(beta_LS, XtX_inv, zero_out_indices)
            beta_cLS[zero_out_indices] = 0.0    # these will be zero up to floating point precision
            return beta_cLS
        else:
            # compute lasso regressor parameters
            beta_Lasso = beta_LS - np.matmul(XtX_inv, rho_opt)
            # crash (dual) interior interior point solution
            for i in range(n_params):
                if 0.25 * (1 + rho_opt[i]/rho_bnd) * (1 - rho_opt[i]/rho_bnd) > self.comp_tol_:
                    beta_Lasso[i] = 0.0
            # return solution
            return beta_Lasso

    def clear(self):
        """
        Reset memory allocated to Gram inverse and pseudo inverse
        """
        self.data_dict.clear()


regressors = {
        "linear_regression": LinearRegression,
        "lasso": Lasso,
        "ridge": Ridge,
        "dso_least_squares" : DSOLeastSquaresRegressor,
        "dso_lasso" : DSOLassoRegressor,
    }

inverse_function_map = {
    "add" : np.subtract,
    "sub" : np.add,
    "mul" : np.divide,
    "div" : np.multiply,
    "sin" : np.arcsin,
    "cos" : np.arccos,
    "tan" : np.arctan,
    "exp" : np.log,
    "log" : np.exp,
    "sqrt" : np.square,
    "n2" : np.sqrt,
    "n3" : np.cbrt,
    "abs" : np.abs,
    "tanh" : np.arctanh,
    "inv" : np.reciprocal
}


def partial_execute(traversal, X):
    """
    Evaluate from terminal nodes all the branches that has no 'poly' token.
    If some (function) value in the partial execution is not finite, None is returned.
    """
    apply_stack = []
    for node in traversal:
        apply_stack.append([node])
        while len(apply_stack[-1]) == apply_stack[-1][0].arity + 1:
            token = apply_stack[-1][0]
            terminals = apply_stack[-1][1:]
            if isinstance(token, Polynomial):
                intermediate_result = "poly"
            elif token.input_var is not None:
                intermediate_result = X[:, token.input_var]
            else:
                if all(isinstance(t, np.ndarray) for t in terminals):
                    if isinstance(token, StateChecker):
                        token.set_state_value(X[:, token.state_index])
                    intermediate_result = token(*terminals)

                    if not np.isfinite(intermediate_result).all():
                        return None
                else:
                    intermediate_result = [token, *terminals]

            if len(apply_stack) != 1:
                apply_stack.pop()
                apply_stack[-1].append(intermediate_result)
            else:
                return intermediate_result


def recursive_inversion(intermediate_result, y):
    """
    Obtain the 'y data' for 'poly' token by inverting tokens starting from root.
    For tokens of arity 2, find out the child that has been evaluated (there must be
    one and only one), and get the value of the other child, until 'poly' is reached.

    If some entry of y is not finite, None is returned.
    """
    if not np.isfinite(y).all():
        return None
    if intermediate_result == "poly":
        return y

    assert len(intermediate_result) < 4
    func = intermediate_result[0]
    if func.arity == 1:
        out = inverse_function_map[func.name](y)
        return recursive_inversion(intermediate_result[1], out)
    else:
        if isinstance(intermediate_result[1], np.ndarray):
            if func.name == "sub" or func.name == "div":
                out = func(intermediate_result[1], y)
            else:
                out = inverse_function_map[func.name](y, intermediate_result[1])
            return recursive_inversion(intermediate_result[2], out)
        else:
            out = inverse_function_map[func.name](y, intermediate_result[2])
            return recursive_inversion(intermediate_result[1], out)


def make_poly_data(traversal, X, y):
    """
    Obtain the 'y data' for 'poly' token in two steps. First is a bottom-up pass of the
    expression tree starting from terminal nodes, all the branches that can be evaluated
    will be evaluated. Effectively this creates a single chain of unary functions with the
    terminal token being the 'poly' token. The second step is a top-down pass inverting
    all the unary functions in partial_results starting from the root.

    If some (function) value in the partial execution or recursive inversion is not finite,
    None is returned.
    """
    partial_results = partial_execute(traversal, X)
    return None if partial_results is None else recursive_inversion(partial_results, y)


def nonnegative_int_tuples_to_sum(length, given_sum):
    """
    generate all tuples of nonnegative integers that are of size length such that sum of entries equals given_sum
    https://stackoverflow.com/questions/7748442/generate-all-possible-lists-of-length-n-that-sum-to-s-in-python
    """
    if length == 1:
        yield (given_sum,)
    else:
        for value in range(given_sum + 1):
            for permutation in nonnegative_int_tuples_to_sum(length - 1, given_sum - value):
                yield (value,) + permutation


def generate_all_exponents(n_input_var, degree):
    """
    Generate a list of tuples of exponents corresponding to all monomials of n_input_var
    variables of degree at most degree.
    """
    out = []
    for monomial_degree in range(degree + 1):
        out.extend(list(nonnegative_int_tuples_to_sum(n_input_var, monomial_degree)))
    return out


class PolyOptimizerData(PolyRegressorMixin):
    """
    Helper class to process and hold data passed to the polynomial optimizer
    """
    def __init__(self, X, degree, X_signature_=None):
        """
        Generate and store the data for all the monomials (basis for poly).
        This allows dso to skip repeated generation of monomials' data for
        the same X data during training.

        Parameters
        ==========
        X : ndarray
            X data
        degree: int
            The (maximal) degree of the polynomials used to fit the data.
        """
        self.all_exponents = generate_all_exponents(X.shape[1], degree)
        self.all_monomials_data = Polynomial.eval_monomials(X, self.all_exponents)
        if X_signature_ is None:
            self.X_signature = self.np_array_signature(X)
        else:
            self.X_signature = X_signature_


class PolyOptimizer(PolyRegressorMixin):
    def __init__(self, degree, coef_tol, regressor, regressor_params):
        """
        Optimizer for fitting a polynomial in traversals to given datasets.

        Parameters
        ==========
        degree : int
            The (maximal) degree of the polynomials used to fit the data.

        coef_tol : float
            Cutoff value for the coefficients of polynomials. Coefficients
            with magnitude less than this value will be regarded as 0.

        regressor : str
            Key to dictionary regressors. Supported options are 'lasso',
            'ridge', and 'linear_regression'.

        regressor_params : dict
            Parameters for the regressor. See sklearn for more information.
        """
        self.degree = degree
        self.coef_tol = coef_tol
        self.regressor = regressors[regressor](**regressor_params)
        self.data_dict = dict()
        self.n_max_records = 10

    def fit(self, X, y):
        """
        Fit a polnomial to the dataset (X, y) based on the regressor.
        Parameters
        ==========
        X : ndarray
            X data
        y : ndarray
            y data

        Returns
        =======
        result : Polynomial(Token)
            The polynomial token of which the underlying polynomial best fits the dataset (X, y)
        """
        X_signature = self.np_array_signature(X)
        if X_signature not in self.data_dict.keys():
            while len(self.data_dict) >= self.n_max_records:
                self.delete_oldest_pair(self.data_dict)
            self.data_dict[X_signature] = PolyOptimizerData(X, self.degree, X_signature)

        # reference to PolyOptimizerData object (to avoid multiple lookups)
        pod = self.data_dict[X_signature]

        try:
            # perform fit; pass monomial data signature if using custom DSO optimizers
            if isinstance(self.regressor, (DSOLeastSquaresRegressor,)):
                self.regressor.fit(pod.all_monomials_data, y, pod.X_signature)
            else:
                self.regressor.fit(pod.all_monomials_data, y)
        except: # the only thing we have seen is ValueError
            return Polynomial([(0,)*X.shape[1]], np.ones(1))

        # Correct the coefficient of the constant term when regressor.intercept_ is nonzero.
        # This can happen when fit_intercept in regressor_params is True.
        if self.regressor.intercept_ != 0.0:
            self.regressor.coef_[0] += self.regressor.intercept_

        if np.isfinite(self.regressor.coef_).all():
            mask = np.abs(self.regressor.coef_) >= self.coef_tol
            if np.count_nonzero(mask) == 0:
                # fit succesful, but all coefficients are zero
                return Polynomial([(0,)*X.shape[1]], np.ones(0))
            return Polynomial(list(compress(pod.all_exponents, mask)), self.regressor.coef_[mask])

        return Polynomial([(0,)*X.shape[1]], np.ones(1))

    def clear(self):
        """
        Reset memory allocated to exponents and monomials data, and to cached regressor data
        """
        self.data_dict.clear()
        if isinstance(self.regressor, (DSOLeastSquaresRegressor, DSOLassoRegressor)):
            self.regressor.clear()



class PolyGenerator(object):
    def __init__(self, degree, n_input_var):
        """
        Parameters
        ----------
        degree : int
            Maximal degree of the polynomials to be generated.
        coef : int
            Number of input (independent) variables.
        """
        self.all_exponents = generate_all_exponents(n_input_var, degree)

    def generate(self, n_terms_mean=2, n_terms_sd=1,
                 coef_mean=0, coef_sd=10, coef_precision=2):
        """
        Generate a Polynomial token. The number of terms and the coefficients of the
        terms are sampled from normal distributions based on the input parameters.
        Parameters
        ----------
        n_terms_mean : int
            Mean of the normal distribution from which number of terms is sampled.
        n_terms_sd : int
            Standard deviation of the normal distribution from which number of terms is sampled.
        coef_mean : float
            Mean of the normal distribution from which the coefficents are sampled.
        coef_sd : float
            Standard deviation of the normal distribution from which the coefficents are sampled.
        coef_precision : int
            Number of decimal places of the coefficients in the generated polynomial.

        Returns
        =======
        result : Polynomial(Token)
            The generated polynomial token
        """
        n_terms = int(max(1, np.random.normal(n_terms_mean, n_terms_sd)))
        n_terms = min(n_terms, len(self.all_exponents))
        coefs = np.random.normal(coef_mean, coef_sd, n_terms)
        coefs = np.around(coefs, decimals=coef_precision)
        coef_pos = np.random.choice(len(self.all_exponents), n_terms, replace=False)
        return Polynomial([self.all_exponents[pos] for pos in coef_pos], coefs)

In [ ]:
#function
"""Common Tokens used for executable Programs."""
import re
import numpy as np
from fractions import Fraction



GAMMA = 0.57721566490153286060651209008240243104215933593992


"""Define custom unprotected operators"""
def logabs(x1):
    """Closure of log for non-positive arguments."""
    return np.log(np.abs(x1))

def expneg(x1):
    return np.exp(-x1)

def n3(x1):
    return np.power(x1, 3)

def n4(x1):
    return np.power(x1, 4)

def sigmoid(x1):
    return 1 / (1 + np.exp(-x1))

def harmonic(x1):
    if all(val.is_integer() for val in x1):
        return np.array([sum(Fraction(1, d) for d in range(1, int(val)+1)) for val in x1], dtype=np.float32)
    else:
        return GAMMA + np.log(x1) + 0.5/x1 - 1./(12*x1**2) + 1./(120*x1**4)


# Annotate unprotected ops
unprotected_ops = [
    # Binary operators
    Token(np.add, "add", arity=2, complexity=1),
    Token(np.subtract, "sub", arity=2, complexity=1),
    Token(np.multiply, "mul", arity=2, complexity=1),
    Token(np.divide, "div", arity=2, complexity=2),

    # Built-in unary operators
    Token(np.sin, "sin", arity=1, complexity=3),
    Token(np.cos, "cos", arity=1, complexity=3),
    Token(np.tan, "tan", arity=1, complexity=4),
    Token(np.exp, "exp", arity=1, complexity=4),
    Token(np.log, "log", arity=1, complexity=4),
    Token(np.sqrt, "sqrt", arity=1, complexity=4),
    Token(np.square, "n2", arity=1, complexity=2),
    Token(np.negative, "neg", arity=1, complexity=1),
    Token(np.abs, "abs", arity=1, complexity=2),
    Token(np.maximum, "max", arity=1, complexity=4),
    Token(np.minimum, "min", arity=1, complexity=4),
    Token(np.tanh, "tanh", arity=1, complexity=4),
    Token(np.reciprocal, "inv", arity=1, complexity=2),

    # Custom unary operators
    Token(logabs, "logabs", arity=1, complexity=4),
    Token(expneg, "expneg", arity=1, complexity=4),
    Token(n3, "n3", arity=1, complexity=3),
    Token(n4, "n4", arity=1, complexity=3),
    Token(sigmoid, "sigmoid", arity=1, complexity=4),
    Token(harmonic, "harmonic", arity=1, complexity=4)
]


"""Define custom protected operators"""
def protected_div(x1, x2):
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)

def protected_exp(x1):
    with np.errstate(over='ignore'):
        return np.where(x1 < 100, np.exp(x1), 0.0)

def protected_log(x1):
    """Closure of log for non-positive arguments."""
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x1) > 0.001, np.log(np.abs(x1)), 0.)

def protected_sqrt(x1):
    """Closure of sqrt for negative arguments."""
    return np.sqrt(np.abs(x1))

def protected_inv(x1):
    """Closure of inverse for zero arguments."""
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(np.abs(x1) > 0.001, 1. / x1, 0.)

def protected_expneg(x1):
    with np.errstate(over='ignore'):
        return np.where(x1 > -100, np.exp(-x1), 0.0)

def protected_n2(x1):
    with np.errstate(over='ignore'):
        return np.where(np.abs(x1) < 1e6, np.square(x1), 0.0)

def protected_n3(x1):
    with np.errstate(over='ignore'):
        return np.where(np.abs(x1) < 1e6, np.power(x1, 3), 0.0)

def protected_n4(x1):
    with np.errstate(over='ignore'):
        return np.where(np.abs(x1) < 1e6, np.power(x1, 4), 0.0)

def protected_sigmoid(x1):
    return 1 / (1 + protected_expneg(x1))

# Annotate protected ops
protected_ops = [
    # Protected binary operators
    Token(protected_div, "div", arity=2, complexity=2),

    # Protected unary operators
    Token(protected_exp, "exp", arity=1, complexity=4),
    Token(protected_log, "log", arity=1, complexity=4),
    Token(protected_log, "logabs", arity=1, complexity=4), # Protected logabs is support, but redundant
    Token(protected_sqrt, "sqrt", arity=1, complexity=4),
    Token(protected_inv, "inv", arity=1, complexity=2),
    Token(protected_expneg, "expneg", arity=1, complexity=4),
    Token(protected_n2, "n2", arity=1, complexity=2),
    Token(protected_n3, "n3", arity=1, complexity=3),
    Token(protected_n4, "n4", arity=1, complexity=3),
    Token(protected_sigmoid, "sigmoid", arity=1, complexity=4)
]

# Add unprotected ops to function map
function_map = {
    op.name : op for op in unprotected_ops
    }

# Add protected ops to function map
function_map.update({
    "protected_{}".format(op.name) : op for op in protected_ops
    })

TERMINAL_TOKENS = set([op.name for op in function_map.values() if op.arity == 0])
UNARY_TOKENS    = set([op.name for op in function_map.values() if op.arity == 1])
BINARY_TOKENS   = set([op.name for op in function_map.values() if op.arity == 2])


def create_state_checkers(n_states, threshold_set):
    """
    Helper function to create StateChecker Tokens.

    Parameters
    ----------
    n_states : int
        Number of state variables.

    threshold_set : list or list of lists
        A list of constants [t1, t2, ..., tn] for constructing StateChecker (si < tj),
        or a list of lists of constants [[t11, t12, t1n], [t21, t22, ..., t2m], ...].
        In the latter case, the i-th list contains the thresholds for state variable si for
        constructing StateChecker (si < tij). The sizes of the threshold lists can be different.
    """
    tokens = []
    if isinstance(threshold_set[0], list):
        assert len(threshold_set) == n_states, \
            "If threshold_set is a list of lists, its length must equal n_states."
    else:
        threshold_set = [threshold_set]*n_states

    for i, thresholds in enumerate(threshold_set):
        assert all([U.is_float(t) for t in thresholds]), \
            "threshold_set must contain only real constant numbers."
        tokens.extend([StateChecker(i, t) for t in thresholds])

    return tokens


def create_tokens(n_input_var, function_set, protected, decision_tree_threshold_set=None):
    """
    Helper function to create Tokens.

    Parameters
    ----------
    n_input_var : int
        Number of input variable Tokens.

    function_set : list
        Names of registered Tokens, or floats that will create new Tokens.

    protected : bool
        Whether to use protected versions of registered Tokens.

    decision_tree_threshold_set : list or list of lists
        A list of constants [t1, t2, ..., tn] for constructing nodes (xi < tj) in decision trees,
        or a list of lists of constants [[t11, t12, t1n], [t21, t22, ..., t2m], ...].
        In the latter case, the i-th list contains the thresholds for input variable xi for constructing
        nodes (xi < tij) in decision trees. The sizes of the threshold lists can be different.
    """

    tokens = []

    # Create input variable Tokens
    for i in range(n_input_var):
        token = Token(name="x{}".format(i + 1), arity=0, complexity=1,
                      function=None, input_var=i)
        tokens.append(token)

    for op in function_set:

        # Registered Token
        if op in function_map:
            # Overwrite available protected operators
            if protected and not op.startswith("protected_"):
                protected_op = "protected_{}".format(op)
                if protected_op in function_map:
                    op = protected_op

            token = function_map[op]

        # Hard-coded floating-point constant
        elif U.is_float(op):
            token = HardCodedConstant(op)

        # Constant placeholder (to-be-optimized)
        elif op == "const":
            token = PlaceholderConstant()

        elif op == "poly":
            token = Polynomial()

        else:
            raise ValueError("Operation {} not recognized.".format(op))

        tokens.append(token)

    if decision_tree_threshold_set is not None and len(decision_tree_threshold_set) > 0:
        state_checkers = create_state_checkers(n_input_var, decision_tree_threshold_set)
        tokens.extend(state_checkers)

    return tokens

In [ ]:
import pandas as pd
import numpy as np
 # Assuming you have this

class BenchmarkDataset:
    def __init__(self, name, benchmark_source="benchmarks.csv", root=None, noise=0.0, seed=0):
        self.name = name
        self.seed = seed
        self.noise = noise

        # Load benchmark data (adjust if your setup is different)
        if root is None:
            root = "/content/benchmarks.csv"  # Replace with the correct path
        benchmark_path = os.path.join(root, benchmark_source)
        self.benchmark_df = pd.read_csv(benchmark_path)
        self.row = self.benchmark_df.loc[name]

        # Create symbolic expression and extract specifications
        self.numpy_expr = self.make_numpy_expr(self.row["expression"])
        self.specs = self.extract_dataset_specs(self.row["train_spec"])

        # Generate X and y, add noise
        self.X, self.y = self.build_dataset(self.specs)
        if self.noise > 0:
            scale = self.noise * np.std(self.y)
            self.y += np.random.normal(loc=0, scale=scale, size=self.y.shape)

    def extract_dataset_specs(self, specs):
        specs = ast.literal_eval(specs)
        if specs is not None:
            specs['distribution'] = list(list(specs.items())[0][1].items())[0][0]
            if specs['distribution'] == "E":
                lower = list(list(specs.items())[0][1].items())[0][1][0]
                upper = list(list(specs.items())[0][1].items())[0][1][1]
                distance = upper - lower
                specs['dataset_size'] = int(distance / list(list(specs.items())[0][1].items())[0][1][2]) + 1
            else:
                specs['dataset_size'] = list(list(specs.items())[0][1].items())[0][1][2]
        return specs

    def build_dataset(self, specs):
        rng = np.random.RandomState(self.seed)
        X = self.make_X(specs, specs['dataset_size'], rng)
        y = self.numpy_expr(X)
        return X, y

    def make_X(self, specs, size, rng):
        features = []
        for i in range(1, self.row["variables"] + 1):
            if "U" in specs["x{}".format(i)]:
                low, high = specs["x{}".format(i)]["U"]
                features.append(rng.uniform(low=low, high=high, size=size))
            else:
                # Handle other distributions if needed
                pass
        return np.column_stack(features)

    def make_numpy_expr(self, s):
        """This isn't pretty, but unlike sympy's lambdify, this ensures we use
        our protected functions. Otherwise, some expressions may have large
        error even if the functional form is correct due to the training set
        not using protected functions."""
        # Replace function names
        s = s.replace("ln(", "log(")
        s = s.replace("pi", "np.pi")
        s = s.replace("pow", "np.power")
        for k in function_map.keys():
            s = s.replace(k + '(', "function_map['{}'].function(".format(k))
        # Replace variable names
        for i in reversed(range(self.n_input_var)):
            old = "x{}".format(i+1)
            new = "x[:, {}]".format(i)
            s = s.replace(old, new)
        #Return numpy expression
        return lambda x : eval(s)



In [ ]:
class BenchmarkDataset(object):
    """
    Class used to generate (X, y) data from a named benchmark expression.

    Parameters
    ----------
    name : str
        Name of benchmark expression.

    benchmark_source : str, optional
        Filename of CSV describing benchmark expressions.

    root : str, optional
        Directory containing benchmark_source and function_sets.csv.

    noise : float, optional
        If not None, Gaussian noise is added to the y values with standard
        deviation = noise * RMS of the noiseless y training values.

    seed : int, optional
        Random number seed used to generate data. Checksum on name is added to
        seed.

    logdir : str, optional
        Directory where experiment logfiles are saved.

    backup : bool, optional
        Save generated dataset in logdir if logdir is provided.
    """

    def __init__(self, name, benchmark_source="benchmarks.csv", root=None, noise=0.0,
                 seed=0, logdir=None, backup=False):
        # Set class variables
        self.name = name
        self.seed = seed
        self.noise = noise if noise is not None else 0.0

        # Set random number generator used for sampling X values
        seed += zlib.adler32(name.encode("utf-8")) # Different seed for each name, otherwise two benchmarks with the same domain will always have the same X values
        self.rng = np.random.RandomState(seed)

        # Load benchmark data
        if root is None:
            root = resource_filename("dso.task", "regression")
        benchmark_path = os.path.join(root, benchmark_source)
        benchmark_df = pd.read_csv(benchmark_path, index_col=0, encoding="ISO-8859-1")
        row = benchmark_df.loc[name]
        self.n_input_var = row["variables"]

        # Create symbolic expression
        self.numpy_expr = self.make_numpy_expr(row["expression"])

        # Get dataset specifications
        self.train_spec = self.extract_dataset_specs(row["train_spec"])
        self.test_spec = self.extract_dataset_specs(row["test_spec"])
        if self.test_spec is None:
            self.test_spec = self.train_spec

        # Create X, y values - Train set
        self.X_train, self.y_train = self.build_dataset(self.train_spec)
        self.y_train_noiseless = self.y_train.copy()

        # Create X, y values - Test set
        self.X_test, self.y_test = self.build_dataset(self.test_spec)
        self.y_test_noiseless = self.y_test.copy()

        # Add Gaussian noise
        if self.noise > 0:
            y_rms = np.sqrt(np.mean(self.y_train**2))
            scale = self.noise * y_rms
            self.y_train += self.rng.normal(loc=0, scale=scale, size=self.y_train.shape)
            self.y_test += self.rng.normal(loc=0, scale=scale, size=self.y_test.shape)
        elif self.noise < 0:
            print('WARNING: Ignoring negative noise value: {}'.format(self.noise))

        # Load default function set
        function_set_path = os.path.join(root, "function_sets.csv")
        function_set_df = pd.read_csv(function_set_path, index_col=0)
        function_set_name = row["function_set"]
        self.function_set = function_set_df.loc[function_set_name].tolist()[0].strip().split(',')

        # Prepare status output
        output_message = '\n-- BUILDING DATASET START -----------\n'
        output_message += 'Generated data for benchmark   : {}\n'.format(name)
        output_message += 'Benchmark path                 : {}\n'.format(benchmark_path)
        output_message += 'Function set                   : {} --> {}\n'.format(function_set_name, self.function_set)
        output_message += 'Function set path              : {}\n'.format(function_set_path)
        test_spec_txt = row["test_spec"] if row["test_spec"] != "None" else "{} (Copy from train!)".format(row["test_spec"])
        output_message += 'Dataset specifications         : \n' \
                          + '        Train --> {}\n'.format(row["train_spec"]) \
                          + '        Test  --> {}\n'.format(test_spec_txt)
        random_choice_train = self.rng.randint(self.X_train.shape[0])
        random_sample_train = "[{}],[{}]".format(self.X_train[random_choice_train], self.y_train[random_choice_train])
        output_message += 'Built data set                 : \n' \
                          + '        Train --> X:{}, y:{}, Sample: {}\n'.format(self.X_train.shape, self.y_train.shape, random_sample_train)
        if row["test_spec"] is not None:
            random_choice_test = self.rng.randint(self.X_test.shape[0])
            random_sample_test = "[{}],[{}]".format(self.X_test[random_choice_test], self.y_test[random_choice_test])
            output_message += '        Test  --> X:{}, y:{}, Sample: {}\n'.format(self.X_test.shape, self.y_test.shape, random_sample_test)
        if backup and logdir is not None:
            output_message += self.save(logdir)
        output_message += '-- BUILDING DATASET END -------------\n'
        print(output_message)

    def extract_dataset_specs(self, specs):
        specs = ast.literal_eval(specs)
        if specs is not None:
            specs['distribution'] = list(list(specs.items())[0][1].items())[0][0]
            if specs['distribution'] == "E":
                lower = list(list(specs.items())[0][1].items())[0][1][0]
                upper = list(list(specs.items())[0][1].items())[0][1][1]
                distance = upper - lower
                specs['dataset_size'] = int(distance / list(list(specs.items())[0][1].items())[0][1][2]) + 1
            else:
                specs['dataset_size'] = list(list(specs.items())[0][1].items())[0][1][2]
        return specs

    def build_dataset(self, specs, max_iterations=1000, max_repeated_empty=100):
        """This function generates an (X,y) dataset by randomly sampling X
        values in a given range and calculating the corresponding y values.
        During generation it is checked that the generated datapoints are
        valid within the given range, removing nan and inf values. The
        generated dataset will be filled up to the desired dataset size or
        the function terminates with an error."""
        current_size = 0
        X_tmp = None
        count_repeated_empty = 0
        count_iterations = 0
        while(current_size < specs["dataset_size"]):
            if count_iterations > max_iterations:
                assert False, "Dataset creation taking too long. Got {} from {}".format(X_tmp.shape, specs)
            missing_value_count = specs["dataset_size"] - current_size
            # Get all X values
            X = self.make_X(specs, missing_value_count)
            assert X.ndim == 2, "Dataset X has wrong dimension: {} != 2".format(X.ndim)
            # Calculate y values
            y = self.numpy_expr(X)
            # Sanity check
            X, y = self.remove_invalid(X, y)
            if y.shape[0] == 0:
                count_repeated_empty += 1
                if count_repeated_empty > max_repeated_empty:
                    assert False, "Dataset cannot be created in the given range: {}".format(specs)
            # Put old and new data together if available
            if not X_tmp is None:
                X = np.append(X, X_tmp, axis=0)
                y = np.append(y, y_tmp, axis=0)
            current_size = X.shape[0]
            # Handle "E" distributions
            if X.shape[0] != specs["dataset_size"] and specs['distribution'] == "E":
                assert False, "Equal distant data points cannot be created in the given range: {}".format(specs)
            X_tmp = X
            y_tmp = y
            count_iterations +=1
        assert X.shape[0] == specs["dataset_size"]
        return X, y

    def remove_invalid(self, X, y, y_limit=100):
        """Removes nan, infs, and out of range datapoints from a dataset."""
        valid = np.logical_and(y > -y_limit, y < y_limit)
        y = y[valid]
        X = X[valid]
        assert X.shape[0] == y.shape[0]
        return X, y

    def make_X(self, spec, size):
        """Creates X values based on provided specification."""

        features = []
        for i in range(1, self.n_input_var + 1):

            # Hierarchy: "all" --> "x{}".format(i)
            input_var = "x{}".format(i)
            if "all" in spec:
                input_var = "all"
            elif input_var not in spec:
                input_var = "x1"

            if "U" in spec[input_var]:
                low, high, n = spec[input_var]["U"]
                feature = self.rng.uniform(low=low, high=high, size=size)
            elif "E" in spec[input_var]:
                start, stop, step = spec[input_var]["E"]
                if step > stop - start:
                    n = step
                else:
                    n = int((stop - start)/step) + 1
                feature = np.linspace(start=start, stop=stop, num=n, endpoint=True)
            else:
                raise ValueError("Did not recognize specification for {}: {}.".format(input_var, spec[input_var]))
            features.append(feature)

        # Do multivariable combinations
        if "E" in spec[input_var] and self.n_input_var > 1:
            X = np.array(list(itertools.product(*features)))
        else:
            X = np.column_stack(features)

        return X

    def make_numpy_expr(self, s):
        """This isn't pretty, but unlike sympy's lambdify, this ensures we use
        our protected functions. Otherwise, some expressions may have large
        error even if the functional form is correct due to the training set
        not using protected functions."""
        # Replace function names
        s = s.replace("ln(", "log(")
        s = s.replace("pi", "np.pi")
        s = s.replace("pow", "np.power")
        for k in function_map.keys():
            s = s.replace(k + '(', "function_map['{}'].function(".format(k))
        # Replace variable names
        for i in reversed(range(self.n_input_var)):
            old = "x{}".format(i+1)
            new = "x[:, {}]".format(i)
            s = s.replace(old, new)
        #Return numpy expression
        return lambda x : eval(s)

    def save(self, logdir='./'):
        """Saves the dataset to a specified location."""
        save_path = os.path.join(logdir,'data_{}_n{:.2f}_s{}.csv'.format(
                self.name, self.noise, self.seed))
        try:
            os.makedirs(logdir, exist_ok=True)
            np.savetxt(
                save_path,
                np.concatenate(
                    (
                        np.hstack((self.X_train, self.y_train[..., np.newaxis])),
                        np.hstack((self.X_test, self.y_test[..., np.newaxis]))
                    ), axis=0),
                delimiter=',', fmt='%1.5f'
            )
            return 'Saved dataset to               : {}\n'.format(save_path)
        except:
            import sys
            e = sys.exc_info()[0]
            print("WARNING: Could not save dataset: {}".format(e))

    def plot(self, logdir='./'):
        """Plot Dataset with underlying ground truth."""
        if self.X_train.shape[1] == 1:
            from matplotlib import pyplot as plt
            save_path = os.path.join(logdir,'plot_{}_n{:.2f}_s{}.png'.format(
                    self.name, self.noise, self.seed))

            # Draw ground truth expression
            bounds = list(list(self.train_spec.values())[0].values())[0][:2]
            x = np.linspace(bounds[0], bounds[1], endpoint=True, num=100)
            y = self.numpy_expr(x[:, None])
            plt.plot(x, y, color='red', linestyle='dashed')
            # Draw the actual points
            plt.scatter(self.X_train, self.y_train)
            # Add a title
            plt.title(
                "{} N:{} S:{}".format(
                    self.name, self.noise, self.seed),
                fontsize=7)
            try:
                os.makedirs(logdir, exist_ok=True)
                plt.savefig(save_path)
                print('Saved plot to                  : {}'.format(save_path))
            except:
                import sys
                e = sys.exc_info()[0]
                print("WARNING: Could not plot dataset: {}".format(e))
            plt.close()
        else:
            print("WARNING: Plotting only supported for 2D datasets.")


@click.command()
@click.argument("benchmark_source", default="benchmarks.csv")
@click.option('--plot', is_flag=True)
@click.option('--save_csv', is_flag=True)
@click.option('--sweep', is_flag=True)
def main(benchmark_source, plot, save_csv, sweep):
    """Plots all benchmark expressions."""

    regression_path = resource_filename("dso.task", "regression/")
    benchmark_path = os.path.join(regression_path, benchmark_source)
    save_dir = os.path.join(regression_path, 'log')
    df = pd.read_csv(benchmark_path, encoding="ISO-8859-1")
    names = df["name"].to_list()
    for name in names:

        if not name.startswith("Nguyen") and not name.startswith("Constant") and not name.startswith("Custom"):
            continue

        datasets = []

        # Noiseless
        d = BenchmarkDataset(
            name=name,
            benchmark_source=benchmark_source)
        datasets.append(d)

        # Generate all combinations of noise levels and dataset size multipliers
        if sweep and name.startswith("Nguyen"):
            noises = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
            for noise in noises:
                d = BenchmarkDataset(
                    name=name,
                    benchmark_source=benchmark_source,
                    noise=noise,
                    backup=save_csv,
                    logdir=save_dir)
                datasets.append(d)

        # Plot and/or save datasets
        for dataset in datasets:
            if plot and dataset.X_train.shape[1] == 1:
                dataset.plot(save_dir)

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import pandas as pd




class RegressionTask(HierarchicalTask):

  task = RegressionTask(function_set=["add", "sub", "mul"],
                      dataset="/content/Nguyen-1.csv",
                      metric="neg_nrmse")

  def __init__(self, function_set, dataset, metric="inv_nrmse",
                 metric_params=(1.0,), extra_metric_test=None,
                 extra_metric_test_params=(), reward_noise=0.0,
                 reward_noise_type="r", threshold=1e-12,
                 normalize_variance=False, protected=False,
                 decision_tree_threshold_set=None,
                 poly_optimizer_params=None):
        """
        Parameters
        ----------
        function_set : list or None
            List of allowable functions. If None, uses function_set according to
            benchmark dataset.

        dataset : dict, str, or tuple
            If dict: .dataset.BenchmarkDataset kwargs.
            If str ending with .csv: filename of dataset.
            If other str: name of benchmark dataset.
            If tuple: (X, y) data

        metric : str
            Name of reward function metric to use.

        metric_params : list
            List of metric-specific parameters.

        extra_metric_test : str
            Name of extra function metric to use for testing.

        extra_metric_test_params : list
            List of metric-specific parameters for extra test metric.

        reward_noise : float
            Noise level to use when computing reward.

        reward_noise_type : "y_hat" or "r"
            "y_hat" : N(0, reward_noise * y_rms_train) is added to y_hat values.
            "r" : N(0, reward_noise) is added to r.

        threshold : float
            Threshold of NMSE on noiseless data used to determine success.

        normalize_variance : bool
            If True and reward_noise_type=="r", reward is multiplied by
            1 / sqrt(1 + 12*reward_noise**2) (We assume r is U[0,1]).

        protected : bool
            Whether to use protected functions.

        decision_tree_threshold_set : list
            A set of constants {tj} for constructing nodes (xi < tj) in decision trees.

        poly_optimizer_params : dict
            Parameters for PolyOptimizer if poly token is in the library.
        """

        super(HierarchicalTask).__init__()

        """
        Configure (X, y) train/test data. There are four supported use cases:
        (1) named benchmark, (2) benchmark config, (3) filename, and (4) direct
        (X, y) data.
        """
        self.X_test = self.y_test = self.y_test_noiseless = None

        # Case 1: Named benchmark dataset (shortcut for Case 2)
        if isinstance(dataset, str) and not dataset.endswith(".csv"):
            dataset = {"name" : dataset}

        # Case 2: Benchmark dataset config
        if isinstance(dataset, dict):
            benchmark = BenchmarkDataset(**dataset)
            self.X_train = benchmark.X_train
            self.y_train = benchmark.y_train
            self.X_test = benchmark.X_test
            self.y_test = benchmark.y_test
            self.y_test_noiseless = benchmark.y_test_noiseless
            self.name = benchmark.name

            # For benchmarks, always use the benchmark function_set.
            # Issue a warning if the user tried to supply a different one.
            if function_set is not None and function_set != benchmark.function_set:
                print("WARNING: function_set provided when running benchmark "
                      "problem. The provided function_set will be ignored; the "
                      "benchmark function_set will be used instead.\nProvided "
                      "function_set:\n  {}\nBenchmark function_set:\n  {}."
                      .format(function_set, benchmark.function_set))
            function_set = benchmark.function_set

        # Case 3: Dataset filename
        elif isinstance(dataset, str) and dataset.endswith("csv"):
            df = pd.read_csv(dataset, header=None) # Assuming data file does not have header rows
            self.X_train = df.values[:, :-1]
            self.y_train = df.values[:, -1]
            self.name = dataset.replace("/", "_")[:-4]

        # Case 4: sklearn-like (X, y) data
        elif isinstance(dataset, tuple):
            self.X_train = dataset[0]
            self.y_train = dataset[1]
            self.name = "regression"

        # If not specified, set test data equal to the training data
        if self.X_test is None:
            self.X_test = self.X_train
            self.y_test = self.y_train
            self.y_test_noiseless = self.y_test

        # Save time by only computing data variances once
        self.var_y_test = np.var(self.y_test)
        self.var_y_test_noiseless = np.var(self.y_test_noiseless)

        """
        Configure train/test reward metrics.
        """
        self.threshold = threshold
        self.metric, self.invalid_reward, self.max_reward = make_regression_metric(metric, self.y_train, *metric_params)
        self.extra_metric_test = extra_metric_test
        if extra_metric_test is not None:
            self.metric_test, _, _ = make_regression_metric(extra_metric_test, self.y_test, *extra_metric_test_params)
        else:
            self.metric_test = None

        """
        Configure reward noise.
        """
        self.reward_noise = reward_noise
        self.reward_noise_type = reward_noise_type
        self.normalize_variance = normalize_variance
        assert reward_noise >= 0.0, "Reward noise must be non-negative."
        if reward_noise > 0:
            assert reward_noise_type in ["y_hat", "r"], "Reward noise type not recognized."
            self.rng = np.random.RandomState(0)
            y_rms_train = np.sqrt(np.mean(self.y_train ** 2))
            if reward_noise_type == "y_hat":
                self.scale = reward_noise * y_rms_train
            elif reward_noise_type == "r":
                self.scale = reward_noise
        else:
            self.rng = None
            self.scale = None

        # Set the Library
        tokens = create_tokens(n_input_var=self.X_train.shape[1],
                               function_set=function_set,
                               protected=protected,
                               decision_tree_threshold_set=decision_tree_threshold_set)
        self.library = Library(tokens)

        # Set stochastic flag
        self.stochastic = reward_noise > 0.0

        # Set neg_nrmse as the metric for const optimization
        self.const_opt_metric, _, _ = make_regression_metric("neg_nrmse", self.y_train)

        # Function to optimize polynomial tokens
        if "poly" in self.library.names:
            if poly_optimizer_params is None:
                poly_optimizer_params = {
                        "degree": 3,
                        "coef_tol": 1e-6,
                        "regressor": "dso_least_squares",
                        "regressor_params": {}
                    }

            self.poly_optimizer = PolyOptimizer(**poly_optimizer_params)

  def reward_function(self, p, optimizing=False):
        # fit a polynomial if p contains a 'poly' token
        if p.poly_pos is not None:
            assert len(p.const_pos) == 0, "A program cannot contain 'poly' and 'const' tokens at the same time"
            poly_data_y = make_poly_data(p.traversal, self.X_train, self.y_train)
            if poly_data_y is None:  # invalid function evaluations (nan or inf) appeared in make_poly_data
                p.traversal[p.poly_pos] = Polynomial([(0,)*self.X_train.shape[1]], np.ones(1))
            else:
                p.traversal[p.poly_pos] = self.poly_optimizer.fit(self.X_train, poly_data_y)

        # Compute estimated values
        y_hat = p.execute(self.X_train)

        # For invalid expressions, return invalid_reward
        if p.invalid:
            return -1.0 if optimizing else self.invalid_reward

        # Observation noise
        # For reward_noise_type == "y_hat", success must always be checked to
        # ensure success cases aren't overlooked due to noise. If successful,
        # return max_reward.
        if self.reward_noise and self.reward_noise_type == "y_hat":
            if p.evaluate.get("success"):
                return self.max_reward
            y_hat += self.rng.normal(loc=0, scale=self.scale, size=y_hat.shape)

        # Compute and return neg_nrmse for constant optimization
        if optimizing:
            return self.const_opt_metric(self.y_train, y_hat)

        # Compute metric
        r = self.metric(self.y_train, y_hat)

        # Direct reward noise
        # For reward_noise_type == "r", success can for ~max_reward metrics be
        # confirmed before adding noise. If successful, must return np.inf to
        # avoid overlooking success cases.
        if self.reward_noise and self.reward_noise_type == "r":
            if r >= self.max_reward - 1e-5 and p.evaluate.get("success"):
                return np.inf
            r += self.rng.normal(loc=0, scale=self.scale)
            if self.normalize_variance:
                r /= np.sqrt(1 + 12 * self.scale ** 2)

        return r

  def evaluate(self, p):

        # Compute predictions on test data
        y_hat = p.execute(self.X_test)
        if p.invalid:
            nmse_test = None
            nmse_test_noiseless = None
            success = False

        else:
            # NMSE on test data (used to report final error)
            nmse_test = np.mean((self.y_test - y_hat) ** 2) / self.var_y_test

            # NMSE on noiseless test data (used to determine recovery)
            nmse_test_noiseless = np.mean((self.y_test_noiseless - y_hat) ** 2) / self.var_y_test_noiseless

            # Success is defined by NMSE on noiseless test data below a threshold
            success = nmse_test_noiseless < self.threshold

        info = {
            "nmse_test" : nmse_test,
            "nmse_test_noiseless" : nmse_test_noiseless,
            "success" : success
        }

        if self.metric_test is not None:
            if p.invalid:
                m_test = None
                m_test_noiseless = None
            else:
                m_test = self.metric_test(self.y_test, y_hat)
                m_test_noiseless = self.metric_test(self.y_test_noiseless, y_hat)

            info.update({
                self.extra_metric_test : m_test,
                self.extra_metric_test + '_noiseless' : m_test_noiseless
            })

        return info


def make_regression_metric(name, y_train, *args):
    """
    Factory function for a regression metric. This includes a closures for
    metric parameters and the variance of the training data.

    Parameters
    ----------

    name : str
        Name of metric. See all_metrics for supported metrics.

    args : args
        Metric-specific parameters

    Returns
    -------

    metric : function
        Regression metric mapping true and estimated values to a scalar.

    invalid_reward: float or None
        Reward value to use for invalid expression. If None, the training
        algorithm must handle it, e.g. by rejecting the sample.

    max_reward: float
        Maximum possible reward under this metric.
    """

    var_y = np.var(y_train)

    all_metrics = {

        # Negative mean squared error
        # Range: [-inf, 0]
        # Value = -var(y) when y_hat == mean(y)
        "neg_mse" :     (lambda y, y_hat : -np.mean((y - y_hat)**2),
                        0),

        # Negative root mean squared error
        # Range: [-inf, 0]
        # Value = -sqrt(var(y)) when y_hat == mean(y)
        "neg_rmse" :     (lambda y, y_hat : -np.sqrt(np.mean((y - y_hat)**2)),
                        0),

        # Negative normalized mean squared error
        # Range: [-inf, 0]
        # Value = -1 when y_hat == mean(y)
        "neg_nmse" :    (lambda y, y_hat : -np.mean((y - y_hat)**2)/var_y,
                        0),

        # Negative normalized root mean squared error
        # Range: [-inf, 0]
        # Value = -1 when y_hat == mean(y)
        "neg_nrmse" :   (lambda y, y_hat : -np.sqrt(np.mean((y - y_hat)**2)/var_y),
                        0),

        # (Protected) negative log mean squared error
        # Range: [-inf, 0]
        # Value = -log(1 + var(y)) when y_hat == mean(y)
        "neglog_mse" : (lambda y, y_hat : -np.log(1 + np.mean((y - y_hat)**2)),
                        0),

        # (Protected) inverse mean squared error
        # Range: [0, 1]
        # Value = 1/(1 + args[0]*var(y)) when y_hat == mean(y)
        "inv_mse" : (lambda y, y_hat : 1/(1 + args[0]*np.mean((y - y_hat)**2)),
                        1),

        # (Protected) inverse normalized mean squared error
        # Range: [0, 1]
        # Value = 1/(1 + args[0]) when y_hat == mean(y)
        "inv_nmse" :    (lambda y, y_hat : 1/(1 + args[0]*np.mean((y - y_hat)**2)/var_y),
                        1),

        # (Protected) inverse normalized root mean squared error
        # Range: [0, 1]
        # Value = 1/(1 + args[0]) when y_hat == mean(y)
        "inv_nrmse" :    (lambda y, y_hat : 1/(1 + args[0]*np.sqrt(np.mean((y - y_hat)**2)/var_y)),
                        1),

        # Fraction of predicted points within p0*abs(y) + p1 band of the true value
        # Range: [0, 1]
        "fraction" :    (lambda y, y_hat : np.mean(abs(y - y_hat) < args[0]*abs(y) + args[1]),
                        2),

        # Pearson correlation coefficient
        # Range: [0, 1]
        "pearson" :     (lambda y, y_hat : scipy.stats.pearsonr(y, y_hat)[0],
                        0),

        # Spearman correlation coefficient
        # Range: [0, 1]
        "spearman" :    (lambda y, y_hat : scipy.stats.spearmanr(y, y_hat)[0],
                        0)
    }

    assert name in all_metrics, "Unrecognized reward function name."
    assert len(args) == all_metrics[name][1], "For {}, expected {} reward function parameters; received {}.".format(name,all_metrics[name][1], len(args))
    metric = all_metrics[name][0]

    # For negative MSE-based rewards, invalid reward is the value of the reward function when y_hat = mean(y)
    # For inverse MSE-based rewards, invalid reward is 0.0
    # For non-MSE-based rewards, invalid reward is the minimum value of the reward function's range
    all_invalid_rewards = {
        "neg_mse" : -var_y,
        "neg_rmse" : -np.sqrt(var_y),
        "neg_nmse" : -1.0,
        "neg_nrmse" : -1.0,
        "neglog_mse" : -np.log(1 + var_y),
        "inv_mse" : 0.0, #1/(1 + args[0]*var_y),
        "inv_nmse" : 0.0, #1/(1 + args[0]),
        "inv_nrmse" : 0.0, #1/(1 + args[0]),
        "fraction" : 0.0,
        "pearson" : 0.0,
        "spearman" : 0.0
    }
    invalid_reward = all_invalid_rewards[name]

    all_max_rewards = {
        "neg_mse" : 0.0,
        "neg_rmse" : 0.0,
        "neg_nmse" : 0.0,
        "neg_nrmse" : 0.0,
        "neglog_mse" : 0.0,
        "inv_mse" : 1.0,
        "inv_nmse" : 1.0,
        "inv_nrmse" : 1.0,
        "fraction" : 1.0,
        "pearson" : 1.0,
        "spearman" : 1.0
    }
    max_reward = all_max_rewards[name]

    return metric, invalid_reward, max_reward

AssertionError: For neg_nrmse, expected 0 reward function parameters; received 1.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from abc import ABC, abstractmethod
import tensorflow as tf
from tensorflow.keras.layers import RNN, LSTM, Dropout, Dense
from tensorflow.compat.v1.nn.rnn_cell import BasicRNNCell

from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import os
# This will give you the current working directory of the notebook
current_directory = os.getcwd()

config_file = '/content/config_common.json'

def get_base_config(task, language_prior):
    config_path = os.path.join(current_directory, config_file)
    with open(config_path, 'r', encoding='utf-8') as file:
        base_config = json.load(file)
    return base_config
# Define your Task and LanguageModel classes here

# Define your DeepSymbolicOptimizer class here

# Define or import additional necessary classes or methods
class Task(ABC):
    """
    Object specifying a symbolic search task.

    Attributes
    ----------
    library : Library
        Library of Tokens.

    stochastic : bool
        Whether the reward function of the task is stochastic.

    task_type : str
        Task type: regression, control, or binding.

    name : str
        Unique name for instance of this task.
    """

    task_type = None

    @abstractmethod
    def reward_function(self, program, optimizing=False):
        """
        The reward function for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to compute reward of.

        optimizing : bool

            Whether the reward is computed for PlaceholderConstant optimization.

        Returns
        -------
        reward : float

            Fitness/reward of the program.
        """
        raise NotImplementedError

    @abstractmethod
    def evaluate(self, program):
        """
        The evaluation metric for this task.

        Parameters
        ----------
        program : dso.program.Program

            The Program to evaluate.

        Returns
        -------

        info : dict

            Dictionary of evaluation metrics. Special key "success" is used to
            trigger early stopping.
        """
        raise NotImplementedError

    @abstractmethod
    def get_next_obs(self, actions, obs, already_finished):
        """
        Produce the next observation and prior from the current observation and
        list of actions so far. Observations must be 1-D np.float32 vectors.

        Parameters
        ----------

        actions : np.ndarray (dtype=np.int32)
            Actions selected so far, shape (batch_size, current_length)

        obs : np.ndarray (dtype=np.float32)
            Previous observation, shape (batch_size, OBS_DIM).

        already_finished : np.ndarray (dtype=bool)
            Whether the object has *already* been completed.

        Returns
        -------

        next_obs : np.ndarray (dtype=np.float32)
            The next observation, shape (batch_size, OBS_DIM).

        prior : np.ndarray (dtype=np.float32)
            Prior for selecting the next token, shape (batch_size,
            self.library.L).

        finished : np.ndarray (dtype=bool)
            Whether the object has *ever* been completed.
        """
        pass

    @abstractmethod
    def reset_task(self):
        """
        Create the starting observation.

        Returns
        -------

        obs : np.ndarray (dtype=np.float32)
            Starting observation, shape (batch_size, OBS_DIM).
        """
        pass


class HierarchicalTask(Task):
    """
    A Task in which the search space is a binary tree. Observations include
    the previous action, the parent, the sibling, and/or the number of dangling
    (unselected) nodes.
    """

    OBS_DIM = 4 # action, parent, sibling, dangling

    def __init__(self):
        super(Task).__init__()

    def get_next_obs(self, actions, obs, already_finished):

        dangling = obs[:, 3] # Shape of obs: (?, 4)
        action = actions[:, -1] # Current action
        lib = self.library

        # Compute parents and siblings
        parent, sibling = parents_siblings(actions,
                                           arities=lib.arities,
                                           parent_adjust=lib.parent_adjust,
                                           empty_parent=lib.EMPTY_PARENT,
                                           empty_sibling=lib.EMPTY_SIBLING)

        # Compute dangling
        dangling += lib.arities[action] - 1

        # Compute finished
        just_finished = (dangling == 0) # Trees that completed _this_ time step
        # [batch_size]
        finished = np.logical_or(just_finished,
                                 already_finished)

        # Compute priors
        prior = self.prior(actions, parent, sibling, dangling, finished) # (?, n_choices)

        # Combine observation dimensions
        next_obs = np.stack([action, parent, sibling, dangling], axis=1) # (?, 4)
        next_obs = next_obs.astype(np.float32)

        return next_obs, prior, finished

    def reset_task(self, prior):
        """
        Returns the initial observation: empty action, parent, and sibling, and
        dangling is 1.
        """

        self.prior = prior

        # Order of observations: action, parent, sibling, dangling
        initial_obs = np.array([self.library.EMPTY_ACTION,
                                self.library.EMPTY_PARENT,
                                self.library.EMPTY_SIBLING,
                                1],
                               dtype=np.float32)
        return initial_obs


class SequentialTask(Task):
    """
    A Task in which the search space is a (possibly variable-length) sequence.
    The observation is simply the previous action.
    """

    pass


def make_task(task_type, **config_task):
    """
    Factory function for Task object.

    Parameters
    ----------

    task_type : str
        Type of task:
        "regression" : Symbolic regression task.
        "control" : Episodic reinforcement learning task.
        "binding": AbAg binding affinity optimization task.

    config_task : kwargs
        Task-specific arguments. See specifications of task_dict.

    Returns
    -------

    task : Task
        Task object.
    """

    # Lazy import of task factory functions
    if task_type == 'binding':
        from dso.task.binding.binding import BindingTask
        task_class = BindingTask
    elif task_type == "regression":
        from dso.task.regression.regression import RegressionTask
        task_class = RegressionTask
    elif task_type == "control":
        from dso.task.control.control import ControlTask
        task_class = ControlTask
    else:
        # Custom task import
        task_class = import_custom_source(task_type)
        assert issubclass(task_class, Task), \
            "Custom task {} must subclass dso.task.Task.".format(task_class)

    task = task_class(**config_task)
    return task


def set_task(config_task):
    """Helper function to make set the Program class Task and execute function
    from task config."""

    # Use of protected functions is the same for all tasks, so it's handled separately
    protected = config_task["protected"] if "protected" in config_task else False

    Program.set_execute(protected)
    task = make_task(**config_task)
    Program.set_task(task)

class LanguageModel(object):
    def __init__(self, vocabulary_size, embedding_size, num_layers, num_hidden, mode='train'):
        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.num_hidden = num_hidden

        self.inputs = tf.placeholder(tf.int32, [None, None], name="inputs")  # Batch size x sequence length
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")

        with tf.variable_scope("embedding"):
            self.embedding = tf.get_variable("embedding", [vocabulary_size, embedding_size])
            self.inputs_embedded = tf.nn.embedding_lookup(self.embedding, self.inputs)

        with tf.variable_scope("rnn"):
            cells = [tf.nn.rnn_cell.BasicRNNCell(num_hidden) for _ in range(num_layers)]
            cells = [tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in cells]
            self.cell = tf.nn.rnn_cell.MultiRNNCell(cells)
            self.initial_state = self.cell.zero_state(tf.shape(self.inputs)[0], tf.float32)
            self.outputs, self.final_state = tf.nn.dynamic_rnn(self.cell, self.inputs_embedded,
                                                               initial_state=self.initial_state)

        with tf.variable_scope("output"):
            self.logits = tf.layers.dense(self.outputs, vocabulary_size)

        if mode == 'train':
            self.targets = tf.placeholder(tf.int32, [None, None], name="targets")
            self.loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.targets, logits=self.logits)
            self.cost = tf.reduce_mean(self.loss)

class DeepSymbolicOptimizer():
    """
    Deep symbolic optimization model. Includes model hyperparameters and
    training configuration.

    Parameters
    ----------
    config : dict or str
        Config dictionary or path to JSON.

    Attributes
    ----------
    config : dict
        Configuration parameters for training.

    Methods
    -------
    train
        Builds and trains the model according to config.
    """

    def __init__(self, config=None):
        self.set_config(config)
        self.sess = None

    def setup(self):

        # Clear the cache and reset the compute graph
        Program.clear_cache()
        tf.reset_default_graph()

        # Generate objects needed for training and set seeds
        self.pool = self.make_pool_and_set_task()
        self.set_seeds() # Must be called _after_ resetting graph and _after_ setting task

        # Limit TF to single thread to prevent "resource not available" errors in parallelized runs
        session_config = tf.ConfigProto(intra_op_parallelism_threads=1,
                                        inter_op_parallelism_threads=1)
        self.sess = tf.Session(config=session_config)

        # Setup logdirs and output files
        self.output_file = self.make_output_file()
        self.save_config()

        # Prepare training parameters
        self.prior = self.make_prior()
        self.state_manager = self.make_state_manager()
        self.policy = self.make_policy()
        self.policy_optimizer = self.make_policy_optimizer()
        self.gp_controller = self.make_gp_controller()
        self.logger = self.make_logger()
        self.trainer = self.make_trainer()
        self.checkpoint = self.make_checkpoint()

    def train_one_step(self, override=None):
        """
        Train one iteration.
        """

        # Setup the model
        if self.sess is None:
            self.setup()

        # Run one step
        assert not self.trainer.done, "Training has already completed!"
        self.trainer.run_one_step(override)

        # Maybe save next checkpoint
        self.checkpoint.update()

        # If complete, return summary
        if self.trainer.done:
            return self.finish()

    def train(self):
        """
        Train the model until completion.
        """

        # Setup the model
        self.setup()

        # Train the model until done
        while not self.trainer.done:
            result = self.train_one_step()

        return result

    def finish(self):
        """
        After training completes, finish up and return summary dict.
        """

        # Return statistics of best Program
        p = self.trainer.p_r_best
        result = {"seed" : self.config_experiment["seed"]} # Seed listed first
        result.update({"r" : p.r})
        result.update(p.evaluate)
        result.update({
            "expression" : repr(p.sympy_expr),
            "traversal" : repr(p),
            "program" : p
        })

        # Save all results available only after all iterations are finished. Also return metrics to be added to the summary file
        results_add = self.logger.save_results(self.pool, self.trainer.nevals)
        result.update(results_add)

        # Close the pool
        if self.pool is not None:
            self.pool.close()

        return result

    def set_config(self, config):
        config = load_config(config)

        self.config = defaultdict(dict, config)
        self.config_task = self.config["task"]
        self.config_prior = self.config["prior"]
        self.config_logger = self.config["logging"]
        self.config_training = self.config["training"]
        self.config_state_manager = self.config["state_manager"]
        self.config_policy = self.config["policy"]
        self.config_policy_optimizer = self.config["policy_optimizer"]
        self.config_gp_meld = self.config["gp_meld"]
        self.config_experiment = self.config["experiment"]
        self.config_checkpoint = self.config["checkpoint"]

    def save_config(self):
        # Save the config file
        if self.output_file is not None:
            path = os.path.join(self.config_experiment["save_path"],
                                "config.json")
            # With run.py, config.json may already exist. To avoid race
            # conditions, only record the starting seed. Use a backup seed
            # in case this worker's seed differs.
            backup_seed = self.config_experiment["seed"]
            if not os.path.exists(path):
                if "starting_seed" in self.config_experiment:
                    self.config_experiment["seed"] = self.config_experiment["starting_seed"]
                    del self.config_experiment["starting_seed"]
                with open(path, 'w') as f:
                    json.dump(self.config, f, indent=3)
            self.config_experiment["seed"] = backup_seed

    def set_seeds(self):
        """
        Set the tensorflow, numpy, and random module seeds based on the seed
        specified in config. If there is no seed or it is None, a time-based
        seed is used instead and is written to config.
        """

        seed = self.config_experiment.get("seed")

        # Default uses current time in milliseconds, modulo 1e9
        if seed is None:
            seed = round(time() * 1000) % int(1e9)
            self.config_experiment["seed"] = seed

        # Shift the seed based on task name
        # This ensures a specified seed doesn't have similarities across different task names
        task_name = Program.task.name
        shifted_seed = seed + zlib.adler32(task_name.encode("utf-8"))

        # Set the seeds using the shifted seed
        tf.set_random_seed(shifted_seed)
        np.random.seed(shifted_seed)
        random.seed(shifted_seed)

    def make_prior(self):
        prior = make_prior(Program.library, self.config_prior)
        return prior

    def make_state_manager(self):
        state_manager = make_state_manager(self.config_state_manager)
        return state_manager

    def make_trainer(self):
        trainer = Trainer(self.sess,
                          self.policy,
                          self.policy_optimizer,
                          self.gp_controller,
                          self.logger,
                          self.pool,
                          **self.config_training)
        return trainer

    def make_logger(self):
        logger = StatsLogger(self.sess,
                             self.output_file,
                             **self.config_logger)
        return logger

    def make_checkpoint(self):
        checkpoint = Checkpoint(self,
                                **self.config_checkpoint)
        return checkpoint

    def make_policy_optimizer(self):
        policy_optimizer = make_policy_optimizer(self.sess,
                                                 self.policy,
                                                 **self.config_policy_optimizer)
        return policy_optimizer

    def make_policy(self):
        policy = make_policy(self.sess,
                             self.prior,
                             self.state_manager,
                             **self.config_policy)
        return policy

    def make_gp_controller(self):
        if self.config_gp_meld.pop("run_gp_meld", False):
            from dso.gp.gp_controller import GPController
            gp_controller = GPController(self.prior,
                                         self.config_prior,
                                         **self.config_gp_meld)
        else:
            gp_controller = None
        return gp_controller

    def make_pool_and_set_task(self):
        # Create the pool and set the Task for each worker

        # Set complexity and const optimizer here so pool can access them
        # Set the complexity function
        complexity = self.config_training["complexity"]
        Program.set_complexity(complexity)

        # Set the constant optimizer
        const_optimizer = self.config_training["const_optimizer"]
        const_params = self.config_training["const_params"]
        const_params = const_params if const_params is not None else {}
        Program.set_const_optimizer(const_optimizer, **const_params)

        pool = None
        n_cores_batch = self.config_training.get("n_cores_batch")
        if n_cores_batch is not None:
            if n_cores_batch == -1:
                n_cores_batch = cpu_count()
            if n_cores_batch > 1:
                pool = Pool(n_cores_batch,
                            initializer=set_task,
                            initargs=(self.config_task,))

        # Set the Task for the parent process
        set_task(self.config_task)

        return pool

    def make_output_file(self):
        """Generates an output filename"""

        # If logdir is not provided (e.g. for pytest), results are not saved
        if self.config_experiment.get("logdir") is None:
            self.save_path = None
            print("WARNING: logdir not provided. Results will not be saved to file.")
            return None

        # When using run.py, timestamp is already generated
        timestamp = self.config_experiment.get("timestamp")
        if timestamp is None:
            timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
            self.config_experiment["timestamp"] = timestamp

        # Generate save path
        task_name = Program.task.name
        if self.config_experiment["exp_name"] is None:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                '_'.join([task_name, timestamp]))
        else:
            save_path = os.path.join(
                self.config_experiment["logdir"],
                self.config_experiment["exp_name"])

        self.config_experiment["task_name"] = task_name
        self.config_experiment["save_path"] = save_path
        os.makedirs(save_path, exist_ok=True)

        seed = self.config_experiment["seed"]
        output_file = os.path.join(save_path,
                                   "dso_{}_{}.csv".format(task_name, seed))

        self.save_path = save_path

        return output_file

    def save(self, save_path=None):
        self.checkpoint.save(save_path)

    def load(self, load_path):
        self.checkpoint.load(load_path)


class RegressionTask(Task):
    def __init__(self, model, features, labels):
        super().__init__()
        self.model = model
        self.features = features
        self.labels = labels

    def reward_function(self, program):
        # Define how the reward is calculated using the model's predictions
        predictions = self.model.predict(self.features)
        return -np.mean(np.square(predictions - self.labels))

    def evaluate(self, program):
        # Define evaluation based on the task's specific metrics
        predictions = self.model.predict(self.features)
        return {'mse': np.mean(np.square(predictions - self.labels))}

    def get_next_obs(self, actions, obs, already_finished):
        # Logic for getting next observations if applicable
        return obs  # Placeholder, implement as needed

    def reset_task(self):
        # Reset the task environment to an initial state
        return np.zeros_like(self.features)  # Placeholder, implement as needed

def main():
    tf.compat.v1.reset_default_graph()  # Reset the graph to clear old variables
    # The rest of your main function code...

    # Load dataset
    train_features = pd.read_csv("/content/train_feature.csv").values
    train_labels = pd.read_csv("/content/train_label.csv").values

    # Configuration for RNN model
    rnn_config = {
        "vocabulary_size": 10000,
        "embedding_size": 300,
        "num_layers": 2,
        "num_hidden": 128,
        "mode": 'train'
    }

    # Instantiate RNN model
    rnn_model = LanguageModel(**rnn_config)

    # Create Task instance
    task = RegressionTask(rnn_model, train_features, train_labels)

    # Configuration for optimizer
    optimizer_config = {
        # Include any specific configurations for the optimizer
    }

    # Instantiate and setup optimizer
    optimizer = DeepSymbolicOptimizer(config=optimizer_config)
    optimizer.setup()

    # Train model
    result = optimizer.train()

    # Print final training outcome
    print("Training completed with result:", result)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:428: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1705: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
<ipython-input-80-9374135f8bfe>:287: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.logits = tf.layers.dense(self.outputs, vocabulary_size)


ModuleNotFoundError: No module named 'dso'

In [ ]:

# Set up your task configuration
task_config = config.default_task_config()
task_config["task"] = "regression"
task_config["dataset_train"] = (train_features.values, train_labels.values)
task_config["dataset_validate"] = (validation_features.values, validation_labels.values)

# Create a Task instance
task = Task(task_config)

# Set up your controller configuration
controller_config = config.default_controller_config()
controller_config["learning_rate"] = 0.01  # Example modification

# Create the RnnController instance
controller = RnnController(task, controller_config)

NameError: name 'config' is not defined